### Quick timer check
Call this before/after long blocks to ensure we stay within the 12h budget.


In [ ]:
runtime_guard(start_time, budget_hours=12)


# PART 0: Configuration & Setup


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from pathlib import Path
from typing import Dict, Any, List, Tuple, Optional
from dataclasses import dataclass, field
import json
import pickle
import os
import random
from datetime import datetime
from tqdm.auto import tqdm

from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.feature_selection import mutual_info_classif, SelectKBest
from sklearn.linear_model import LogisticRegression, Lasso
from sklearn.ensemble import RandomForestClassifier
from sklearn.calibration import calibration_curve, CalibratedClassifierCV

from sklearn.metrics import (
    roc_auc_score, average_precision_score, roc_curve, precision_recall_curve,
    confusion_matrix, accuracy_score, precision_score, recall_score, f1_score,
    brier_score_loss, log_loss
)

import xgboost as xgb
import lightgbm as lgb

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, TensorDataset

import optuna
from optuna.samplers import TPESampler

from scipy import stats
from scipy.stats import bootstrap, ttest_rel, chi2_contingency
warnings.filterwarnings("ignore", category=RuntimeWarning)
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=DeprecationWarning)
optuna.logging.set_verbosity(optuna.logging.WARNING)

plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

In [ ]:
from dataclasses import dataclass, field
from pathlib import Path
from typing import Optional, Dict, Any, List, Tuple
import json
from datetime import datetime

@dataclass
class Config:
    RANDOM_STATE: int = 42

    BASE_DIR: Path = field(default_factory=lambda: Path("/kaggle/working"))
    RAW_DATA_PATH: Optional[Path] = None
    PROCESSED_DIR: Optional[Path] = None
    RESULTS_DIR: Optional[Path] = None
    MODELS_DIR: Optional[Path] = None
    FIGURES_DIR: Optional[Path] = None

    TRAIN_RATIO: float = 0.70
    VAL_RATIO: float = 0.10
    TEST_RATIO: float = 0.20
    MISSINGNESS_THRESHOLD: float = 0.40
    CORRELATION_THRESHOLD: float = 0.95
    MICE_MAX_ITER: int = 50
    MICE_TOL: float = 1e-3
    OUTLIER_THRESHOLD: float = 5.0

    TOP_VARIANCE_FEATURES: int = 10
    TOP_INTERACTION_FEATURES: int = 5

    CV_FOLDS: int = 5
    CV_SHUFFLE: bool = True

    OPTUNA_N_TRIALS: int = 150
    OPTUNA_TIMEOUT: Optional[int] = 7200

    EARLY_STOPPING_PATIENCE: int = 10
    MAX_EPOCHS: int = 100
    BATCH_SIZE: int = 64

    N_BOOTSTRAP: int = 1000
    CONFIDENCE_LEVEL: float = 0.95
    ALPHA: float = 0.05

    FL_ROUNDS: int = 50
    FL_LOCAL_EPOCHS: int = 5
    FL_CLIENT_FRACTION: float = 1.0
    FEDPROX_MU_RANGE: Tuple[float, float] = (0.001, 0.1)
    DITTO_LAMBDA_RANGE: Tuple[float, float] = (0.01, 1.0)

    CALIBRATION_BINS: int = 10
    CALIBRATION_METHOD: str = 'isotonic'

    FIG_DPI: int = 300
    FIG_FORMAT: List[str] = field(default_factory=lambda: ['png', 'pdf'])
    FIG_SIZE_SMALL: Tuple[int, int] = (8, 6)
    FIG_SIZE_LARGE: Tuple[int, int] = (12, 8)
    FIG_SIZE_WIDE: Tuple[int, int] = (14, 6)

    SAINT_PARAMS: Dict[str, Any] = field(default_factory=lambda: {
        'd_model': 256,
        'n_heads': 8,
        'n_layers': 2,
        'dropout': 0.42,
        'lr': 1.67e-05,
        'batch_size': 64
    })

    LGBM_PARAMS: Dict[str, Any] = field(default_factory=lambda: {
        'n_estimators': 500,
        'max_depth': 6,
        'learning_rate': 0.05,
        'colsample_bytree': 0.8,
        'subsample': 0.8,
        'random_state': 42,
        'verbose': -1
    })

    XGB_PARAMS: Dict[str, Any] = field(default_factory=lambda: {
        'n_estimators': 500,
        'max_depth': 6,
        'learning_rate': 0.05,
        'colsample_bytree': 0.8,
        'subsample': 0.8,
        'random_state': 42,
        'tree_method': 'hist',
        'eval_metric': 'logloss'
    })

    def __post_init__(self):
        self.PROCESSED_DIR = self.BASE_DIR / "processed_data_final"
        self.RESULTS_DIR = self.BASE_DIR / "results"
        self.MODELS_DIR = self.BASE_DIR / "models"
        self.FIGURES_DIR = self.BASE_DIR / "figures"

        for directory in [self.PROCESSED_DIR, self.RESULTS_DIR,
                          self.MODELS_DIR, self.FIGURES_DIR]:
            directory.mkdir(parents=True, exist_ok=True)

        possible_paths = [
            Path("/kaggle/input/parquet/mimic_iv_0_24h_features.parquet"),
            Path("/kaggle/input/mimic-iv-features/mimic_iv_0_24h_features.parquet"),
            Path("/kaggle/input/your-dataset-name/mimic_iv_0_24h_features.parquet"),
            Path("/home/achraf/Documents/pfe/data/mimic_iv_0_24h_features.parquet"),
            Path("/home/achraf/Documents/pfe/mimic_iv_0_24h_features.parquet"),
        ]

        for path in possible_paths:
            if path.exists():
                self.RAW_DATA_PATH = path
                break

    def save_config(self, filename: str = "experiment_config.json"):
        config_dict = {
            'timestamp': datetime.now().isoformat(),
            'random_state': self.RANDOM_STATE,
            'preprocessing': {
                'train_ratio': self.TRAIN_RATIO,
                'val_ratio': self.VAL_RATIO,
                'test_ratio': self.TEST_RATIO,
                'missingness_threshold': self.MISSINGNESS_THRESHOLD,
                'correlation_threshold': self.CORRELATION_THRESHOLD,
                'mice_max_iter': self.MICE_MAX_ITER,
                'outlier_threshold': self.OUTLIER_THRESHOLD
            },
            'cross_validation': {
                'cv_folds': self.CV_FOLDS,
                'cv_shuffle': self.CV_SHUFFLE
            },
            'federated_learning': {
                'fl_rounds': self.FL_ROUNDS,
                'fl_local_epochs': self.FL_LOCAL_EPOCHS
            }
        }

        filepath = self.RESULTS_DIR / filename
        with open(filepath, 'w') as f:
            json.dump(config_dict, f, indent=2)

config = Config()
config.save_config()

In [ ]:
def set_all_seeds(seed: int = 42) -> None:
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

def worker_init_fn(worker_id: int) -> None:
    worker_seed = config.RANDOM_STATE + worker_id
    np.random.seed(worker_seed)
    random.seed(worker_seed)

set_all_seeds(config.RANDOM_STATE)

## Utility Functions

In [ ]:
def compute_metrics(
    y_true: np.ndarray,
    y_pred: np.ndarray,
    y_proba: np.ndarray,
    prefix: str = ""
) -> Dict[str, float]:
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()

    metrics = {
        f"{prefix}auroc": roc_auc_score(y_true, y_proba),
        f"{prefix}auprc": average_precision_score(y_true, y_proba),
        f"{prefix}accuracy": accuracy_score(y_true, y_pred),
        f"{prefix}precision": precision_score(y_true, y_pred, zero_division=0),
        f"{prefix}recall": recall_score(y_true, y_pred, zero_division=0),
        f"{prefix}f1": f1_score(y_true, y_pred, zero_division=0),
        f"{prefix}specificity": tn / (tn + fp) if (tn + fp) > 0 else 0,
        f"{prefix}brier": brier_score_loss(y_true, y_proba),
        f"{prefix}log_loss": log_loss(y_true, y_proba),
        f"{prefix}tp": int(tp),
        f"{prefix}tn": int(tn),
        f"{prefix}fp": int(fp),
        f"{prefix}fn": int(fn)
    }

    return metrics

def bootstrap_metric(
    y_true: np.ndarray,
    y_pred: np.ndarray,
    metric_fn: callable,
    n_bootstraps: int = 1000,
    confidence_level: float = 0.95,
    random_state: int = 42
) -> Tuple[float, float, float]:
    rng = np.random.RandomState(random_state)
    n_samples = len(y_true)

    scores = []
    for _ in range(n_bootstraps):
        indices = rng.choice(n_samples, n_samples, replace=True)
        try:
            score = metric_fn(y_true[indices], y_pred[indices])
            scores.append(score)
        except:
            continue

    scores = np.array(scores)
    point_estimate = metric_fn(y_true, y_pred)

    alpha = 1 - confidence_level
    lower_percentile = (alpha / 2) * 100
    upper_percentile = (1 - alpha / 2) * 100

    lower = np.percentile(scores, lower_percentile)
    upper = np.percentile(scores, upper_percentile)

    return point_estimate, lower, upper

def compute_metrics_with_ci(
    y_true: np.ndarray,
    y_pred: np.ndarray,
    y_proba: np.ndarray,
    n_bootstraps: int = 1000
) -> Dict[str, Tuple[float, float, float]]:
    metrics_with_ci = {}

    metrics_with_ci['auroc'] = bootstrap_metric(
        y_true, y_proba, roc_auc_score, n_bootstraps
    )

    metrics_with_ci['auprc'] = bootstrap_metric(
        y_true, y_proba, average_precision_score, n_bootstraps
    )

    metrics_with_ci['accuracy'] = bootstrap_metric(
        y_true, y_pred, accuracy_score, n_bootstraps
    )

    metrics_with_ci['f1'] = bootstrap_metric(
        y_true, y_pred, f1_score, n_bootstraps
    )

    metrics_with_ci['brier'] = bootstrap_metric(
        y_true, y_proba, brier_score_loss, n_bootstraps
    )

    return metrics_with_ci

def delong_roc_variance(y_true: np.ndarray, y_score: np.ndarray) -> Tuple[float, np.ndarray]:
    y_true = np.asarray(y_true).ravel()
    y_score = np.asarray(y_score).ravel()
    
    pos_idx = np.where(y_true == 1)[0]
    neg_idx = np.where(y_true == 0)[0]
    
    n_pos = len(pos_idx)
    n_neg = len(neg_idx)
    
    if n_pos == 0 or n_neg == 0:
        return 0.5, np.array([0.0])
    
    pos_scores = y_score[pos_idx]
    neg_scores = y_score[neg_idx]
    
    
    V10 = np.zeros(n_pos)
    for i, ps in enumerate(pos_scores):
        V10[i] = np.mean(ps > neg_scores) + 0.5 * np.mean(ps == neg_scores)
    
    V01 = np.zeros(n_neg)
    for j, ns in enumerate(neg_scores):
        V01[j] = np.mean(pos_scores > ns) + 0.5 * np.mean(pos_scores == ns)
    
    auc = np.mean(V10)
    
    S10 = np.var(V10, ddof=1) if n_pos > 1 else 0.0
    S01 = np.var(V01, ddof=1) if n_neg > 1 else 0.0
    
    var_auc = S10 / n_pos + S01 / n_neg
    
    return auc, np.sqrt(var_auc) if var_auc > 0 else 0.0


def delong_test(
    y_true: np.ndarray,
    y_pred1: np.ndarray,
    y_pred2: np.ndarray
) -> Tuple[float, float]:
    from scipy import stats
    
    y_true = np.asarray(y_true).ravel()
    y_pred1 = np.asarray(y_pred1).ravel()
    y_pred2 = np.asarray(y_pred2).ravel()
    
    pos_idx = np.where(y_true == 1)[0]
    neg_idx = np.where(y_true == 0)[0]
    
    n_pos = len(pos_idx)
    n_neg = len(neg_idx)
    
    if n_pos < 2 or n_neg < 2:
        return 0.0, 1.0
    
    # Compute placement values for both predictions
    def compute_placements(y_score):
        pos_scores = y_score[pos_idx]
        neg_scores = y_score[neg_idx]
        
        V10 = np.array([
            np.mean(ps > neg_scores) + 0.5 * np.mean(ps == neg_scores)
            for ps in pos_scores
        ])
        V01 = np.array([
            np.mean(pos_scores > ns) + 0.5 * np.mean(pos_scores == ns)
            for ns in neg_scores
        ])
        return V10, V01
    
    V10_1, V01_1 = compute_placements(y_pred1)
    V10_2, V01_2 = compute_placements(y_pred2)
    
    auc1 = np.mean(V10_1)
    auc2 = np.mean(V10_2)
    
    D10 = V10_1 - V10_2
    D01 = V01_1 - V01_2
    
    S10 = np.var(D10, ddof=1)
    S01 = np.var(D01, ddof=1)
    
    var_diff = S10 / n_pos + S01 / n_neg
    
    if var_diff <= 0:
        return 0.0, 1.0
    
    z_score = (auc1 - auc2) / np.sqrt(var_diff)
    p_value = 2 * (1 - stats.norm.cdf(np.abs(z_score)))
    
    return float(z_score), float(p_value)
def save_figure(
    fig: plt.Figure,
    filename: str,
    tight: bool = True
) -> None:
    if tight:
        fig.tight_layout()

    for fmt in config.FIG_FORMAT:
        filepath = config.FIGURES_DIR / f"{filename}.{fmt}"
        fig.savefig(filepath, dpi=config.FIG_DPI, bbox_inches='tight')

    plt.close(fig)

def print_metrics_with_ci(
    metrics: Dict[str, Tuple[float, float, float]],
    title: str = "Metrics"
) -> None:
    print(f"\n{title}:")
    for metric_name, (point, lower, upper) in metrics.items():
        print(f"{metric_name:15s}: {point:.4f} (95% CI: {lower:.4f}-{upper:.4f})")

In [ ]:
import numpy as np
from typing import Dict, Tuple, Optional
from sklearn.metrics import roc_auc_score, brier_score_loss


def compute_calibration_metrics(
    y_true: np.ndarray,
    y_prob: np.ndarray,
    n_bins: int = 10,
    strategy: str = 'uniform'
) -> Dict[str, float]:
    y_true = np.asarray(y_true).ravel()
    y_prob = np.asarray(y_prob).ravel()
    
    if strategy == 'uniform':
        bins = np.linspace(0.0, 1.0, n_bins + 1)
    elif strategy == 'quantile':
        bins = np.percentile(y_prob, np.linspace(0, 100, n_bins + 1))
        bins[0] = 0.0
        bins[-1] = 1.0
    else:
        raise ValueError(f"Unknown strategy: {strategy}")
    
    bin_indices = np.digitize(y_prob, bins[1:-1])
    
    ece = 0.0
    mce = 0.0
    ace = 0.0
    
    bin_stats = []
    n_samples = len(y_true)
    n_nonempty_bins = 0
    
    for i in range(n_bins):
        mask = bin_indices == i
        bin_size = mask.sum()
        
        if bin_size > 0:
            bin_acc = y_true[mask].mean()
            bin_conf = y_prob[mask].mean()
            bin_error = np.abs(bin_acc - bin_conf)
            
            ece += (bin_size / n_samples) * bin_error
            mce = max(mce, bin_error)
            ace += bin_error
            n_nonempty_bins += 1
            
            bin_stats.append({
                'bin_idx': i,
                'bin_lower': bins[i],
                'bin_upper': bins[i + 1],
                'count': int(bin_size),
                'accuracy': float(bin_acc),
                'confidence': float(bin_conf),
                'error': float(bin_error)
            })
    
    ace = ace / max(n_nonempty_bins, 1)
    
    return {
        'ece': float(ece),
        'mce': float(mce),
        'ace': float(ace),
        'n_bins': n_bins,
        'n_nonempty_bins': n_nonempty_bins,
        'bin_stats': bin_stats
    }


def compute_full_metrics(
    y_true: np.ndarray,
    y_prob: np.ndarray,
    n_bins: int = 10
) -> Dict[str, float]:
    y_true = np.asarray(y_true).ravel()
    y_prob = np.asarray(y_prob).ravel()
    
    if len(np.unique(y_true)) < 2:
        return {
            'auroc': 0.5,
            'brier': 0.25,
            'ece': 0.0,
            'mce': 0.0,
            'ace': 0.0
        }
    
    cal_metrics = compute_calibration_metrics(y_true, y_prob, n_bins)
    
    return {
        'auroc': float(roc_auc_score(y_true, y_prob)),
        'brier': float(brier_score_loss(y_true, y_prob)),
        'ece': cal_metrics['ece'],
        'mce': cal_metrics['mce'],
        'ace': cal_metrics['ace']
    }



In [ ]:
def get_optimal_threshold(y_true, y_probs):
    from sklearn.metrics import precision_recall_curve
    precision, recall, thresholds = precision_recall_curve(y_true, y_probs)
    f1_scores = 2 * (precision * recall) / (precision + recall + 1e-10)
    ix = np.argmax(f1_scores)
    return thresholds[ix], f1_scores[ix]

# PART 1: Data Preprocessing

In [ ]:
class BestPracticeFeatureSelector:
    def __init__(
        self,
        missingness_threshold: float = 0.50,
        correlation_threshold: float = 0.98,
        random_state: int = 42
    ):
        self.missingness_threshold = missingness_threshold
        self.correlation_threshold = correlation_threshold
        self.random_state = random_state

        self.selected_features = None
        self.feature_importance = None
        self.selection_info = {}

    def fit(self, X_train: pd.DataFrame, y_train: np.ndarray) -> 'BestPracticeFeatureSelector':
        missing_pct = X_train.isnull().sum() / len(X_train)
        features_step1 = missing_pct[missing_pct <= self.missingness_threshold].index.tolist()
        self.selection_info['after_missingness'] = features_step1

        X_step1 = X_train[features_step1].copy()

        X_for_corr = X_step1.fillna(X_step1.median())
        corr_matrix = X_for_corr.corr().abs()

        upper_tri = corr_matrix.where(
            np.triu(np.ones(corr_matrix.shape), k=1).astype(bool)
        )

        to_drop = [col for col in upper_tri.columns
                   if any(upper_tri[col] > self.correlation_threshold)]

        features_step2 = [f for f in features_step1 if f not in to_drop]
        self.selection_info['after_correlation'] = features_step2

        X_step2 = X_train[features_step2].copy()

        X_for_mi = X_step2.copy()
        for col in X_for_mi.columns:
            if X_for_mi[col].isnull().any():
                X_for_mi[col] = X_for_mi[col].fillna(X_for_mi[col].median())

        mi_scores = mutual_info_classif(
            X_for_mi, y_train,
            random_state=self.random_state
        )
        mi_threshold = np.percentile(mi_scores, 10)
        features_step3 = X_step2.columns[mi_scores > mi_threshold].tolist()
        self.selection_info['after_mi'] = features_step3

        X_step3 = X_train[features_step3].copy()

        X_for_l1 = X_step3.copy()
        for col in X_for_l1.columns:
            if X_for_l1[col].isnull().any():
                X_for_l1[col] = X_for_l1[col].fillna(X_for_l1[col].median())

        scaler_temp = StandardScaler()
        X_scaled = scaler_temp.fit_transform(X_for_l1)

        lasso = Lasso(alpha=0.001, random_state=self.random_state, max_iter=5000)
        lasso.fit(X_scaled, y_train)

        features_step4 = X_step3.columns[np.abs(lasso.coef_) > 0].tolist()
        self.selection_info['after_l1'] = features_step4

        X_for_lgb = X_train[features_step4].copy()
        for col in X_for_lgb.columns:
            if X_for_lgb[col].isnull().any():
                X_for_lgb[col] = X_for_lgb[col].fillna(X_for_lgb[col].median())

        lgb_model = lgb.LGBMClassifier(
            n_estimators=500,
            random_state=self.random_state,
            verbose=-1
        )
        lgb_model.fit(X_for_lgb, y_train)

        importance_scores = lgb_model.feature_importances_
        importance_threshold = np.percentile(importance_scores, 5)
        features_step5 = X_for_lgb.columns[importance_scores > importance_threshold].tolist()

        self.selected_features = features_step5
        self.selection_info['final'] = features_step5

        self.feature_importance = pd.DataFrame({
            'feature': features_step5,
            'importance': importance_scores[X_for_lgb.columns.isin(features_step5)]
        }).sort_values('importance', ascending=False)

        return self

    def transform(self, X: pd.DataFrame) -> pd.DataFrame:
        if self.selected_features is None:
            raise ValueError("FeatureSelector has not been fitted yet.")

        return X[self.selected_features].copy()

    def fit_transform(self, X: pd.DataFrame, y: np.ndarray) -> pd.DataFrame:
        return self.fit(X, y).transform(X)

In [ ]:
class BestPracticeImputer:
    def __init__(
        self,
        max_iter: int = 50,
        tol: float = 1e-3,
        random_state: int = 42
    ):
        self.max_iter = max_iter
        self.tol = tol
        self.random_state = random_state

        self.imputer = IterativeImputer(
            max_iter=max_iter,
            tol=tol,
            random_state=random_state,
            verbose=0
        )

        self.feature_names = None
        self.convergence_info = {}

    def fit(self, X_train: pd.DataFrame) -> 'BestPracticeImputer':
        self.feature_names = X_train.columns.tolist()

        self.imputer.fit(X_train)

        if hasattr(self.imputer, 'n_iter_'):
            actual_iters = self.imputer.n_iter_
            self.convergence_info['iterations'] = actual_iters

            if actual_iters >= self.max_iter:
                self.convergence_info['converged'] = False
            else:
                self.convergence_info['converged'] = True

        return self

    def transform(self, X: pd.DataFrame) -> pd.DataFrame:
        if self.feature_names is None:
            raise ValueError("Imputer has not been fitted yet.")

        X_imputed = self.imputer.transform(X)
        X_imputed = pd.DataFrame(X_imputed, columns=self.feature_names, index=X.index)

        return X_imputed

    def fit_transform(self, X: pd.DataFrame) -> pd.DataFrame:
        return self.fit(X).transform(X)

In [ ]:
class OutlierHandler:
    def __init__(self, iqr_multiplier: float = 5.0):
        self.iqr_multiplier = iqr_multiplier
        self.lower_bounds = {}
        self.upper_bounds = {}
        self.outlier_info = {}

    def fit(self, X_train: pd.DataFrame) -> 'OutlierHandler':
        for col in X_train.columns:
            if X_train[col].dtype in [np.float64, np.float32, np.int64, np.int32]:
                Q1 = X_train[col].quantile(0.25)
                Q3 = X_train[col].quantile(0.75)
                IQR = Q3 - Q1

                self.lower_bounds[col] = Q1 - self.iqr_multiplier * IQR
                self.upper_bounds[col] = Q3 + self.iqr_multiplier * IQR

        return self

    def transform(self, X: pd.DataFrame) -> pd.DataFrame:
        X_clipped = X.copy()

        for col in X_clipped.columns:
            if col in self.lower_bounds:
                lower = self.lower_bounds[col]
                upper = self.upper_bounds[col]
                X_clipped[col] = X_clipped[col].clip(lower, upper)

        return X_clipped

    def fit_transform(self, X: pd.DataFrame) -> pd.DataFrame:
        return self.fit(X).transform(X)

In [ ]:
class BestPracticeScaler:
    def __init__(self, method: str = 'standard'):
        self.method = method

        if method == 'standard':
            self.scaler = StandardScaler()
        elif method == 'robust':
            self.scaler = RobustScaler()
        else:
            raise ValueError(f"Unknown scaling method: {method}")

        self.feature_names = None

    def fit(self, X_train: pd.DataFrame) -> 'BestPracticeScaler':
        self.feature_names = X_train.columns.tolist()
        self.scaler.fit(X_train)
        return self

    def transform(self, X: pd.DataFrame) -> pd.DataFrame:
        if self.feature_names is None:
            raise ValueError("Scaler has not been fitted yet.")

        X_scaled = self.scaler.transform(X)
        X_scaled = pd.DataFrame(X_scaled, columns=self.feature_names, index=X.index)

        return X_scaled

    def fit_transform(self, X: pd.DataFrame) -> pd.DataFrame:
        return self.fit(X).transform(X)

## Load and Split Data

In [ ]:
if config.RAW_DATA_PATH is None or not config.RAW_DATA_PATH.exists():
    print("ERROR: Data file not found!")
    print("Please update config.RAW_DATA_PATH to point to your data file.")
    print("Expected location: MIMIC-IV ICU data in Parquet or CSV format")
    print("You can set it manually:")
    print('config.RAW_DATA_PATH = Path("/path/to/your/data.parquet")')
    raise FileNotFoundError(f"Data file not found at {config.RAW_DATA_PATH}")

if config.RAW_DATA_PATH.suffix == '.parquet':
    df_raw = pd.read_parquet(config.RAW_DATA_PATH)
elif config.RAW_DATA_PATH.suffix == '.csv':
    df_raw = pd.read_csv(config.RAW_DATA_PATH)
else:
    raise ValueError(f"Unsupported file format: {config.RAW_DATA_PATH.suffix}")

if 'label' in df_raw.columns:
    mortality_rate = df_raw['label'].mean()

df_raw.head()

In [ ]:
TARGET_COL = 'label'
ID_COLS = ['subject_id', 'hadm_id', 'stay_id', 'icustay_id']

id_cols_present = [col for col in ID_COLS if col in df_raw.columns]

exclude_cols = [TARGET_COL] + id_cols_present
feature_cols = [col for col in df_raw.columns if col not in exclude_cols]

print(f"Target column: {TARGET_COL}")
print(f"ID columns: {id_cols_present}")
print(f"Feature columns: {len(feature_cols)}")

X = df_raw[feature_cols].copy()
y = df_raw[TARGET_COL].values

if id_cols_present:
    ids_df = df_raw[id_cols_present].copy()
else:
    ids_df = pd.DataFrame(index=df_raw.index)

print(f"\nFeature matrix: {X.shape}")
print(f"Target vector: {y.shape}")
print(f"Class distribution: {np.bincount(y.astype(int))}")

In [ ]:
X_temp, X_test, y_temp, y_test, ids_temp, ids_test = train_test_split(
    X, y, ids_df,
    test_size=config.TEST_RATIO,
    random_state=config.RANDOM_STATE,
    stratify=y
)

val_size_adjusted = config.VAL_RATIO / (config.TRAIN_RATIO + config.VAL_RATIO)
X_train, X_val, y_train, y_val, ids_train, ids_val = train_test_split(
    X_temp, y_temp, ids_temp,
    test_size=val_size_adjusted,
    random_state=config.RANDOM_STATE,
    stratify=y_temp
)

print(f"Train set: {X_train.shape[0]:,} samples ({y_train.mean():.2%} mortality)")
print(f"Val set:   {X_val.shape[0]:,} samples ({y_val.mean():.2%} mortality)")
print(f"Test set:  {X_test.shape[0]:,} samples ({y_test.mean():.2%} mortality)")

del X_temp, y_temp, ids_temp

In [ ]:
print("EXTRACTING ICU TYPES FOR FL PARTITIONING")

icu_column_candidates = [
    'first_careunit',
    'last_careunit',
    'careunit',
    'icu_type',
    'care_unit',
    'unit_name'
]

icu_column = None
for col in icu_column_candidates:
    if col in df_raw.columns:
        icu_column = col
        print(f"Found ICU column: '{icu_column}'")
        break

if icu_column is None:
    icu_cols = [col for col in df_raw.columns
                if 'icu' in col.lower() or 'care' in col.lower() or 'unit' in col.lower()]
    if icu_cols:
        icu_column = icu_cols[0]
        print(f"Found ICU-related column: '{icu_column}'")
    else:
        raise ValueError("No ICU/careunit column found in raw data!")

print(f"Unique ICU types in raw data ({df_raw[icu_column].nunique()}):")
icu_value_counts = df_raw[icu_column].value_counts()
for val, count in icu_value_counts.items():
    pct = count / len(df_raw) * 100
    print(f"{str(val):50s}: {count:6d} ({pct:5.1f}%)")

def group_icu_types(careunit: str) -> str:
    cu = str(careunit).lower()
    if 'micu' in cu or 'medical' in cu:
        return 'MICU'
    elif 'sicu' in cu or 'surgical' in cu or 'trauma' in cu:
        return 'SICU'
    elif 'ccu' in cu and 'cvicu' not in cu:
        return 'CCU'
    elif 'cvicu' in cu or 'cardiac vascular' in cu:
        return 'CVICU'
    elif 'neuro' in cu or 'nsicu' in cu:
        return 'Neuro'
    else:
        return 'Mixed'

train_indices = ids_train.index
val_indices = ids_val.index
test_indices = ids_test.index

icu_train_raw = df_raw.loc[train_indices, icu_column]
icu_val_raw = df_raw.loc[val_indices, icu_column]
icu_test_raw = df_raw.loc[test_indices, icu_column]

icu_types_train = icu_train_raw.apply(group_icu_types).values
icu_types_val = icu_val_raw.apply(group_icu_types).values
icu_types_test = icu_test_raw.apply(group_icu_types).values

print(f"Extracted {len(icu_types_train)} train ICU types")
print(f"Extracted {len(icu_types_val)} val ICU types")
print(f"Extracted {len(icu_types_test)} test ICU types")

print("TRAIN SET:")
train_dist = pd.Series(icu_types_train).value_counts()
for icu_type, count in train_dist.items():
    pct = count / len(icu_types_train) * 100
    mortality = y_train[icu_types_train == icu_type].mean() * 100
    print(f"{icu_type:8s}: {count:6d} ({pct:5.1f}%) | Mortality: {mortality:5.1f}%")

print("VAL SET:")
val_dist = pd.Series(icu_types_val).value_counts()
for icu_type, count in val_dist.items():
    pct = count / len(icu_types_val) * 100
    mortality = y_val[icu_types_val == icu_type].mean() * 100
    print(f"{icu_type:8s}: {count:6d} ({pct:5.1f}%) | Mortality: {mortality:5.1f}%")

print("TEST SET:")
test_dist = pd.Series(icu_types_test).value_counts()
for icu_type, count in test_dist.items():
    pct = count / len(icu_types_test) * 100
    mortality = y_test[icu_types_test == icu_type].mean() * 100
    print(f"{icu_type:8s}: {count:6d} ({pct:5.1f}%) | Mortality: {mortality:5.1f}%")

import numpy as np
from pathlib import Path

np.save(config.PROCESSED_DIR / "icu_types_train.npy", icu_types_train)
np.save(config.PROCESSED_DIR / "icu_types_val.npy", icu_types_val)
np.save(config.PROCESSED_DIR / "icu_types_test.npy", icu_types_test)

icu_mortality_rates = {}
for icu_type in train_dist.index:
    mortality_rate = y_train[icu_types_train == icu_type].mean()
    icu_mortality_rates[icu_type] = mortality_rate

mortality_std = np.std(list(icu_mortality_rates.values()))
print(f"Mortality rate std dev across ICU types: {mortality_std:.4f}")
print(f"Total ICU types: {len(train_dist)}")

In [ ]:
X_train = X_train.select_dtypes(include=[np.number]).copy()
X_val = X_val.select_dtypes(include=[np.number]).copy()
X_test = X_test.select_dtypes(include=[np.number]).copy()

feature_selector = BestPracticeFeatureSelector(
    missingness_threshold=config.MISSINGNESS_THRESHOLD,
    correlation_threshold=config.CORRELATION_THRESHOLD,
    random_state=config.RANDOM_STATE
)

X_train_selected = feature_selector.fit_transform(X_train, y_train)
X_val_selected = feature_selector.transform(X_val)
X_test_selected = feature_selector.transform(X_test)

print(f"Features after selection:")
print(f"  Train: {X_train_selected.shape}")
print(f"  Val:   {X_val_selected.shape}")
print(f"  Test:  {X_test_selected.shape}")

In [ ]:
outlier_handler = OutlierHandler(iqr_multiplier=config.OUTLIER_THRESHOLD)

X_train_no_outliers = outlier_handler.fit_transform(X_train_selected)
X_val_no_outliers = outlier_handler.transform(X_val_selected)
X_test_no_outliers = outlier_handler.transform(X_test_selected)

In [ ]:
class ClinicalFeatureEngineer:
    def __init__(
        self,
        n_top_features: int = 10,
        n_interactions: int = 10
    ):
        self.n_top_features = n_top_features
        self.n_interactions = n_interactions
        self.lab_quantiles = {}
        self.vital_quantiles = {}
        self.top_variance_features = None
        self.interaction_pairs = []
        self.organ_thresholds = {}
        self.vital_thresholds = {}
        self.lab_keywords = [
            'wbc', 'hgb', 'hemoglobin', 'hematocrit', 'platelet', 'plt',
            'sodium', 'na', 'potassium', 'k', 'chloride', 'cl',
            'bicarbonate', 'hco3', 'bun', 'creatinine', 'cr', 'glucose',
            'calcium', 'ca', 'magnesium', 'mg', 'phosphate', 'lactate',
            'troponin', 'bnp', 'procalcitonin', 'crp', 'alt', 'ast',
            'bilirubin', 'albumin', 'inr', 'pt', 'ptt', 'ph', 'pao2',
            'pco2', 'fio2', 'spo2'
        ]
        self.vital_keywords = [
            'heart_rate', 'hr', 'pulse', 'sbp', 'dbp', 'mbp', 'map',
            'respiratory_rate', 'rr', 'resprate', 'temperature', 'temp',
            'spo2', 'o2sat'
        ]
        self.feature_info = {
            'original_features': [],
            'engineered_features': [],
            'n_features_added': 0
        }

    def _find_features(
        self,
        keywords: List[str],
        feature_names: List[str]
    ) -> List[str]:
        matched_features = []
        for feat in feature_names:
            feat_lower = feat.lower()
            for keyword in keywords:
                if keyword in feat_lower:
                    matched_features.append(feat)
                    break
        return matched_features

    def _find_column(self, X: pd.DataFrame, keywords: List[str]) -> Optional[str]:
        for col in X.columns:
            col_lower = col.lower()
            for keyword in keywords:
                if keyword in col_lower:
                    return col
        return None

    def fit(self, X_train: pd.DataFrame) -> 'ClinicalFeatureEngineer':
        self.feature_info['original_features'] = X_train.columns.tolist()
        lab_features = self._find_features(self.lab_keywords, X_train.columns.tolist())
        vital_features = self._find_features(self.vital_keywords, X_train.columns.tolist())
        key_labs = ['creatinine', 'bun', 'glucose', 'lactate', 'wbc',
                    'hemoglobin', 'platelet', 'sodium', 'potassium', 'bilirubin']
        for lab_keyword in key_labs:
            col = self._find_column(X_train, [lab_keyword])
            if col:
                self.lab_quantiles[col] = {
                    'q01': X_train[col].quantile(0.01),
                    'q05': X_train[col].quantile(0.05),
                    'q25': X_train[col].quantile(0.25),
                    'q75': X_train[col].quantile(0.75),
                    'q95': X_train[col].quantile(0.95),
                    'q99': X_train[col].quantile(0.99),
                    'median': X_train[col].median()
                }
        for col in vital_features:
            if col in X_train.columns:
                self.vital_quantiles[col] = {
                    'q05': X_train[col].quantile(0.05),
                    'q95': X_train[col].quantile(0.95),
                    'median': X_train[col].median()
                }
        self.organ_thresholds['liver'] = {
            'alt': 80,
            'ast': 80,
            'bilirubin': 2.0
        }
        cr_col = self._find_column(X_train, ['creatinine', 'cr'])
        if cr_col:
            self.organ_thresholds['renal'] = {
                'creatinine': 2.0
            }
        self.organ_thresholds['coagulation'] = {
            'inr': 1.5,
            'platelet': 100
        }
        self.vital_thresholds = {
            'sbp_low': 90,
            'hr_high': 100,
            'rr_high': 20,
            'temp_high': 38.0,
            'temp_low': 36.0
        }
        feature_variances = X_train.var().sort_values(ascending=False)
        self.top_variance_features = feature_variances.head(self.n_top_features).index.tolist()
        if len(self.top_variance_features) >= 2:
            for i in range(min(self.n_interactions, len(self.top_variance_features) - 1)):
                self.interaction_pairs.append(
                    (self.top_variance_features[i], self.top_variance_features[i + 1])
                )
        return self

    def transform(self, X: pd.DataFrame) -> pd.DataFrame:
        if self.top_variance_features is None:
            raise ValueError("FeatureEngineer has not been fitted yet.")
        X_new = X.copy()
        engineered_features = []
        bun_col = self._find_column(X, ['bun'])
        cr_col = self._find_column(X, ['creatinine', 'cr'])
        if bun_col and cr_col:
            X_new['ratio_bun_cr'] = X[bun_col] / (X[cr_col] + 1e-6)
            X_new['ratio_cr_bun'] = X[cr_col] / (X[bun_col] + 1e-6)
            engineered_features.extend(['ratio_bun_cr', 'ratio_cr_bun'])
        pao2_col = self._find_column(X, ['pao2', 'po2'])
        fio2_col = self._find_column(X, ['fio2'])
        if pao2_col and fio2_col:
            X_new['ratio_pf'] = X[pao2_col] / (X[fio2_col] + 0.01)
            engineered_features.append('ratio_pf')
        pco2_col = self._find_column(X, ['pco2'])
        ph_col = self._find_column(X, ['ph'])
        if pco2_col and ph_col:
            X_new['ratio_pco2_ph'] = X[pco2_col] / (X[ph_col] + 1e-6)
            engineered_features.append('ratio_pco2_ph')
        lactate_col = self._find_column(X, ['lactate'])
        albumin_col = self._find_column(X, ['albumin'])
        if lactate_col and albumin_col:
            X_new['ratio_lactate_albumin'] = X[lactate_col] / (X[albumin_col] + 1e-6)
            engineered_features.append('ratio_lactate_albumin')
        glucose_col = self._find_column(X, ['glucose'])
        if glucose_col and albumin_col:
            X_new['ratio_glucose_albumin'] = X[glucose_col] / (X[albumin_col] + 1e-6)
            engineered_features.append('ratio_glucose_albumin')
        hgb_col = self._find_column(X, ['hemoglobin', 'hgb'])
        wbc_col = self._find_column(X, ['wbc'])
        if hgb_col and wbc_col:
            X_new['ratio_hgb_wbc'] = X[hgb_col] / (X[wbc_col] + 1e-6)
            engineered_features.append('ratio_hgb_wbc')
        plt_col = self._find_column(X, ['platelet', 'plt'])
        if plt_col and wbc_col:
            X_new['ratio_plt_wbc'] = X[plt_col] / (X[wbc_col] + 1e-6)
            engineered_features.append('ratio_plt_wbc')
        alt_col = self._find_column(X, ['alt'])
        ast_col = self._find_column(X, ['ast'])
        bili_col = self._find_column(X, ['bilirubin', 'bili'])
        liver_dysfn = 0
        if alt_col:
            liver_dysfn += (X[alt_col] > self.organ_thresholds['liver']['alt']).astype(int)
        if ast_col:
            liver_dysfn += (X[ast_col] > self.organ_thresholds['liver']['ast']).astype(int)
        if bili_col:
            liver_dysfn += (X[bili_col] > self.organ_thresholds['liver']['bilirubin']).astype(int)
        X_new['liver_dysfunction'] = (liver_dysfn >= 2).astype(int)
        X_new['liver_dysfunction_count'] = liver_dysfn
        engineered_features.extend(['liver_dysfunction', 'liver_dysfunction_count'])
        if cr_col:
            X_new['renal_dysfunction'] = (X[cr_col] > self.organ_thresholds['renal']['creatinine']).astype(int)
            engineered_features.append('renal_dysfunction')
        inr_col = self._find_column(X, ['inr'])
        coag_dysfn = 0
        if inr_col:
            coag_dysfn += (X[inr_col] > self.organ_thresholds['coagulation']['inr']).astype(int)
        if plt_col:
            coag_dysfn += (X[plt_col] < self.organ_thresholds['coagulation']['platelet']).astype(int)
        X_new['coagulopathy'] = (coag_dysfn >= 1).astype(int)
        engineered_features.append('coagulopathy')
        sbp_col = self._find_column(X, ['sbp', 'sys'])
        if sbp_col:
            X_new['cardiovascular_dysfunction'] = (X[sbp_col] < self.vital_thresholds['sbp_low']).astype(int)
            engineered_features.append('cardiovascular_dysfunction')
        if pao2_col and fio2_col:
            pf_ratio = X[pao2_col] / (X[fio2_col] + 0.01)
            X_new['respiratory_dysfunction'] = (pf_ratio < 300).astype(int)
            engineered_features.append('respiratory_dysfunction')
        organ_cols = [c for c in X_new.columns if 'dysfunction' in c and 'count' not in c]
        X_new['total_organ_dysfunction'] = X_new[organ_cols].sum(axis=1)
        engineered_features.append('total_organ_dysfunction')
        critical_count = 0
        severe_count = 0
        moderate_count = 0
        for lab_col, quantiles in self.lab_quantiles.items():
            if lab_col in X.columns:
                is_critical = ((X[lab_col] < quantiles['q01']) | (X[lab_col] > quantiles['q99'])).astype(int)
                critical_count += is_critical
                is_severe = ((X[lab_col] < quantiles['q05']) | (X[lab_col] > quantiles['q95'])).astype(int)
                is_severe = is_severe & (~is_critical.astype(bool))
                severe_count += is_severe.astype(int)
                is_moderate = ((X[lab_col] < quantiles['q25']) | (X[lab_col] > quantiles['q75'])).astype(int)
                is_moderate = is_moderate & (~is_severe.astype(bool)) & (~is_critical.astype(bool))
                moderate_count += is_moderate.astype(int)
        X_new['abnormal_labs_critical'] = critical_count
        X_new['abnormal_labs_severe'] = severe_count
        X_new['abnormal_labs_moderate'] = moderate_count
        engineered_features.extend(['abnormal_labs_critical', 'abnormal_labs_severe', 'abnormal_labs_moderate'])
        renal_labs = ['creatinine', 'bun']
        hepatic_labs = ['bilirubin', 'alt', 'ast']
        heme_labs = ['wbc', 'hemoglobin', 'platelet']
        for organ, labs_keywords in [('renal', renal_labs), ('hepatic', hepatic_labs), ('heme', heme_labs)]:
            count = 0
            for keyword in labs_keywords:
                col = self._find_column(X, [keyword])
                if col and col in self.lab_quantiles:
                    q = self.lab_quantiles[col]
                    count += ((X[col] < q['q05']) | (X[col] > q['q95'])).astype(int)
            X_new[f'abnormal_{organ}_count'] = count
            engineered_features.append(f'abnormal_{organ}_count')
        hr_col = self._find_column(X, ['heart_rate', 'hr', 'pulse'])
        rr_col = self._find_column(X, ['respiratory_rate', 'rr', 'resprate'])
        temp_col = self._find_column(X, ['temperature', 'temp'])
        if sbp_col:
            X_new['hypotensive'] = (X[sbp_col] < self.vital_thresholds['sbp_low']).astype(int)
            engineered_features.append('hypotensive')
        if hr_col:
            X_new['tachycardic'] = (X[hr_col] > self.vital_thresholds['hr_high']).astype(int)
            engineered_features.append('tachycardic')
        if rr_col:
            X_new['tachypneic'] = (X[rr_col] > self.vital_thresholds['rr_high']).astype(int)
            engineered_features.append('tachypneic')
        if temp_col:
            X_new['febrile'] = (X[temp_col] > self.vital_thresholds['temp_high']).astype(int)
            X_new['hypothermic'] = (X[temp_col] < self.vital_thresholds['temp_low']).astype(int)
            engineered_features.extend(['febrile', 'hypothermic'])
        if all(c in X_new.columns for c in ['hypotensive', 'tachycardic', 'tachypneic']):
            X_new['septic_pattern'] = (X_new['hypotensive'] & X_new['tachycardic'] & X_new['tachypneic']).astype(int)
            X_new['cardiogenic_pattern'] = (X_new['hypotensive'] & ~X_new['tachycardic']).astype(int)
            engineered_features.extend(['septic_pattern', 'cardiogenic_pattern'])
        vital_binary_cols = [c for c in X_new.columns if c in ['hypotensive', 'tachycardic', 'tachypneic', 'febrile', 'hypothermic']]
        if vital_binary_cols:
            X_new['vital_instability_score'] = X_new[vital_binary_cols].sum(axis=1)
            engineered_features.append('vital_instability_score')
        if cr_col and glucose_col:
            X_new['interact_cr_glucose'] = X[cr_col] * X[glucose_col]
            engineered_features.append('interact_cr_glucose')
        if bun_col and glucose_col:
            X_new['interact_bun_glucose'] = X[bun_col] * X[glucose_col]
            engineered_features.append('interact_bun_glucose')
        if sbp_col and hr_col:
            X_new['interact_sbp_hr'] = X[sbp_col] * X[hr_col]
            engineered_features.append('interact_sbp_hr')
        if wbc_col and temp_col:
            X_new['interact_wbc_temp'] = X[wbc_col] * X[temp_col]
            engineered_features.append('interact_wbc_temp')
        spo2_col = self._find_column(X, ['spo2', 'o2sat'])
        if hgb_col and spo2_col:
            X_new['interact_hgb_spo2'] = X[hgb_col] * X[spo2_col]
            engineered_features.append('interact_hgb_spo2')
        for feat1, feat2 in self.interaction_pairs[:5]:
            if feat1 in X.columns and feat2 in X.columns:
                interaction_name = f'interact_{feat1}_{feat2}'
                X_new[interaction_name] = X[feat1] * X[feat2]
                engineered_features.append(interaction_name)
        for feat in self.top_variance_features[:3]:
            if feat in X.columns:
                X_new[f'{feat}_squared'] = X[feat] ** 2
                engineered_features.append(f'{feat}_squared')
        self.feature_info['engineered_features'] = engineered_features
        self.feature_info['n_features_added'] = len(engineered_features)
        return X_new

    def fit_transform(self, X: pd.DataFrame) -> pd.DataFrame:
        return self.fit(X).transform(X)

In [ ]:
imputer = BestPracticeImputer(
    max_iter=config.MICE_MAX_ITER,
    tol=config.MICE_TOL,
    random_state=config.RANDOM_STATE
)

X_train_imputed = imputer.fit_transform(X_train_no_outliers)
X_val_imputed = imputer.transform(X_val_no_outliers)
X_test_imputed = imputer.transform(X_test_no_outliers)

In [ ]:
feature_engineer = ClinicalFeatureEngineer(
    n_top_features=config.TOP_VARIANCE_FEATURES,
    n_interactions=config.TOP_INTERACTION_FEATURES
)

X_train_scaled = feature_engineer.fit_transform(X_train_imputed)
X_val_scaled = feature_engineer.transform(X_val_imputed)
X_test_scaled = feature_engineer.transform(X_test_imputed)

print(f"\nFinal shapes after feature engineering:")
print(f"  Train: {X_train_scaled.shape}")
print(f"  Val:   {X_val_scaled.shape}")
print(f"  Test:  {X_test_scaled.shape}")

engineered_feature_names = feature_engineer.feature_info['engineered_features']
print(f"\nEngineered features ({len(engineered_feature_names)}):")
for feat in engineered_feature_names[:10]:
    print(f"  - {feat}")
if len(engineered_feature_names) > 10:
    print(f"  ... and {len(engineered_feature_names) - 10} more")

In [ ]:
scaler = BestPracticeScaler(method='standard')

X_train_scaled = scaler.fit_transform(X_train_scaled)
X_val_scaled = scaler.transform(X_val_scaled)
X_test_scaled = scaler.transform(X_test_scaled)

print(f"Train: {X_train_scaled.shape}")
print(f"Val:   {X_val_scaled.shape}")
print(f"Test:  {X_test_scaled.shape}")

## Save Preprocessed Data

In [ ]:
preprocessed_data = {
    'X_train': X_train_scaled,
    'X_val': X_val_scaled,
    'X_test': X_test_scaled,
    'y_train': y_train,
    'y_val': y_val,
    'y_test': y_test,
    'ids_train': ids_train,
    'ids_val': ids_val,
    'ids_test': ids_test,
    'feature_names': X_train_scaled.columns.tolist(),
    'icu_types_train': icu_types_train,
    'icu_types_val': icu_types_val,
    'icu_types_test': icu_types_test,
}

preprocessed_data_path = config.PROCESSED_DIR / "preprocessed_data.pkl"
with open(preprocessed_data_path, 'wb') as f:
    pickle.dump(preprocessed_data, f)

preprocessing_artifacts = {
    'scaler': scaler,
    'imputer': imputer,
    'selected_features': X_train_scaled.columns.tolist(),
    'icu_column_name': icu_column,
}

if 'selector' in locals():
    preprocessing_artifacts['feature_selector'] = selector

artifacts_path = config.PROCESSED_DIR / "preprocessing_artifacts.pkl"
with open(artifacts_path, 'wb') as f:
    pickle.dump(preprocessing_artifacts, f)

## Data Distribution Visualization

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=config.FIG_SIZE_LARGE)

ax = axes[0, 0]
splits = ['Train', 'Val', 'Test']
sizes = [len(y_train), len(y_val), len(y_test)]
colors = ['#3498db', '#e74c3c', '#2ecc71']
ax.bar(splits, sizes, color=colors, alpha=0.7)
ax.set_ylabel('Number of Samples')
ax.set_title('Dataset Split Sizes')
for i, (split, size) in enumerate(zip(splits, sizes)):
    ax.text(i, size + 500, f'{size:,}', ha='center', va='bottom', fontweight='bold')

ax = axes[0, 1]
mortality_rates = [y_train.mean(), y_val.mean(), y_test.mean()]
ax.bar(splits, mortality_rates, color=colors, alpha=0.7)
ax.set_ylabel('Mortality Rate')
ax.set_title('Mortality Rate by Split')
ax.set_ylim([0, max(mortality_rates) * 1.3])
for i, (split, rate) in enumerate(zip(splits, mortality_rates)):
    ax.text(i, rate + 0.005, f'{rate:.2%}', ha='center', va='bottom', fontweight='bold')

ax = axes[1, 0]
feature_idx = 0
feature_name = X_train_scaled.columns[feature_idx]
ax.hist(X_train_scaled.iloc[:, feature_idx], bins=50, alpha=0.5, label='Train', color=colors[0])
ax.hist(X_val_scaled.iloc[:, feature_idx], bins=50, alpha=0.5, label='Val', color=colors[1])
ax.hist(X_test_scaled.iloc[:, feature_idx], bins=50, alpha=0.5, label='Test', color=colors[2])
ax.set_xlabel('Scaled Value')
ax.set_ylabel('Frequency')
ax.set_title(f'Distribution of {feature_name}')
ax.legend()

ax = axes[1, 1]
missing_info = [
    ('Initial', X_train.isnull().sum().sum() / X_train.size),
    ('After Selection', X_train_selected.isnull().sum().sum() / X_train_selected.size),
    ('After Outlier', X_train_no_outliers.isnull().sum().sum() / X_train_no_outliers.size),
    ('After MICE', X_train_imputed.isnull().sum().sum() / X_train_imputed.size)
]
stages, missing_pcts = zip(*missing_info)
ax.plot(range(len(stages)), [m * 100 for m in missing_pcts], marker='o', linewidth=2, markersize=8)
ax.set_xticks(range(len(stages)))
ax.set_xticklabels(stages, rotation=45, ha='right')
ax.set_ylabel('Missing Value %')
ax.set_title('Missing Values Through Pipeline')
ax.grid(True, alpha=0.3)

plt.tight_layout()
save_figure(fig, 'preprocessing_overview')
plt.show()

# PART 2: Feature Engineering

## Ablation Study: Impact of Feature Engineering

In [ ]:
def quick_baseline_test(
    X_train: pd.DataFrame,
    y_train: np.ndarray,
    X_val: pd.DataFrame,
    y_val: np.ndarray,
    label: str
) -> Dict[str, float]:
    model = LogisticRegression(
        max_iter=1000,
        random_state=config.RANDOM_STATE,
        n_jobs=-1
    )

    model.fit(X_train, y_train)
    y_pred_proba = model.predict_proba(X_val)[:, 1]

    auroc = roc_auc_score(y_val, y_pred_proba)
    auprc = average_precision_score(y_val, y_pred_proba)

    return {
        'label': label,
        'n_features': X_train.shape[1],
        'auroc': auroc,
        'auprc': auprc
    }

ablation_results = []

engineered_prefixes = ['ratio_', 'organ_', 'abnormal_', 'pattern_', 'interact_', '_squared']
engineered_cols = [col for col in X_train_scaled.columns
                   if any(prefix in col for prefix in engineered_prefixes)]
original_cols = [col for col in X_train_scaled.columns if col not in engineered_cols]

print(f"\nFeature breakdown:")
print(f"   Original features: {len(original_cols)}")
print(f"   Engineered features: {len(engineered_cols)}")
print(f"   Total: {X_train_scaled.shape[1]}")

if len(original_cols) + len(engineered_cols) != X_train_scaled.shape[1]:
    print("\nWARNING: Feature separation may be incomplete!")

print(f"\n1. Testing: Original features only ({len(original_cols)} features)")
result = quick_baseline_test(
    X_train_scaled[original_cols], y_train,
    X_val_scaled[original_cols], y_val,
    'Original Features Only'
)
ablation_results.append(result)
print(f"   AUROC: {result['auroc']:.4f}, AUPRC: {result['auprc']:.4f}")

print(f"\n2. Testing: Original + Engineered features ({X_train_scaled.shape[1]} features)")
result = quick_baseline_test(
    X_train_scaled, y_train,
    X_val_scaled, y_val,
    'Original + Engineered'
)
ablation_results.append(result)
print(f"   AUROC: {result['auroc']:.4f}, AUPRC: {result['auprc']:.4f}")

ablation_df = pd.DataFrame(ablation_results)

print("\nAblation Study Results:")
print(ablation_df.to_string(index=False))

improvement_auroc = ablation_df.iloc[1]['auroc'] - ablation_df.iloc[0]['auroc']
improvement_auprc = ablation_df.iloc[1]['auprc'] - ablation_df.iloc[0]['auprc']
pct_improvement_auroc = (improvement_auroc / ablation_df.iloc[0]['auroc']) * 100
pct_improvement_auprc = (improvement_auprc / ablation_df.iloc[0]['auprc']) * 100

print(f"\nImprovement from feature engineering:")
print(f"  AUROC: {improvement_auroc:+.4f} ({pct_improvement_auroc:+.2f}%)")
print(f"  AUPRC: {improvement_auprc:+.4f} ({pct_improvement_auprc:+.2f}%)")

fig, axes = plt.subplots(1, 2, figsize=(12, 5))

axes[0].bar(ablation_df['label'], ablation_df['auroc'], color=['steelblue', 'coral'])
axes[0].set_ylabel('AUROC')
axes[0].set_title('Feature Engineering Impact on AUROC')
axes[0].set_ylim([0.7, 0.9])
axes[0].grid(axis='y', alpha=0.3)
for i, row in ablation_df.iterrows():
    axes[0].text(i, row['auroc'] + 0.005, f"{row['auroc']:.4f}", ha='center', fontsize=10)

axes[1].bar(ablation_df['label'], ablation_df['auprc'], color=['steelblue', 'coral'])
axes[1].set_ylabel('AUPRC')
axes[1].set_title('Feature Engineering Impact on AUPRC')
axes[1].set_ylim([0.3, 0.5])
axes[1].grid(axis='y', alpha=0.3)
for i, row in ablation_df.iterrows():
    axes[1].text(i, row['auprc'] + 0.005, f"{row['auprc']:.4f}", ha='center', fontsize=10)

fig.tight_layout()
plt.savefig(config.BASE_DIR / 'figures' / 'feature_engineering_ablation.png',
            dpi=300, bbox_inches='tight')
plt.show()

ablation_df.to_csv(config.BASE_DIR / 'results' / 'feature_engineering_ablation.csv', index=False)

print(f"\nEngineered features ({len(engineered_cols)}):")
for feat in sorted(engineered_cols)[:15]:
    print(f"  - {feat}")
if len(engineered_cols) > 15:
    print(f"  ... and {len(engineered_cols) - 15} more")

## Save Enhanced Features

In [ ]:
enhanced_data = {
    'X_train': X_train_scaled,
    'X_val': X_val_scaled,
    'X_test': X_test_scaled,
    'y_train': y_train,
    'y_val': y_val,
    'y_test': y_test,
    'feature_names': X_train_scaled.columns.tolist(),
    'engineered_feature_names': engineered_feature_names
}

enhanced_path = config.PROCESSED_DIR / "enhanced_features.pkl"
with open(enhanced_path, 'wb') as f:
    pickle.dump(enhanced_data, f)

engineer_path = config.PROCESSED_DIR / "feature_engineer.pkl"
with open(engineer_path, 'wb') as f:
    pickle.dump(feature_engineer, f)

In [ ]:
if processed_dir:
   write_manifest(processed_dir, defaults)


In [ ]:
from pathlib import Path
import os, json, time, pickle, numpy as np

class ExperimentDefaults:
    random_seed = 42
    dp_epsilon_grid = (0.5, 1, 2, 4, 8, 16)
    dp_delta = 1e-5
    dp_max_grad_norm = 1.0
    dp_rounds = 15
    dp_local_epochs = 3
    fl_rounds = 30
    fl_local_epochs = 5
    patience = 7
    batch_size = 64
    results_dir = "/kaggle/working/results"
    figures_dir = "/kaggle/working/figures"
    processed_dir = "/kaggle/working/processed_data_final"

def write_manifest_autodetect(processed_dir: Path, defaults) -> Path:
    candidates = ["preprocessed_data.pkl", "enhanced_features.pkl"]
    pkl_path = None
    for name in candidates:
        cand = processed_dir / name
        if cand.exists():
            pkl_path = cand
            break
    if pkl_path is None:
        raise FileNotFoundError(f"No pickle found in {processed_dir}; contents: {os.listdir(processed_dir)}")

    with open(pkl_path, "rb") as f:
        data = pickle.load(f)

    def get_arr(key):
        if key in data: return data[key]
        alt = f"{key}_scaled"
        return data.get(alt)

    def shape(key):
        arr = get_arr(key)
        try: return (arr.shape[0], arr.shape[1])
        except Exception: return None

    def balance(key):
        y = data.get(key)
        if y is None: return None
        y = np.asarray(y).ravel()
        pos, neg = int((y==1).sum()), int((y==0).sum())
        tot = pos + neg
        return {"pos": pos, "neg": neg, "pos_pct": float(pos/tot) if tot else 0.0}

    manifest = {
        "generated_at": int(time.time()),
        "hostname": os.uname().nodename if hasattr(os, "uname") else "unknown",
        "defaults": defaults.__dict__,
        "artifacts": {
            "source": str(pkl_path),
            "shapes": {
                "X_train": shape("X_train"),
                "X_val": shape("X_val"),
                "X_test": shape("X_test"),
            },
            "class_balance": {
                "train": balance("y_train"),
                "val": balance("y_val"),
                "test": balance("y_test"),
            },
        },
    }
    out = processed_dir / "manifest.json"
    with open(out, "w") as f:
        json.dump(manifest, f, indent=2)
    return out

processed_dir = Path("/kaggle/working/processed_data_final")
out = write_manifest_autodetect(processed_dir, ExperimentDefaults())

# PART 3: Baseline Models


In [ ]:
class EnhancedMLP(nn.Module):
    def __init__(
        self,
        input_dim: int,
        hidden_dims: List[int] = [256, 128, 64],
        dropout: float = 0.3,
        use_batch_norm: bool = True
    ):
        super(EnhancedMLP, self).__init__()

        layers = []
        prev_dim = input_dim

        for hidden_dim in hidden_dims:
            layers.append(nn.Linear(prev_dim, hidden_dim))

            if use_batch_norm:
                layers.append(nn.BatchNorm1d(hidden_dim))

            layers.append(nn.ReLU())
            layers.append(nn.Dropout(dropout))

            prev_dim = hidden_dim

        layers.append(nn.Linear(prev_dim, 1))

        self.model = nn.Sequential(*layers)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.model(x).squeeze(-1)

In [ ]:
class SAINTModel(nn.Module):
    def __init__(
        self,
        input_dim: int,
        d_model: int = 256,
        n_heads: int = 8,
        n_layers: int = 2,
        dropout: float = 0.1,
        activation: str = 'gelu'
    ):
        super(SAINTModel, self).__init__()

        self.input_dim = input_dim
        self.d_model = d_model

        self.feature_embedding = nn.Linear(input_dim, d_model)
        self.positional_encoding = nn.Parameter(torch.randn(1, 1, d_model))

        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model,
            nhead=n_heads,
            dim_feedforward=d_model * 4,
            dropout=dropout,
            activation=activation,
            batch_first=True
        )
        self.transformer_encoder = nn.TransformerEncoder(
            encoder_layer,
            num_layers=n_layers
        )

        self.layer_norm = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)
        self.fc_out = nn.Linear(d_model, 1)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.feature_embedding(x)
        x = x.unsqueeze(1)

        x = x + self.positional_encoding

        x = self.transformer_encoder(x)

        x = x.squeeze(1)

        x = self.layer_norm(x)
        x = self.dropout(x)
        x = self.fc_out(x)
        x = x.squeeze(-1)

        return x

In [ ]:
class ResNetBlock(nn.Module):
    def __init__(self, dim, dropout=0.2):
        super().__init__()
        self.block = nn.Sequential(
            nn.Linear(dim, dim),
            nn.GroupNorm(8, dim),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(dim, dim),
            nn.GroupNorm(8, dim)
        )
        self.activation = nn.ReLU()

    def forward(self, x):
        return self.activation(x + self.block(x))

class ClinicalResNet(nn.Module):
    def __init__(self, input_dim, hidden_dim=256, num_blocks=3, dropout=0.2):
        super().__init__()
        self.input_layer = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU()
        )
        self.blocks = nn.Sequential(
            *[ResNetBlock(hidden_dim, dropout) for _ in range(num_blocks)]
        )
        self.head = nn.Linear(hidden_dim, 1)

    def forward(self, x):
        x = self.input_layer(x)
        x = self.blocks(x)
        return self.head(x).squeeze(-1)

## Training Utilities for Deep Learning Models


In [ ]:
class FocalLoss(nn.Module):
    def __init__(self, alpha=0.25, gamma=2.0, reduction='mean'):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.reduction = reduction

    def forward(self, inputs, targets):
        BCE_loss = F.binary_cross_entropy_with_logits(inputs, targets, reduction='none')
        pt = torch.exp(-BCE_loss)
        F_loss = self.alpha * (1-pt)**self.gamma * BCE_loss

        if self.reduction == 'mean':
            return torch.mean(F_loss)
        elif self.reduction == 'sum':
            return torch.sum(F_loss)
        else:
            return F_loss

In [ ]:
import numpy as np
import torch
import torch.nn as nn
from sklearn.metrics import roc_auc_score, average_precision_score
from torch.utils.data import DataLoader
from typing import Dict, List, Tuple

def compute_pos_weight(y_train: np.ndarray) -> float:
    pos = (y_train == 1).sum()
    neg = (y_train == 0).sum()
    return float(neg / pos) if pos > 0 else 1.0

def train_deep_model(
    model: nn.Module,
    train_loader: DataLoader,
    val_loader: DataLoader,
    pos_weight: float,
    n_epochs: int = 100,
    learning_rate: float = 1e-4,
    weight_decay: float = 1e-5,
    patience: int = 10,
    device: str = "cpu",
):
    model = model.to(device)

    criterion = FocalLoss(alpha=0.25, gamma=2.0)
    optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode="max", factor=0.5, patience=5, verbose=False
    )

    history = {"train_loss": [], "val_loss": [], "val_auroc": [], "val_auprc": []}
    best_val_auroc = 0.0
    best_model_state = None
    bad_rounds = 0

    for epoch in range(n_epochs):
        model.train()
        train_losses = []
        for xb, yb in train_loader:
            xb, yb = xb.to(device), yb.to(device).float()
            optimizer.zero_grad()
            logits = model(xb)
            loss = criterion(logits, yb)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            train_losses.append(loss.item())
        avg_train_loss = float(np.mean(train_losses))

        model.eval()
        val_losses, preds, targets = [], [], []
        with torch.no_grad():
            for xb, yb in val_loader:
                xb, yb = xb.to(device), yb.to(device).float()
                logits = model(xb)
                loss = criterion(logits, yb)
                val_losses.append(loss.item())
                probs = torch.sigmoid(logits)
                preds.append(probs.cpu())
                targets.append(yb.cpu())

        preds = torch.cat(preds).numpy()
        targets = torch.cat(targets).numpy()
        avg_val_loss = float(np.mean(val_losses))
        val_auroc = roc_auc_score(targets, preds)
        val_auprc = average_precision_score(targets, preds)

        history["train_loss"].append(avg_train_loss)
        history["val_loss"].append(avg_val_loss)
        history["val_auroc"].append(val_auroc)
        history["val_auprc"].append(val_auprc)

        scheduler.step(val_auroc)

        improved = val_auroc > best_val_auroc
        if improved:
            best_val_auroc = val_auroc
            best_model_state = model.state_dict().copy()
            bad_rounds = 0
        else:
            bad_rounds += 1

        if (epoch + 1) % 10 == 0:
            print(
                f"Epoch {epoch + 1}/{n_epochs} | "
                f"Train Loss: {avg_train_loss:.4f} | "
                f"Val Loss: {avg_val_loss:.4f} | "
                f"Val AUROC: {val_auroc:.4f}"
            )

        if bad_rounds >= patience:
            print(f"Early stopping at epoch {epoch + 1}")
            break

    if best_model_state is not None:
        model.load_state_dict(best_model_state)

    print(f"Training complete. Best Val AUROC: {best_val_auroc:.4f}")
    return history

In [ ]:
import torch
import numpy as np
from typing import Tuple, Union


def evaluate_model(
    model: torch.nn.Module,
    dataloader: torch.utils.data.DataLoader,
    device: Union[torch.device, str],
) -> Tuple[np.ndarray, np.ndarray]:
    model.eval()
    all_probs = []
    all_labels = []

    device_obj = device if isinstance(device, torch.device) else torch.device(device)
    device_type = device_obj.type

    with torch.no_grad(), torch.amp.autocast(
        device_type=device_type,
        enabled=(device_type == "cuda")
    ):
        for batch_x, batch_y in dataloader:
            batch_x = batch_x.to(device_obj, non_blocking=True)
            batch_y = batch_y.to(device_obj, non_blocking=True)

            logits = model(batch_x)
            probs = torch.sigmoid(logits.squeeze(-1))

            all_probs.append(probs.cpu().numpy())
            all_labels.append(batch_y.cpu().numpy())

    return np.concatenate(all_probs), np.concatenate(all_labels)

## Cross-Validation and Hyperparameter Optimization


In [ ]:
def cross_validate_sklearn_model(
    model_class,
    model_params: Dict[str, Any],
    X: pd.DataFrame,
    y: np.ndarray,
    cv_folds: int = 5
) -> Tuple[float, float, List[float]]:
    cv = StratifiedKFold(
        n_splits=cv_folds,
        shuffle=True,
        random_state=config.RANDOM_STATE
    )

    scores = []

    for fold, (train_idx, val_idx) in enumerate(cv.split(X, y)):
        X_train_cv = X.iloc[train_idx]
        y_train_cv = y[train_idx]
        X_val_cv = X.iloc[val_idx]
        y_val_cv = y[val_idx]

        model = model_class(**model_params)
        model.fit(X_train_cv, y_train_cv)

        y_pred_proba = model.predict_proba(X_val_cv)[:, 1]
        auroc = roc_auc_score(y_val_cv, y_pred_proba)
        scores.append(auroc)

    return np.mean(scores), np.std(scores), scores

In [ ]:
def objective_xgboost(trial, X, y):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'max_depth': trial.suggest_int('max_depth', 3, 15),
        'learning_rate': trial.suggest_float('learning_rate', 0.005, 0.3, log=True),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.4, 1.0),
        'subsample': trial.suggest_float('subsample', 0.4, 1.0),
        'gamma': trial.suggest_float('gamma', 0, 10),
        'reg_alpha': trial.suggest_float('reg_alpha', 0, 20),
        'reg_lambda': trial.suggest_float('reg_lambda', 0, 20),
        'scale_pos_weight': trial.suggest_float('scale_pos_weight', 1.0, 15.0),
        'random_state': config.RANDOM_STATE,
        'tree_method': 'hist',
        'eval_metric': 'logloss'
    }

    mean_score, _, _ = cross_validate_sklearn_model(
        xgb.XGBClassifier, params, X, y, cv_folds=config.CV_FOLDS
    )

    return mean_score

def objective_lightgbm(trial, X, y):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'max_depth': trial.suggest_int('max_depth', 3, 15),
        'learning_rate': trial.suggest_float('learning_rate', 0.005, 0.3, log=True),
        'num_leaves': trial.suggest_int('num_leaves', 20, 200),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 10, 100),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.4, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.4, 1.0),
        'subsample': trial.suggest_float('subsample', 0.4, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 0, 20),
        'reg_lambda': trial.suggest_float('reg_lambda', 0, 20),
        'scale_pos_weight': trial.suggest_float('scale_pos_weight', 1.0, 15.0),
        'random_state': config.RANDOM_STATE,
        'verbose': -1
    }

    mean_score, _, _ = cross_validate_sklearn_model(
        lgb.LGBMClassifier, params, X, y, cv_folds=config.CV_FOLDS
    )

    return mean_score

def objective_randomforest(trial, X, y):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 500),
        'max_depth': trial.suggest_int('max_depth', 5, 30),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 20),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 10),
        'max_features': trial.suggest_categorical('max_features', ['sqrt', 'log2', 0.5, 0.7, 0.9]),
        'random_state': config.RANDOM_STATE
    }

    mean_score, _, _ = cross_validate_sklearn_model(
        RandomForestClassifier, params, X, y, cv_folds=config.CV_FOLDS
    )

    return mean_score

def objective_logistic(trial, X, y):
    params = {
        'C': trial.suggest_float('C', 1e-4, 100, log=True),
        'penalty': trial.suggest_categorical('penalty', ['l1', 'l2', 'elasticnet']),
        'solver': 'saga',
        'l1_ratio': trial.suggest_float('l1_ratio', 0, 1) if trial.params.get('penalty') == 'elasticnet' else None,
        'max_iter': 1000,
        'random_state': config.RANDOM_STATE
    }

    if params['l1_ratio'] is None:
        del params['l1_ratio']

    mean_score, _, _ = cross_validate_sklearn_model(
        LogisticRegression, params, X, y, cv_folds=config.CV_FOLDS
    )

    return mean_score

## Load Enhanced Features from Part 2


In [ ]:
enhanced_path = config.PROCESSED_DIR / "enhanced_features.pkl"

if not enhanced_path.exists():
    raise FileNotFoundError(f"Enhanced features not found at {enhanced_path}. Please run Part 2 first.")

with open(enhanced_path, 'rb') as f:
    enhanced_data = pickle.load(f)

X_train_scaled = enhanced_data['X_train']
X_val_scaled = enhanced_data['X_val']
X_test_scaled = enhanced_data['X_test']
y_train = enhanced_data['y_train']
y_val = enhanced_data['y_val']
y_test = enhanced_data['y_test']

## Train Baseline ML Models


In [ ]:
baseline_models = {}
baseline_results = {}

In [ ]:
# 1. Initialize and Train
lr_model = LogisticRegression(
    max_iter=1000,
    random_state=config.RANDOM_STATE,
    n_jobs=-1,
    class_weight='balanced'
)

lr_model.fit(X_train_scaled, y_train)

val_probs_lr = lr_model.predict_proba(X_val_scaled)[:, 1]
best_thresh_lr, best_f1_lr = get_optimal_threshold(y_val, val_probs_lr)

print(f"Logistic Regression - Optimal Threshold: {best_thresh_lr:.4f}")
print(f"Logistic Regression - Best Validation F1: {best_f1_lr:.4f}")

# 3. Apply Optimal Threshold to TEST set
y_test_proba_lr = lr_model.predict_proba(X_test_scaled)[:, 1]
y_test_pred_lr = (y_test_proba_lr >= best_thresh_lr).astype(int)

metrics_lr = compute_metrics_with_ci(y_test, y_test_pred_lr, y_test_proba_lr, n_bootstraps=config.N_BOOTSTRAP)
print_metrics_with_ci(metrics_lr, "Logistic Regression Test Results")

baseline_models['Logistic Regression'] = lr_model
baseline_results['Logistic Regression'] = {
    'predictions': y_test_pred_lr,
    'probabilities': y_test_proba_lr,
    'metrics': metrics_lr,
    'optimal_threshold': best_thresh_lr
}

with open(config.MODELS_DIR / "logistic_regression.pkl", 'wb') as f:
    pickle.dump(lr_model, f)

In [ ]:
rf_model = RandomForestClassifier(
    n_estimators=200,
    max_depth=10,
    min_samples_split=10,
    min_samples_leaf=5,
    max_features='sqrt',
    random_state=config.RANDOM_STATE,
    n_jobs=-1,
    class_weight='balanced'
)

rf_model.fit(X_train_scaled, y_train)

val_probs_rf = rf_model.predict_proba(X_val_scaled)[:, 1]
best_thresh_rf, best_f1_rf = get_optimal_threshold(y_val, val_probs_rf)

print(f"Random Forest - Optimal Threshold: {best_thresh_rf:.4f}")
print(f"Random Forest - Best Validation F1: {best_f1_rf:.4f}")

y_test_proba_rf = rf_model.predict_proba(X_test_scaled)[:, 1]
y_test_pred_rf = (y_test_proba_rf >= best_thresh_rf).astype(int)

metrics_rf = compute_metrics_with_ci(y_test, y_test_pred_rf, y_test_proba_rf, n_bootstraps=config.N_BOOTSTRAP)
print_metrics_with_ci(metrics_rf, "Random Forest Test Results")

baseline_models['Random Forest'] = rf_model
baseline_results['Random Forest'] = {
    'predictions': y_test_pred_rf,
    'probabilities': y_test_proba_rf,
    'metrics': metrics_rf,
    'optimal_threshold': best_thresh_rf
}

with open(config.MODELS_DIR / "random_forest.pkl", 'wb') as f:
    pickle.dump(rf_model, f)

In [ ]:
study_xgb = optuna.create_study(
    direction='maximize',
    sampler=TPESampler(seed=config.RANDOM_STATE)
)

study_xgb.optimize(
    lambda trial: objective_xgboost(trial, X_train_scaled, y_train),
    n_trials=config.OPTUNA_N_TRIALS,
    timeout=config.OPTUNA_TIMEOUT,
    show_progress_bar=True
)

print(f"Best AUROC (CV): {study_xgb.best_value:.4f}")
print(f"Best parameters: {study_xgb.best_params}")

xgb_best_params = study_xgb.best_params.copy()
xgb_best_params.update({
    'random_state': config.RANDOM_STATE,
    'tree_method': 'hist',
    'eval_metric': 'logloss'
})

xgb_model = xgb.XGBClassifier(**xgb_best_params)
xgb_model.fit(X_train_scaled, y_train)

y_test_pred_xgb = xgb_model.predict(X_test_scaled)
y_test_proba_xgb = xgb_model.predict_proba(X_test_scaled)[:, 1]

metrics_xgb = compute_metrics_with_ci(y_test, y_test_pred_xgb, y_test_proba_xgb, n_bootstraps=config.N_BOOTSTRAP)
print_metrics_with_ci(metrics_xgb, "XGBoost Test Results")

baseline_models['XGBoost'] = xgb_model
baseline_results['XGBoost'] = {
    'predictions': y_test_pred_xgb,
    'probabilities': y_test_proba_xgb,
    'metrics': metrics_xgb,
    'best_params': study_xgb.best_params,
    'cv_auroc': study_xgb.best_value
}

with open(config.MODELS_DIR / "xgboost_model.pkl", 'wb') as f:
    pickle.dump(xgb_model, f)
with open(config.MODELS_DIR / "xgboost_study.pkl", 'wb') as f:
    pickle.dump(study_xgb, f)

In [ ]:
xgb_best_params = {
    'n_estimators': 944,
    'max_depth': 13,
    'learning_rate': 0.0162832534520785,
    'min_child_weight': 6,
    'colsample_bytree': 0.40160892521374203,
    'subsample': 0.47574287837455026,
    'gamma': 4.775973089877698,
    'reg_alpha': 8.985970884575682,
    'reg_lambda': 8.608624997371484,
    'scale_pos_weight': 3.084756766110043,
    # Add standard config
    'random_state': config.RANDOM_STATE,
    'tree_method': 'hist',
    'eval_metric': 'logloss'
}

print("Skipping Optuna. Training XGBoost with best parameters from previous run...")

xgb_model = xgb.XGBClassifier(**xgb_best_params)
xgb_model.fit(X_train_scaled, y_train)

val_probs_xgb = xgb_model.predict_proba(X_val_scaled)[:, 1]
best_thresh_xgb, best_f1_xgb = get_optimal_threshold(y_val, val_probs_xgb)

print(f"XGBoost - Optimal Threshold: {best_thresh_xgb:.4f}")
print(f"XGBoost - Best Validation F1: {best_f1_xgb:.4f}")

y_test_proba_xgb = xgb_model.predict_proba(X_test_scaled)[:, 1]
y_test_pred_xgb = (y_test_proba_xgb >= best_thresh_xgb).astype(int)

metrics_xgb = compute_metrics_with_ci(y_test, y_test_pred_xgb, y_test_proba_xgb, n_bootstraps=config.N_BOOTSTRAP)
print_metrics_with_ci(metrics_xgb, "XGBoost Test Results")

baseline_models['XGBoost'] = xgb_model
baseline_results['XGBoost'] = {
    'predictions': y_test_pred_xgb,
    'probabilities': y_test_proba_xgb,
    'metrics': metrics_xgb,
    'best_params': xgb_best_params,
    'optimal_threshold': best_thresh_xgb
}

with open(config.MODELS_DIR / "xgboost_model.pkl", 'wb') as f:
    pickle.dump(xgb_model, f)

In [ ]:
study_lgb = optuna.create_study(
    direction='maximize',
    sampler=TPESampler(seed=config.RANDOM_STATE)
)

study_lgb.optimize(
    lambda trial: objective_lightgbm(trial, X_train_scaled, y_train),
    n_trials=config.OPTUNA_N_TRIALS,
    timeout=config.OPTUNA_TIMEOUT,
    show_progress_bar=True
)

print(f"Best AUROC (CV): {study_lgb.best_value:.4f}")
print(f"Best parameters: {study_lgb.best_params}")

lgb_best_params = study_lgb.best_params.copy()
lgb_best_params.update({
    'random_state': config.RANDOM_STATE,
    'verbose': -1
})

lgb_model = lgb.LGBMClassifier(**lgb_best_params)
lgb_model.fit(X_train_scaled, y_train)

y_test_pred_lgb = lgb_model.predict(X_test_scaled)
y_test_proba_lgb = lgb_model.predict_proba(X_test_scaled)[:, 1]

metrics_lgb = compute_metrics_with_ci(y_test, y_test_pred_lgb, y_test_proba_lgb, n_bootstraps=config.N_BOOTSTRAP)
print_metrics_with_ci(metrics_lgb, "LightGBM Test Results")

baseline_models['LightGBM'] = lgb_model
baseline_results['LightGBM'] = {
    'predictions': y_test_pred_lgb,
    'probabilities': y_test_proba_lgb,
    'metrics': metrics_lgb,
    'best_params': study_lgb.best_params,
    'cv_auroc': study_lgb.best_value
}

with open(config.MODELS_DIR / "lightgbm_model.pkl", 'wb') as f:
    pickle.dump(lgb_model, f)
with open(config.MODELS_DIR / "lightgbm_study.pkl", 'wb') as f:
    pickle.dump(study_lgb, f)

In [ ]:
lgb_best_params = {
    'n_estimators': 961,
    'max_depth': 13,
    'learning_rate': 0.009692970163443861,
    'num_leaves': 162,
    'min_data_in_leaf': 51,
    'feature_fraction': 0.4513246305977976,
    'colsample_bytree': 0.600983489371053,
    'subsample': 0.8629834144981184,
    'reg_alpha': 1.4598070154359528,
    'reg_lambda': 13.236885590293403,
    'scale_pos_weight': 4.217331563736673,
    'random_state': config.RANDOM_STATE,
    'verbose': -1
}

print("Skipping Optuna. Training LightGBM with best parameters from previous run...")

lgb_model = lgb.LGBMClassifier(**lgb_best_params)
lgb_model.fit(X_train_scaled, y_train)

val_probs_lgb = lgb_model.predict_proba(X_val_scaled)[:, 1]
best_thresh_lgb, best_f1_lgb = get_optimal_threshold(y_val, val_probs_lgb)

print(f"LightGBM - Optimal Threshold: {best_thresh_lgb:.4f}")
print(f"LightGBM - Best Validation F1: {best_f1_lgb:.4f}")

y_test_proba_lgb = lgb_model.predict_proba(X_test_scaled)[:, 1]
y_test_pred_lgb = (y_test_proba_lgb >= best_thresh_lgb).astype(int)

metrics_lgb = compute_metrics_with_ci(y_test, y_test_pred_lgb, y_test_proba_lgb, n_bootstraps=config.N_BOOTSTRAP)
print_metrics_with_ci(metrics_lgb, "LightGBM Test Results")

baseline_models['LightGBM'] = lgb_model
baseline_results['LightGBM'] = {
    'predictions': y_test_pred_lgb,
    'probabilities': y_test_proba_lgb,
    'metrics': metrics_lgb,
    'best_params': lgb_best_params,
    'optimal_threshold': best_thresh_lgb
}

with open(config.MODELS_DIR / "lightgbm_model.pkl", 'wb') as f:
    pickle.dump(lgb_model, f)

## Train Deep Learning Models


In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

train_dataset = TensorDataset(
    torch.FloatTensor(X_train_scaled.values),
    torch.FloatTensor(y_train)
)
val_dataset = TensorDataset(
    torch.FloatTensor(X_val_scaled.values),
    torch.FloatTensor(y_val)
)
test_dataset = TensorDataset(
    torch.FloatTensor(X_test_scaled.values),
    torch.FloatTensor(y_test)
)

train_loader = DataLoader(
    train_dataset, batch_size=config.BATCH_SIZE,
    shuffle=True, worker_init_fn=worker_init_fn
)
val_loader = DataLoader(
    val_dataset, batch_size=config.BATCH_SIZE, shuffle=False
)
test_loader = DataLoader(
    test_dataset, batch_size=config.BATCH_SIZE, shuffle=False
)

print(f"Train batches: {len(train_loader)}")
print(f"Val batches: {len(val_loader)}")
print(f"Test batches: {len(test_loader)}")

In [ ]:
pos_weight = compute_pos_weight(y_train)

mlp_params = {
    "input_dim": X_train_scaled.shape[1],
    "hidden_dims": [256, 128, 64],
    "dropout": 0.3,
    "use_batch_norm": True,
}
mlp_model = EnhancedMLP(**mlp_params)
learning_rate = 1e-4

history_mlp = train_deep_model(
    mlp_model,
    train_loader,
    val_loader,
    pos_weight=pos_weight,
    n_epochs=config.MAX_EPOCHS,
    learning_rate=learning_rate,
    patience=config.EARLY_STOPPING_PATIENCE,
    device=device,
)

val_probs_mlp, val_true_mlp = evaluate_model(mlp_model, val_loader, device=device)
best_thresh_mlp, best_f1_mlp = get_optimal_threshold(val_true_mlp, val_probs_mlp)

print(f"Enhanced MLP - Optimal Threshold: {best_thresh_mlp:.4f}")
print(f"Enhanced MLP - Best Validation F1: {best_f1_mlp:.4f}")

y_test_proba_mlp, y_test_true_mlp = evaluate_model(mlp_model, test_loader, device=device)
y_test_pred_mlp = (y_test_proba_mlp >= best_thresh_mlp).astype(int)

metrics_mlp = compute_metrics_with_ci(
    y_test_true_mlp, y_test_pred_mlp, y_test_proba_mlp, n_bootstraps=config.N_BOOTSTRAP
)
print_metrics_with_ci(metrics_mlp, "Enhanced MLP Test Results")

baseline_models["Enhanced MLP"] = mlp_model
baseline_results["Enhanced MLP"] = {
    "predictions": y_test_pred_mlp,
    "probabilities": y_test_proba_mlp,
    "metrics": metrics_mlp,
    "history": history_mlp,
    "optimal_threshold": best_thresh_mlp
}

torch.save(mlp_model.state_dict(), config.MODELS_DIR / "enhanced_mlp.pt")

In [ ]:
saint_params = {
    "input_dim": X_train_scaled.shape[1],
    "d_model": config.SAINT_PARAMS["d_model"],
    "n_heads": config.SAINT_PARAMS["n_heads"],
    "n_layers": config.SAINT_PARAMS["n_layers"],
    "dropout": config.SAINT_PARAMS["dropout"],
    "activation": "gelu",
}
saint_model = SAINTModel(**saint_params)
learning_rate = config.SAINT_PARAMS["lr"]

history_saint = train_deep_model(
    saint_model,
    train_loader,
    val_loader,
    pos_weight=pos_weight,
    n_epochs=config.MAX_EPOCHS,
    learning_rate=learning_rate,
    patience=config.EARLY_STOPPING_PATIENCE,
    device=device,
)

val_probs_saint, val_true_saint = evaluate_model(saint_model, val_loader, device=device)
best_thresh_saint, best_f1_saint = get_optimal_threshold(val_true_saint, val_probs_saint)

print(f"SAINT - Optimal Threshold: {best_thresh_saint:.4f}")
print(f"SAINT - Best Validation F1: {best_f1_saint:.4f}")

y_test_proba_saint, y_test_true_saint = evaluate_model(saint_model, test_loader, device=device)
y_test_pred_saint = (y_test_proba_saint >= best_thresh_saint).astype(int)

metrics_saint = compute_metrics_with_ci(
    y_test_true_saint, y_test_pred_saint, y_test_proba_saint, n_bootstraps=config.N_BOOTSTRAP
)
print_metrics_with_ci(metrics_saint, "SAINT Test Results")

baseline_models["SAINT"] = saint_model
baseline_results["SAINT"] = {
    "predictions": y_test_pred_saint,
    "probabilities": y_test_proba_saint,
    "metrics": metrics_saint,
    "history": history_saint,
    "optimal_threshold": best_thresh_saint
}

torch.save(saint_model.state_dict(), config.MODELS_DIR / "saint_model.pt")

In [ ]:
print("Training Clinical ResNet...")

resnet_params = {
    "input_dim": X_train_scaled.shape[1],
    "hidden_dim": 128,
    "num_blocks": 2,
    "dropout": 0.4
}
resnet_model = ClinicalResNet(**resnet_params)

optimizer = torch.optim.AdamW(
    resnet_model.parameters(),
    lr=3e-4,
    weight_decay=1e-2
)

resnet_model = resnet_model.to(device)
criterion = FocalLoss(alpha=0.25, gamma=2.0)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode="max", factor=0.5, patience=5, verbose=True
)

best_val_auroc = 0.0
best_model_state = None
patience = 15
bad_rounds = 0
history_resnet = {"val_auroc": []}

for epoch in range(100):
    resnet_model.train()
    for xb, yb in train_loader:
        xb, yb = xb.to(device), yb.to(device).float()
        optimizer.zero_grad()
        logits = resnet_model(xb)
        loss = criterion(logits, yb)
        loss.backward()
        optimizer.step()

    val_probs, val_true = evaluate_model(resnet_model, val_loader, device)
    val_auroc = roc_auc_score(val_true, val_probs)
    history_resnet["val_auroc"].append(val_auroc)

    scheduler.step(val_auroc)

    if val_auroc > best_val_auroc:
        best_val_auroc = val_auroc
        best_model_state = resnet_model.state_dict().copy()
        bad_rounds = 0
    else:
        bad_rounds += 1

    if (epoch + 1) % 10 == 0:
        print(f"Epoch {epoch + 1} | Val AUROC: {val_auroc:.4f}")

    if bad_rounds >= patience:
        print(f"Early stopping at epoch {epoch + 1}")
        break

if best_model_state:
    resnet_model.load_state_dict(best_model_state)

print(f"Training complete. Best Val AUROC: {best_val_auroc:.4f}")

val_probs, val_true = evaluate_model(resnet_model, val_loader, device)
best_thresh_rn, best_f1_rn = get_optimal_threshold(val_true, val_probs)

print(f"Clinical ResNet - Optimal Threshold: {best_thresh_rn:.4f}")
print(f"Clinical ResNet - Best Validation F1: {best_f1_rn:.4f}")

y_test_proba_rn, y_test_true_rn = evaluate_model(resnet_model, test_loader, device)
y_test_pred_rn = (y_test_proba_rn >= best_thresh_rn).astype(int)

metrics_rn = compute_metrics_with_ci(
    y_test_true_rn, y_test_pred_rn, y_test_proba_rn, n_bootstraps=config.N_BOOTSTRAP
)
print_metrics_with_ci(metrics_rn, "Clinical ResNet Test Results")

baseline_models["ClinicalResNet"] = resnet_model
baseline_results["ClinicalResNet"] = {
    "predictions": y_test_pred_rn,
    "probabilities": y_test_proba_rn,
    "metrics": metrics_rn,
    "optimal_threshold": best_thresh_rn
}
torch.save(resnet_model.state_dict(), config.MODELS_DIR / "resnet_model.pt")

In [ ]:
import pickle
import torch
from torch.utils.data import DataLoader, TensorDataset
import optuna
from optuna.samplers import TPESampler

def make_loaders(Xtr, ytr, Xv, yv, batch_size):
    train_ds = TensorDataset(
        torch.tensor(Xtr.values, dtype=torch.float32),
        torch.tensor(ytr, dtype=torch.float32),
    )
    val_ds = TensorDataset(
        torch.tensor(Xv.values, dtype=torch.float32),
        torch.tensor(yv, dtype=torch.float32),
    )
    train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_ds, batch_size=batch_size * 2, shuffle=False)
    return train_loader, val_loader

def run_mlp_direct():
    print("Training Enhanced MLP with Best Parameters (Skipping Optuna)...")

    best_params = {
        'n_layers': 3,
        'h0': 451, 'h1': 349, 'h2': 132,
        'dropout': 0.446,
        'lr': 9.61e-05,
        'weight_decay': 0.000755,
        'batch_size': 64
    }

    pos_weight = compute_pos_weight(y_train)

    hidden_dims = [best_params[f"h{i}"] for i in range(best_params["n_layers"])]
    mlp_model = EnhancedMLP(
        input_dim=X_train_scaled.shape[1],
        hidden_dims=hidden_dims,
        dropout=best_params["dropout"],
    )

    train_loader, val_loader = make_loaders(X_train_scaled, y_train, X_val_scaled, y_val, best_params["batch_size"])

    history_mlp = train_deep_model(
        mlp_model,
        train_loader,
        val_loader,
        pos_weight=pos_weight,
        n_epochs=config.MAX_EPOCHS,
        learning_rate=best_params["lr"],
        weight_decay=best_params["weight_decay"],
        patience=config.EARLY_STOPPING_PATIENCE,
        device=device,
    )

    val_probs, val_true = evaluate_model(mlp_model, val_loader, device=device)
    best_thresh, _ = get_optimal_threshold(val_true, val_probs)
    print(f"Enhanced MLP - Optimal Threshold: {best_thresh:.4f}")

    y_test_proba, y_test_true = evaluate_model(mlp_model, test_loader, device=device)
    y_test_pred = (y_test_proba >= best_thresh).astype(int)

    metrics_mlp = compute_metrics_with_ci(
        y_test_true, y_test_pred, y_test_proba, n_bootstraps=config.N_BOOTSTRAP
    )
    print_metrics_with_ci(metrics_mlp, "Enhanced MLP Test Results")

    baseline_models["Enhanced MLP"] = mlp_model
    baseline_results["Enhanced MLP"] = {
        "predictions": y_test_pred,
        "probabilities": y_test_proba,
        "metrics": metrics_mlp,
        "history": history_mlp,
        "optimal_threshold": best_thresh
    }
    torch.save(mlp_model.state_dict(), config.MODELS_DIR / "enhanced_mlp_optimized.pt")
    print("Enhanced MLP Saved!")

def run_saint_direct():
    print("\nTraining SAINT with Best Parameters (Skipping Optuna)...")

    best_params = {
        'd_model': 256,
        'n_heads': 4,
        'n_layers': 2,
        'dropout': 0.125,
        'lr': 2.59e-05,
        'weight_decay': 1.59e-06,
        'batch_size': 64
    }

    pos_weight = compute_pos_weight(y_train)

    saint_model = SAINTModel(
        input_dim=X_train_scaled.shape[1],
        d_model=best_params["d_model"],
        n_heads=best_params["n_heads"],
        n_layers=best_params["n_layers"],
        dropout=best_params["dropout"],
        activation="gelu",
    )

    train_loader, val_loader = make_loaders(X_train_scaled, y_train, X_val_scaled, y_val, best_params["batch_size"])

    history_saint = train_deep_model(
        saint_model,
        train_loader,
        val_loader,
        pos_weight=pos_weight,
        n_epochs=config.MAX_EPOCHS,
        learning_rate=best_params["lr"],
        weight_decay=best_params["weight_decay"],
        patience=config.EARLY_STOPPING_PATIENCE,
        device=device,
    )

    val_probs, val_true = evaluate_model(saint_model, val_loader, device=device)
    best_thresh, _ = get_optimal_threshold(val_true, val_probs)
    print(f"SAINT - Optimal Threshold: {best_thresh:.4f}")

    y_test_proba, y_test_true = evaluate_model(saint_model, test_loader, device=device)
    y_test_pred = (y_test_proba >= best_thresh).astype(int)

    metrics_saint = compute_metrics_with_ci(
        y_test_true, y_test_pred, y_test_proba, n_bootstraps=config.N_BOOTSTRAP
    )
    print_metrics_with_ci(metrics_saint, "SAINT Test Results")

    baseline_models["SAINT"] = saint_model
    baseline_results["SAINT"] = {
        "predictions": y_test_pred,
        "probabilities": y_test_proba,
        "metrics": metrics_saint,
        "history": history_saint,
        "optimal_threshold": best_thresh
    }
    torch.save(saint_model.state_dict(), config.MODELS_DIR / "saint_model_optimized.pt")
    print("SAINT Saved!")

def run_resnet_optuna():
    print("\nStarting Optuna Tuning for Clinical ResNet...")
    pos_weight = compute_pos_weight(y_train)

    def objective(trial):
        hidden_dim = trial.suggest_categorical("hidden_dim", [128, 256, 512])
        num_blocks = trial.suggest_int("num_blocks", 2, 4)
        dropout = trial.suggest_float("dropout", 0.1, 0.5)
        lr = trial.suggest_float("lr", 1e-4, 5e-3, log=True)
        weight_decay = trial.suggest_float("weight_decay", 1e-5, 1e-2, log=True)

        model = ClinicalResNet(
            input_dim=X_train_scaled.shape[1],
            hidden_dim=hidden_dim,
            num_blocks=num_blocks,
            dropout=dropout
        ).to(device)

        pos_weight_tensor = torch.tensor([pos_weight], device=device)
        criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight_tensor)
        optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)

        for epoch in range(15):
            model.train()
            for xb, yb in train_loader:
                xb, yb = xb.to(device), yb.to(device).float()
                optimizer.zero_grad()
                logits = model(xb)
                loss = criterion(logits, yb)
                loss.backward()
                optimizer.step()

            val_probs, val_true = evaluate_model(model, val_loader, device)
            val_auroc = roc_auc_score(val_true, val_probs)

            trial.report(val_auroc, epoch)
            if trial.should_prune():
                raise optuna.exceptions.TrialPruned()

        return val_auroc

    study = optuna.create_study(direction="maximize", sampler=TPESampler(seed=42))
    study.optimize(objective, n_trials=30, timeout=1800)

    print("Best ResNet Params:", study.best_params)

    print("Retraining Best ResNet Fully...")
    best = study.best_params
    final_resnet = ClinicalResNet(
        input_dim=X_train_scaled.shape[1],
        hidden_dim=best["hidden_dim"],
        num_blocks=best["num_blocks"],
        dropout=best["dropout"]
    ).to(device)

    pos_weight_tensor = torch.tensor([pos_weight], device=device)
    criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight_tensor)
    optimizer = torch.optim.AdamW(final_resnet.parameters(), lr=best["lr"], weight_decay=best["weight_decay"])
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=5)

    best_val_auc = 0.0
    best_state = None

    for epoch in range(config.MAX_EPOCHS):
        final_resnet.train()
        for xb, yb in train_loader:
            xb, yb = xb.to(device), yb.to(device).float()
            optimizer.zero_grad()
            logits = final_resnet(xb)
            loss = criterion(logits, yb)
            loss.backward()
            optimizer.step()

        val_probs, val_true = evaluate_model(final_resnet, val_loader, device)
        val_auc = roc_auc_score(val_true, val_probs)
        scheduler.step(val_auc)

        if val_auc > best_val_auc:
            best_val_auc = val_auc
            best_state = final_resnet.state_dict().copy()

        if (epoch+1) % 10 == 0:
            print(f"ResNet Epoch {epoch+1}: Val AUROC {val_auc:.4f}")

    final_resnet.load_state_dict(best_state)
    val_probs, val_true = evaluate_model(final_resnet, val_loader, device)
    best_thresh, _ = get_optimal_threshold(val_true, val_probs)
    print(f"ResNet - Optimal Threshold: {best_thresh:.4f}")

    y_test_prob, y_test_true = evaluate_model(final_resnet, test_loader, device)
    y_test_pred = (y_test_prob >= best_thresh).astype(int)

    metrics = compute_metrics_with_ci(y_test_true, y_test_pred, y_test_prob, n_bootstraps=config.N_BOOTSTRAP)
    print_metrics_with_ci(metrics, "Optimized Clinical ResNet")

    baseline_models["ClinicalResNet"] = final_resnet
    baseline_results["ClinicalResNet"] = {
        "predictions": y_test_pred,
        "probabilities": y_test_prob,
        "metrics": metrics,
        "optimal_threshold": best_thresh
    }
    torch.save(final_resnet.state_dict(), config.MODELS_DIR / "resnet_model_optimized.pt")

if __name__ == "__main__":
    run_mlp_direct()
    run_saint_direct().
    run_resnet_optuna()

In [ ]:
import optuna
from optuna.samplers import TPESampler
from optuna.pruners import HyperbandPruner
import torch.optim as optim

def run_resnet_full_optuna():
    print("Starting RIGOROUS Optuna Search for Clinical ResNet...")


    pos_weight = compute_pos_weight(y_train)
    pos_weight_tensor = torch.tensor([pos_weight], device=device)

    def objective(trial):
        hidden_dim = trial.suggest_categorical("hidden_dim", [128, 256])
        num_blocks = trial.suggest_int("num_blocks", 2, 4)
        dropout = trial.suggest_float("dropout", 0.2, 0.5)

        lr = trial.suggest_float("lr", 1e-4, 2e-3, log=True)
        weight_decay = trial.suggest_float("weight_decay", 1e-5, 1e-2, log=True)

        batch_size = trial.suggest_categorical("batch_size", [64, 128])

        model = ClinicalResNet(
            input_dim=X_train_scaled.shape[1],
            hidden_dim=hidden_dim,
            num_blocks=num_blocks,
            dropout=dropout
        ).to(device)

        train_loader_trial, val_loader_trial = make_loaders(
            X_train_scaled, y_train, X_val_scaled, y_val, batch_size
        )

        criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight_tensor)
        optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)

        for epoch in range(25):
            model.train()
            for xb, yb in train_loader_trial:
                xb, yb = xb.to(device), yb.to(device).float()
                optimizer.zero_grad()
                logits = model(xb)
                loss = criterion(logits, yb)
                loss.backward()
                torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
                optimizer.step()

            val_probs, val_true = evaluate_model(model, val_loader_trial, device)
            val_auroc = roc_auc_score(val_true, val_probs)

            trial.report(val_auroc, epoch)

            if trial.should_prune():
                raise optuna.exceptions.TrialPruned()

        return val_auroc

    study = optuna.create_study(
        direction="maximize",
        sampler=TPESampler(seed=42),
        pruner=HyperbandPruner(min_resource=5, max_resource=25, reduction_factor=3)
    )

    study.optimize(objective, n_trials=50, timeout=5400, show_progress_bar=True)

    print("SEARCH COMPLETE")
    print(f"Best Val AUROC: {study.best_value:.4f}")
    print("Best Params:", study.best_params)

    print("Retraining best architecture for full convergence...")
    bp = study.best_params

    final_model = ClinicalResNet(
        input_dim=X_train_scaled.shape[1],
        hidden_dim=bp["hidden_dim"],
        num_blocks=bp["num_blocks"],
        dropout=bp["dropout"]
    ).to(device)

    train_loader, val_loader = make_loaders(
        X_train_scaled, y_train, X_val_scaled, y_val, bp["batch_size"]
    )

    criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight_tensor)
    optimizer = optim.AdamW(final_model.parameters(), lr=bp["lr"], weight_decay=bp["weight_decay"])

    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='max', factor=0.5, patience=5, verbose=True
    )

    best_final_auc = 0.0
    best_state = None

    for epoch in range(config.MAX_EPOCHS):
        final_model.train()
        for xb, yb in train_loader:
            xb, yb = xb.to(device), yb.to(device).float()
            optimizer.zero_grad()
            logits = final_model(xb)
            loss = criterion(logits, yb)
            loss.backward()
            optimizer.step()

        val_probs, val_true = evaluate_model(final_model, val_loader, device)
        val_auc = roc_auc_score(val_true, val_probs)
        scheduler.step(val_auc)

        if val_auc > best_final_auc:
            best_final_auc = val_auc
            best_state = final_model.state_dict().copy()

        if (epoch+1) % 10 == 0:
            print(f"Final Training Epoch {epoch+1}: Val AUROC {val_auc:.4f}")

    print(f"\nFinal Best Val AUROC: {best_final_auc:.4f}")

    final_model.load_state_dict(best_state)

    val_probs, val_true = evaluate_model(final_model, val_loader, device)
    best_thresh, _ = get_optimal_threshold(val_true, val_probs)
    print(f"Optimal Threshold: {best_thresh:.4f}")

    y_test_probs, y_test_true = evaluate_model(final_model, test_loader, device)
    y_test_pred = (y_test_probs >= best_thresh).astype(int)

    metrics = compute_metrics_with_ci(y_test_true, y_test_pred, y_test_probs, n_bootstraps=config.N_BOOTSTRAP)
    print_metrics_with_ci(metrics, "Optimized Clinical ResNet")

    baseline_models["ClinicalResNet"] = final_model
    baseline_results["ClinicalResNet"] = {
        "predictions": y_test_pred,
        "probabilities": y_test_probs,
        "metrics": metrics,
        "optimal_threshold": best_thresh,
        "best_params": bp
    }
    torch.save(final_model.state_dict(), config.MODELS_DIR / "resnet_model_optimized.pt")

    with open(config.MODELS_DIR / "resnet_optuna_study.pkl", 'wb') as f:
        pickle.dump(study, f)

if __name__ == "__main__":
    run_resnet_full_optuna()

In [ ]:
!pip install pytorch_tabnet

In [ ]:
from sklearn.base import BaseEstimator, ClassifierMixin

class TabNetWrapper(BaseEstimator, ClassifierMixin):
    def __init__(self, **params):
        self.params = params.copy()
        self.params.setdefault("verbose", 1)
        self.model = None

    def fit(self, X, y, eval_set=None, eval_metric=['auc']):
        self.model = TabNetClassifier(**self.params)

        X_np = X.values if hasattr(X, 'values') else X
        y_np = y

        eval_set_np = []
        if eval_set:
            for X_val, y_val in eval_set:
                X_val_np = X_val.values if hasattr(X_val, 'values') else X_val
                eval_set_np.append((X_val_np, y_val))

        self.model.fit(
            X_train=X_np, y_train=y_np,
            eval_set=eval_set_np,
            eval_metric=eval_metric,
            max_epochs=config.MAX_EPOCHS,
            patience=config.EARLY_STOPPING_PATIENCE,
            batch_size=config.BATCH_SIZE,
            virtual_batch_size=128,
            num_workers=0,
            drop_last=False
        )
        return self

    def predict(self, X):
        X_np = X.values if hasattr(X, 'values') else X
        return self.model.predict(X_np)

    def predict_proba(self, X):
        X_np = X.values if hasattr(X, 'values') else X
        return self.model.predict_proba(X_np)

class FTTransformer(nn.Module):
    def __init__(
        self,
        n_cont_features: int,
        d_token: int = 192,
        n_layers: int = 3,
        n_heads: int = 8,
        d_ffn: int = 256,
        dropout: float = 0.1,
        attention_dropout: float = 0.1,
    ):
        super().__init__()
        self.n_cont_features = n_cont_features

        self.feature_tokenizer = nn.ModuleList([
            nn.Linear(1, d_token) for _ in range(n_cont_features)
        ])

        self.cls_token = nn.Parameter(torch.randn(1, 1, d_token))

        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_token,
            nhead=n_heads,
            dim_feedforward=d_ffn,
            dropout=dropout,
            activation='gelu',
            batch_first=True
        )
        self.transformer = nn.TransformerEncoder(
            encoder_layer,
            num_layers=n_layers
        )

        self.head = nn.Sequential(
            nn.LayerNorm(d_token),
            nn.ReLU(),
            nn.Linear(d_token, 1)
        )

        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    nn.init.zeros_(m.bias)

    def forward(self, x):
        batch_size = x.shape[0]

        tokens = []
        for i in range(self.n_cont_features):
            feature_i = x[:, i].unsqueeze(1)
            token_i = self.feature_tokenizer[i](feature_i)
            tokens.append(token_i.unsqueeze(1))

        x_emb = torch.cat(tokens, dim=1)

        cls_tokens = self.cls_token.expand(batch_size, -1, -1)
        x_emb = torch.cat((cls_tokens, x_emb), dim=1)

        x_out = self.transformer(x_emb)

        cls_out = x_out[:, 0, :]

        logits = self.head(cls_out).squeeze(-1)
        return logits

import torch
import torch.nn as nn

class EnhancedMLP(nn.Module):
    def __init__(self, input_dim, hidden_dims=[451, 349, 132], dropout=0.1):
        super().__init__()
        layers = []
        in_dim = input_dim
        for h_dim in hidden_dims:
            layers.append(nn.Linear(in_dim, h_dim))
            layers.append(nn.BatchNorm1d(h_dim))
            layers.append(nn.ReLU())
            if dropout > 0:
                layers.append(nn.Dropout(dropout))
            in_dim = h_dim
        layers.append(nn.Linear(in_dim, 1))
        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)

In [ ]:
def run_tabnet_optuna():
    def objective_tabnet(trial):
        params = {
            'n_d': trial.suggest_int('n_d', 8, 64),
            'n_a': trial.suggest_int('n_a', 8, 64),
            'n_steps': trial.suggest_int('n_steps', 3, 10),
            'gamma': trial.suggest_float('gamma', 1.0, 2.0),
            'lambda_sparse': trial.suggest_float('lambda_sparse', 1e-6, 1e-3, log=True),
            'optimizer_params': dict(lr=trial.suggest_float('lr', 1e-3, 5e-2, log=True)),
            'mask_type': 'entmax',
            'verbose': 1
        }
        model = TabNetWrapper(**params)
        model.fit(
            X_train_scaled, y_train,
            eval_set=[(X_val_scaled, y_val)],
            eval_metric=['auc']
        )
        preds = model.predict_proba(X_val_scaled)[:, 1]
        val_auc = roc_auc_score(y_val, preds)
        print(f"[TabNet][Trial {trial.number + 1}] val AUROC={val_auc:.4f}", flush=True)
        return val_auc

    def log_best_tabnet(study, trial):
        if study.best_trial:
            best = study.best_trial
            print(
                f"[TabNet][Best so far] Trial {best.number} AUROC={best.value:.4f} params={best.params}",
                flush=True,
            )

    study_tabnet = optuna.create_study(direction="maximize", sampler=TPESampler(seed=config.RANDOM_STATE))
    study_tabnet.optimize(
        objective_tabnet,
        n_trials=config.OPTUNA_N_TRIALS,
        timeout=config.OPTUNA_TIMEOUT,
        show_progress_bar=True,
        callbacks=[log_best_tabnet],
    )

    print(f"\nBest AUROC (val): {study_tabnet.best_value:.4f}")
    print(f"Best params: {study_tabnet.best_params}")

    best_params = study_tabnet.best_params.copy()
    if 'lr' in best_params:
        best_params['optimizer_params'] = dict(lr=best_params.pop('lr'))
    best_params['verbose'] = 1
    best_params['mask_type'] = 'entmax'

    tabnet_model = TabNetWrapper(**best_params)
    print(f"[TabNet] Training final model with best params and {config.MAX_EPOCHS} max epochs...", flush=True)
    tabnet_model.fit(
        X_train_scaled, y_train,
        eval_set=[(X_val_scaled, y_val)],
        eval_metric=['auc']
    )
    print("[TabNet] Final training finished.", flush=True)

    y_test_proba_tab = tabnet_model.predict_proba(X_test_scaled)[:, 1]
    y_test_pred_tab = (y_test_proba_tab >= 0.5).astype(int)

    metrics_tab = compute_metrics_with_ci(
        y_test, y_test_pred_tab, y_test_proba_tab, n_bootstraps=config.N_BOOTSTRAP
    )
    print_metrics_with_ci(metrics_tab, "TabNet Test Results")

    baseline_models["TabNet"] = tabnet_model
    baseline_results["TabNet"] = {
        "predictions": y_test_pred_tab,
        "probabilities": y_test_proba_tab,
        "metrics": metrics_tab,
        "best_params": study_tabnet.best_params,
        "best_val_auroc": study_tabnet.best_value,
    }
    with open(config.MODELS_DIR / "tabnet_model.pkl", "wb") as f:
        pickle.dump(tabnet_model, f)
    print("TabNet model saved!")


def run_fttransformer_optuna():
    pos_weight = compute_pos_weight(y_train)

    def objective_ftt(trial):
        d_token = trial.suggest_categorical("d_token", [64, 128, 192, 256])
        n_layers = trial.suggest_int("n_layers", 1, 4)
        n_heads = trial.suggest_categorical("n_heads", [4, 8])
        d_ffn_factor = trial.suggest_float("d_ffn_factor", 1.0, 4.0)
        dropout = trial.suggest_float("dropout", 0.0, 0.5)
        lr = trial.suggest_float("lr", 1e-5, 1e-3, log=True)
        weight_decay = trial.suggest_float("weight_decay", 1e-6, 1e-3, log=True)
        batch_size = trial.suggest_categorical("batch_size", [64, 128])

        if d_token % n_heads != 0:
            return 0.0

        model = FTTransformer(
            n_cont_features=X_train_scaled.shape[1],
            d_token=d_token,
            n_layers=n_layers,
            n_heads=n_heads,
            d_ffn=int(d_token * d_ffn_factor),
            dropout=dropout,
            attention_dropout=dropout
        )

        train_loader, val_loader = make_loaders(X_train_scaled, y_train, X_val_scaled, y_val, batch_size)

        hist = train_deep_model(
            model,
            train_loader,
            val_loader,
            pos_weight=pos_weight,
            n_epochs=40,
            learning_rate=lr,
            weight_decay=weight_decay,
            patience=8,
            device=device,
        )
        best_val_auc = max(hist["val_auroc"])
        print(f"[FTT][Trial {trial.number + 1}] best val AUROC={best_val_auc:.4f}", flush=True)
        return best_val_auc

    def log_best_ftt(study, trial):
        if study.best_trial:
            best = study.best_trial
            print(
                f"[FTT][Best so far] Trial {best.number} AUROC={best.value:.4f} params={best.params}",
                flush=True,
            )

    study_ftt = optuna.create_study(direction="maximize", sampler=TPESampler(seed=config.RANDOM_STATE))
    study_ftt.optimize(
        objective_ftt,
        n_trials=config.OPTUNA_N_TRIALS,
        timeout=config.OPTUNA_TIMEOUT,
        show_progress_bar=True,
        callbacks=[log_best_ftt],
    )

    print(f"\nBest AUROC (val): {study_ftt.best_value:.4f}")
    print(f"Best params: {study_ftt.best_params}")

    bp = study_ftt.best_params
    ftt_model = FTTransformer(
        n_cont_features=X_train_scaled.shape[1],
        d_token=bp["d_token"],
        n_layers=bp["n_layers"],
        n_heads=bp["n_heads"],
        d_ffn=int(bp["d_token"] * bp["d_ffn_factor"]),
        dropout=bp["dropout"],
        attention_dropout=bp["dropout"]
    )

    train_loader, val_loader = make_loaders(X_train_scaled, y_train, X_val_scaled, y_val, bp["batch_size"])

    print(f"[FTT] Training final model for up to {config.MAX_EPOCHS} epochs (batch_size={bp['batch_size']})...", flush=True)
    history_ftt = train_deep_model(
        ftt_model,
        train_loader,
        val_loader,
        pos_weight=pos_weight,
        n_epochs=config.MAX_EPOCHS,
        learning_rate=bp["lr"],
        weight_decay=bp["weight_decay"],
        patience=config.EARLY_STOPPING_PATIENCE,
        device=device,
    )
    print(f"[FTT] Final training finished. Best val AUROC={max(history_ftt['val_auroc']):.4f}", flush=True)

    y_test_proba_ftt, y_test_true_ftt = evaluate_model(ftt_model, test_loader, device=device)
    y_test_pred_ftt = (y_test_proba_ftt >= 0.5).astype(int)

    metrics_ftt = compute_metrics_with_ci(
        y_test, y_test_pred_ftt, y_test_proba_ftt, n_bootstraps=config.N_BOOTSTRAP
    )
    print_metrics_with_ci(metrics_ftt, "FT-Transformer Test Results")

    baseline_models["FT-Transformer"] = ftt_model
    baseline_results["FT-Transformer"] = {
        "predictions": y_test_pred_ftt,
        "probabilities": y_test_proba_ftt,
        "metrics": metrics_ftt,
        "history": history_ftt,
        "best_params": study_ftt.best_params,
        "best_val_auroc": study_ftt.best_value,
    }
    torch.save(ftt_model.state_dict(), config.MODELS_DIR / "fttransformer_model.pt")
    print("FT-Transformer model saved!")

if __name__ == "__main__":
    run_tabnet_optuna()
    run_fttransformer_optuna()

In [ ]:
import pickle
import torch


def run_tabnet_direct(X_train_scaled, y_train, X_val_scaled, y_val, X_test_scaled, y_test,
                      config, baseline_models, baseline_results):
    print("[TabNet] Training with best params (no Optuna)...")
    best_params = {
        'n_d': 32,
        'n_a': 24,
        'n_steps': 7,
        'gamma': 1.139493860652042,
        'lambda_sparse': 7.523742884534855e-06,
        'optimizer_params': dict(lr=0.004192159350410977),
        'mask_type': 'entmax',
        'verbose': 1,
    }
    model = TabNetWrapper(**best_params)
    model.fit(
        X_train_scaled, y_train,
        eval_set=[(X_val_scaled, y_val)],
        eval_metric=['auc']
    )

    val_probs = model.predict_proba(X_val_scaled)[:, 1]
    best_thresh, _ = get_optimal_threshold(y_val, val_probs)
    print(f"[TabNet] Optimal Threshold: {best_thresh:.4f}")

    test_probs = model.predict_proba(X_test_scaled)[:, 1]
    test_pred = (test_probs >= best_thresh).astype(int)

    metrics_tab = compute_metrics_with_ci(
        y_test, test_pred, test_probs, n_bootstraps=config.N_BOOTSTRAP
    )
    print_metrics_with_ci(metrics_tab, "[TabNet] Test Results")

    baseline_models["TabNet"] = model
    baseline_results["TabNet"] = {
        "predictions": test_pred,
        "probabilities": test_probs,
        "metrics": metrics_tab,
        "best_params": best_params,
        "optimal_threshold": best_thresh,
    }
    with open(config.MODELS_DIR / "tabnet_model.pkl", "wb") as f:
        pickle.dump(model, f)
    print("[TabNet] Saved to tabnet_model.pkl")


def run_ftt_direct(X_train_scaled, y_train, X_val_scaled, y_val, X_test_scaled, y_test,
                   config, device, test_loader, baseline_models, baseline_results):
    print("[FT-Transformer] Training with best params (no Optuna)...")
    best_params = {
        'd_token': 192,
        'n_layers': 4,
        'n_heads': 4,
        'd_ffn_factor': 1.0165663513708072,
        'dropout': 0.4077307142274171,
        'lr': 0.0002592475660475158,
        'weight_decay': 0.00015382308040278996,
        'batch_size': 64,
    }

    pos_weight = compute_pos_weight(y_train)
    train_loader, val_loader = make_loaders(
        X_train_scaled, y_train,
        X_val_scaled, y_val,
        best_params["batch_size"]
    )

    ftt_model = FTTransformer(
        n_cont_features=X_train_scaled.shape[1],
        d_token=best_params["d_token"],
        n_layers=best_params["n_layers"],
        n_heads=best_params["n_heads"],
        d_ffn=int(best_params["d_token"] * best_params["d_ffn_factor"]),
        dropout=best_params["dropout"],
        attention_dropout=best_params["dropout"],
    )

    history_ftt = train_deep_model(
        ftt_model,
        train_loader,
        val_loader,
        pos_weight=pos_weight,
        n_epochs=config.MAX_EPOCHS,
        learning_rate=best_params["lr"],
        weight_decay=best_params["weight_decay"],
        patience=config.EARLY_STOPPING_PATIENCE,
        device=device,
    )

    val_probs, val_true = evaluate_model(ftt_model, val_loader, device=device)
    best_thresh, _ = get_optimal_threshold(val_true, val_probs)
    print(f"[FT-Transformer] Optimal Threshold: {best_thresh:.4f}")

    test_probs, test_true = evaluate_model(ftt_model, test_loader, device=device)
    test_pred = (test_probs >= best_thresh).astype(int)

    metrics_ftt = compute_metrics_with_ci(
        test_true, test_pred, test_probs, n_bootstraps=config.N_BOOTSTRAP
    )
    print_metrics_with_ci(metrics_ftt, "[FT-Transformer] Test Results")

    baseline_models["FT-Transformer"] = ftt_model
    baseline_results["FT-Transformer"] = {
        "predictions": test_pred,
        "probabilities": test_probs,
        "metrics": metrics_ftt,
        "history": history_ftt,
        "best_params": best_params,
        "optimal_threshold": best_thresh,
    }
    torch.save(ftt_model.state_dict(), config.MODELS_DIR / "fttransformer_model.pt")
    print("[FT-Transformer] Saved to fttransformer_model.pt")


if __name__ == "__main__":
    import sys
    ns = sys.modules.get("__main__")

    if not hasattr(ns, "TabNetClassifier"):
        try:
            from pytorch_tabnet.tab_model import TabNetClassifier
            setattr(ns, "TabNetClassifier", TabNetClassifier)
        except Exception as exc:
            print(f"TabNetClassifier not found. Error: {exc}")
            sys.exit(1)

    required = [
        "X_train_scaled", "y_train", "X_val_scaled", "y_val", "X_test_scaled", "y_test",
        "config", "device", "test_loader", "baseline_models", "baseline_results",
        "make_loaders", "compute_pos_weight", "evaluate_model", "get_optimal_threshold",
        "compute_metrics_with_ci", "print_metrics_with_ci", "TabNetWrapper", "FTTransformer", "train_deep_model",
    ]

    missing = [name for name in required if not hasattr(ns, name)]

    if missing:
        print(f"Missing prerequisites: {', '.join(missing)}")
    else:
        print("All prerequisites found; running TabNet and FT-Transformer with best params (no Optuna).")
        run_tabnet_direct(
            ns.X_train_scaled, ns.y_train,
            ns.X_val_scaled, ns.y_val,
            ns.X_test_scaled, ns.y_test,
            ns.config, ns.baseline_models, ns.baseline_results
        )
        run_ftt_direct(
            ns.X_train_scaled, ns.y_train,
            ns.X_val_scaled, ns.y_val,
            ns.X_test_scaled, ns.y_test,
            ns.config, ns.device, ns.test_loader,
            ns.baseline_models, ns.baseline_results
        )

In [ ]:
from pathlib import Path
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
from sklearn.metrics import roc_auc_score, average_precision_score, accuracy_score, f1_score
import optuna
from optuna.samplers import TPESampler

DATA_DIR = Path("/kaggle/input/databefore/processed_data_final")
N_TRIALS = 10
TIMEOUT = 3600
MAX_EPOCHS = 60
PATIENCE = 10
BATCH_SIZES = [64, 128]
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
SEED = 42

def load_splits():
    npy_names = [
        "X_train_scaled.npy",
        "X_val_scaled.npy",
        "X_test_scaled.npy",
        "y_train.npy",
        "y_val.npy",
        "y_test.npy",
    ]
    if all((DATA_DIR / name).exists() for name in npy_names):
        x_train = np.load(DATA_DIR / "X_train_scaled.npy")
        x_val = np.load(DATA_DIR / "X_val_scaled.npy")
        x_test = np.load(DATA_DIR / "X_test_scaled.npy")
        y_train = np.load(DATA_DIR / "y_train.npy")
        y_val = np.load(DATA_DIR / "y_val.npy")
        y_test = np.load(DATA_DIR / "y_test.npy")
        return x_train, x_val, x_test, y_train, y_val, y_test

    pkl_path = DATA_DIR / "preprocessed_data.pkl"
    if pkl_path.exists():
        import pickle

        with open(pkl_path, "rb") as f:
            obj = pickle.load(f)
        if not isinstance(obj, dict):
            raise ValueError(f"{pkl_path} does not contain a dict; got {type(obj)}")

        def grab(key_opts):
            for k in key_opts:
                if k in obj:
                    return obj[k]
            raise KeyError(f"None of {key_opts} found in {pkl_path}. Available keys: {list(obj.keys())}")

        x_train = grab(["X_train_scaled", "X_train"])
        x_val = grab(["X_val_scaled", "X_val"])
        x_test = grab(["X_test_scaled", "X_test"])
        y_train = grab(["y_train"])
        y_val = grab(["y_val"])
        y_test = grab(["y_test"])
        return x_train, x_val, x_test, y_train, y_val, y_test

    raise FileNotFoundError(
        "Could not find expected preprocessed splits. "
        "Place npy files or preprocessed_data.pkl in DATA_DIR."
    )

class FTTransformer(nn.Module):
    def __init__(
        self,
        n_cont_features: int,
        d_token: int = 192,
        n_layers: int = 3,
        n_heads: int = 8,
        d_ffn: int = 256,
        dropout: float = 0.1,
        attention_dropout: float = 0.1,
    ):
        super().__init__()
        self.n_cont_features = n_cont_features
        self.feature_tokenizer = nn.ModuleList([nn.Linear(1, d_token) for _ in range(n_cont_features)])
        self.cls_token = nn.Parameter(torch.randn(1, 1, d_token))
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_token,
            nhead=n_heads,
            dim_feedforward=d_ffn,
            dropout=dropout,
            activation="gelu",
            batch_first=True,
        )
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=n_layers)
        self.head = nn.Sequential(nn.LayerNorm(d_token), nn.ReLU(), nn.Linear(d_token, 1))
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    nn.init.zeros_(m.bias)

    def forward(self, x):
        batch_size = x.shape[0]
        tokens = []
        for i in range(self.n_cont_features):
            feature_i = x[:, i].unsqueeze(1)
            token_i = self.feature_tokenizer[i](feature_i)
            tokens.append(token_i.unsqueeze(1))
        x_emb = torch.cat(tokens, dim=1)
        cls_tokens = self.cls_token.expand(batch_size, -1, -1)
        x_emb = torch.cat((cls_tokens, x_emb), dim=1)
        x_out = self.transformer(x_emb)
        cls_out = x_out[:, 0, :]
        logits = self.head(cls_out).squeeze(-1)
        return logits

def set_seeds(seed: int = 42):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)

def compute_pos_weight(y: np.ndarray) -> torch.Tensor:
    pos = (y == 1).sum()
    neg = (y == 0).sum()
    w = neg / max(pos, 1)
    return torch.tensor([w], dtype=torch.float32, device=DEVICE)

def make_loaders(x_train, y_train, x_val, y_val, batch_size):
    train_ds = TensorDataset(torch.tensor(x_train, dtype=torch.float32), torch.tensor(y_train, dtype=torch.float32))
    val_ds = TensorDataset(torch.tensor(x_val, dtype=torch.float32), torch.tensor(y_val, dtype=torch.float32))
    train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True, num_workers=0)
    val_loader = DataLoader(val_ds, batch_size=batch_size, shuffle=False, num_workers=0)
    return train_loader, val_loader

def train_deep_model(
    model,
    train_loader,
    val_loader,
    pos_weight,
    n_epochs,
    learning_rate,
    weight_decay,
    patience,
    device,
):
    model.to(device)
    optim = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
    criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
    best_auc = 0.0
    best_state = None
    bad_epochs = 0
    hist = {"val_auroc": []}
    for epoch in range(n_epochs):
        model.train()
        for xb, yb in train_loader:
            xb, yb = xb.to(device), yb.to(device)
            optim.zero_grad()
            logits = model(xb)
            loss = criterion(logits, yb)
            loss.backward()
            optim.step()
        model.eval()
        all_logits, all_y = [], []
        with torch.no_grad():
            for xb, yb in val_loader:
                xb = xb.to(device)
                logits = model(xb)
                all_logits.append(logits.cpu())
                all_y.append(yb)
        all_logits = torch.cat(all_logits).numpy()
        all_y = torch.cat(all_y).numpy()
        probs = 1 / (1 + np.exp(-all_logits))
        val_auc = roc_auc_score(all_y, probs)
        hist["val_auroc"].append(val_auc)
        print(f"[FTT][Epoch {epoch}] val AUROC={val_auc:.4f}", flush=True)
        if val_auc > best_auc:
            best_auc = val_auc
            best_state = model.state_dict()
            bad_epochs = 0
        else:
            bad_epochs += 1
            if bad_epochs >= patience:
                print(f"[FTT] Early stopping at epoch {epoch} best AUROC={best_auc:.4f}", flush=True)
                break
    if best_state:
        model.load_state_dict(best_state)
    return hist

def evaluate_model(model, x_test, batch_size=256):
    ds = TensorDataset(torch.tensor(x_test, dtype=torch.float32))
    loader = DataLoader(ds, batch_size=batch_size, shuffle=False, num_workers=0)
    model.eval()
    probs = []
    with torch.no_grad():
        for (xb,) in loader:
            xb = xb.to(DEVICE)
            logits = model(xb)
            probs.append(torch.sigmoid(logits).cpu())
    probs = torch.cat(probs).numpy()
    return probs

def run_fttransformer_optuna():
    x_train, x_val, x_test, y_train, y_val, y_test = load_splits()
    x_train = np.asarray(x_train)
    x_val = np.asarray(x_val)
    x_test = np.asarray(x_test)
    y_train = np.asarray(y_train).ravel()
    y_val = np.asarray(y_val).ravel()
    y_test = np.asarray(y_test).ravel()
    set_seeds(SEED)
    pos_weight = compute_pos_weight(y_train)

    def objective(trial):
        d_token = trial.suggest_categorical("d_token", [64, 128, 192, 256])
        n_layers = trial.suggest_int("n_layers", 1, 4)
        n_heads = trial.suggest_categorical("n_heads", [4, 8])
        d_ffn_factor = trial.suggest_float("d_ffn_factor", 1.0, 4.0)
        dropout = trial.suggest_float("dropout", 0.0, 0.5)
        lr = trial.suggest_float("lr", 1e-5, 1e-3, log=True)
        weight_decay = trial.suggest_float("weight_decay", 1e-6, 1e-3, log=True)
        batch_size = trial.suggest_categorical("batch_size", BATCH_SIZES)
        if d_token % n_heads != 0:
            return 0.0
        model = FTTransformer(
            n_cont_features=x_train.shape[1],
            d_token=d_token,
            n_layers=n_layers,
            n_heads=n_heads,
            d_ffn=int(d_token * d_ffn_factor),
            dropout=dropout,
            attention_dropout=dropout,
        )
        train_loader, val_loader = make_loaders(x_train, y_train, x_val, y_val, batch_size)
        hist = train_deep_model(
            model,
            train_loader,
            val_loader,
            pos_weight=pos_weight,
            n_epochs=40,
            learning_rate=lr,
            weight_decay=weight_decay,
            patience=8,
            device=DEVICE,
        )
        best_val_auc = max(hist["val_auroc"])
        print(f"[FTT][Trial {trial.number + 1}] best val AUROC={best_val_auc:.4f}", flush=True)
        return best_val_auc

    def log_best(study, trial):
        if study.best_trial:
            b = study.best_trial
            print(f"[FTT][Best so far] Trial {b.number} AUROC={b.value:.4f} params={b.params}", flush=True)

    study = optuna.create_study(direction="maximize", sampler=TPESampler(seed=SEED))
    study.optimize(objective, n_trials=N_TRIALS, timeout=TIMEOUT, show_progress_bar=True, callbacks=[log_best])
    print(f"\nBest AUROC (val): {study.best_value:.4f}")
    print(f"Best params: {study.best_params}")

    bp = study.best_params
    model = FTTransformer(
        n_cont_features=x_train.shape[1],
        d_token=bp["d_token"],
        n_layers=bp["n_layers"],
        n_heads=bp["n_heads"],
        d_ffn=int(bp["d_token"] * bp["d_ffn_factor"]),
        dropout=bp["dropout"],
        attention_dropout=bp["dropout"],
    )
    train_loader, val_loader = make_loaders(x_train, y_train, x_val, y_val, bp["batch_size"])
    history = train_deep_model(
        model,
        train_loader,
        val_loader,
        pos_weight=pos_weight,
        n_epochs=MAX_EPOCHS,
        learning_rate=bp["lr"],
        weight_decay=bp["weight_decay"],
        patience=PATIENCE,
        device=DEVICE,
    )
    print(f"[FTT] Final training done. Best val AUROC={max(history['val_auroc']):.4f}", flush=True)

    test_probs = evaluate_model(model, x_test)
    test_pred = (test_probs >= 0.5).astype(int)
    print("\nFT-Transformer Test Metrics:")
    print(f"AUROC: {roc_auc_score(y_test, test_probs):.4f}")
    print(f"AUPRC: {average_precision_score(y_test, test_probs):.4f}")
    print(f"Accuracy: {accuracy_score(y_test, test_pred):.4f}")
    print(f"F1: {f1_score(y_test, test_pred):.4f}")

if __name__ == "__main__":
    run_fttransformer_optuna()

In [ ]:
from __future__ import annotations

from pathlib import Path
import pickle
import importlib.util
import sys
import numpy as np
import torch
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import (
    roc_auc_score,
    average_precision_score,
    accuracy_score,
    f1_score,
    brier_score_loss,
)

BASE = Path("/kaggle/working")
DATA_DIR = BASE / "processed_data_final"
MODELS_DIR = BASE / "models"
device = "cuda" if torch.cuda.is_available() else "cpu"
N_BOOTSTRAPS = 1000


def load_cell_models():
    search_paths = [
        BASE / "cell_models.py",
        Path.cwd() / "cell_models.py",
        DATA_DIR.parent / "cell_models.py",
    ]
    for path in search_paths:
        if path.exists():
            spec = importlib.util.spec_from_file_location("cell_models", path)
            if spec and spec.loader:
                try:
                    module = importlib.util.module_from_spec(spec)
                    sys.modules["cell_models"] = module
                    spec.loader.exec_module(module)
                    for name in ("TabNetWrapper", "EnhancedMLP", "SAINTModel", "FTTransformer", "ClinicalResNet"):
                        if hasattr(module, name):
                            globals()[name] = getattr(module, name)
                    return True
                except Exception:
                    pass
    print("cell_models.py not found; PyTorch/TabNet classes must already be in scope.")
    return False


load_cell_models()


def compute_metrics_with_ci(y_true, y_pred, y_proba, n_bootstraps=1000, random_state=42):
    rng = np.random.RandomState(random_state)
    n = len(y_true)

    def boot(fn, use_proba=False):
        scores = []
        for _ in range(n_bootstraps):
            idx = rng.choice(n, n, replace=True)
            try:
                scores.append(fn(y_true[idx], (y_proba if use_proba else y_pred)[idx]))
            except Exception:
                continue
        scores = np.array(scores)
        point = fn(y_true, y_proba if use_proba else y_pred)
        lo, hi = np.percentile(scores, [2.5, 97.5])
        return (point, lo, hi)

    return {
        "auroc": boot(roc_auc_score, use_proba=True),
        "auprc": boot(average_precision_score, use_proba=True),
        "accuracy": boot(accuracy_score),
        "f1": boot(f1_score),
        "brier": boot(brier_score_loss, use_proba=True),
    }


def print_metrics_with_ci(metrics, title="Metrics"):
    print(f"\n{title}:")
    for k, (m, lo, hi) in metrics.items():
        print(f"{k:10s}: {m:.4f} (95% CI: {lo:.4f}-{hi:.4f})")


def eval_torch_model(model, x):
    ds = TensorDataset(torch.tensor(x, dtype=torch.float32))
    loader = DataLoader(ds, batch_size=256, shuffle=False)
    model.to(device).eval()
    out = []
    with torch.no_grad():
        for (xb,) in loader:
            xb = xb.to(device)
            out.append(torch.sigmoid(model(xb)).cpu())
    return torch.cat(out).numpy().squeeze()


def build_mlp_from_state(state_dict, input_dim):
    w_keys = [k for k in state_dict if k.endswith("weight") and "model" in k and state_dict[k].dim() == 2]
    w_keys = sorted(w_keys, key=lambda k: int(k.split(".")[1]))
    sizes = []
    for k in w_keys:
        out_f, in_f = state_dict[k].shape
        sizes.append((in_f, out_f))
    if len(sizes) < 2:
        raise ValueError("Cannot infer MLP shapes from checkpoint.")
    h1 = sizes[0][1]
    h2 = sizes[1][1]
    h3 = sizes[2][1] if len(sizes) > 2 else sizes[1][1] // 2
    import torch.nn as nn
    mlp = nn.Sequential(
        nn.Linear(input_dim, h1),
        nn.BatchNorm1d(h1),
        nn.ReLU(),
        nn.Linear(h1, h2),
        nn.BatchNorm1d(h2),
        nn.ReLU(),
        nn.Linear(h2, h3),
        nn.BatchNorm1d(h3),
        nn.ReLU(),
        nn.Linear(h3, 1),
    )
    return mlp


with open(DATA_DIR / "preprocessed_data.pkl", "rb") as f:
    data = pickle.load(f)


def grab(keys):
    for k in keys:
        if k in data:
            return data[k]
    raise KeyError(keys)


X_test = np.asarray(grab(["X_test_scaled", "X_test"]))
y_test = np.asarray(grab(["y_test"])).ravel()

X_val = np.asarray(grab(["X_val_scaled", "X_val"]))
y_val = np.asarray(grab(["y_val"])).ravel()

test_loader = DataLoader(TensorDataset(torch.tensor(X_test, dtype=torch.float32)), batch_size=256, shuffle=False)
val_loader = DataLoader(TensorDataset(torch.tensor(X_val, dtype=torch.float32)), batch_size=256, shuffle=False)

baseline_results = {}

sk_models = {
    "Logistic Regression": MODELS_DIR / "logistic_regression.pkl",
    "Random Forest": MODELS_DIR / "random_forest.pkl",
    "XGBoost": MODELS_DIR / "xgboost_model.pkl",
    "LightGBM": MODELS_DIR / "lightgbm_model.pkl",
}
for name, path in sk_models.items():
    if not path.exists():
        continue
    with open(path, "rb") as f:
        model = pickle.load(f)
    proba = model.predict_proba(X_test)[:, 1]
    pred = (proba >= 0.5).astype(int)
    metrics = compute_metrics_with_ci(y_test, pred, proba, n_bootstraps=N_BOOTSTRAPS)
    print_metrics_with_ci(metrics, f"{name} Test Results")
    baseline_results[name] = {"model": model, "predictions": pred, "probabilities": proba, "metrics": metrics}

tabnet_path = MODELS_DIR / "tabnet_model.pkl"
if tabnet_path.exists() and "TabNetWrapper" in globals():
    with open(tabnet_path, "rb") as f:
        tabnet_model = pickle.load(f)
    proba = tabnet_model.predict_proba(X_test)[:, 1]
    pred = (proba >= 0.5).astype(int)
    metrics = compute_metrics_with_ci(y_test, pred, proba, n_bootstraps=N_BOOTSTRAPS)
    print_metrics_with_ci(metrics, "TabNet Test Results")
    baseline_results["TabNet"] = {"model": tabnet_model, "predictions": pred, "probabilities": proba, "metrics": metrics}
else:
    print("TabNet skipped (model file missing or TabNetWrapper not in scope)")

mlp_paths = [
    MODELS_DIR / "enhanced_mlp_optimized.pt",
    MODELS_DIR / "mlp_model.pt",
    MODELS_DIR / "mlp_study.pkl",
]
mlp_state_path = next((p for p in mlp_paths if p.exists()), None)
if mlp_state_path is not None:
    mlp_model = None
    try:
        if mlp_state_path.suffix == ".pkl":
            with open(mlp_state_path, "rb") as f:
                maybe_model = pickle.load(f)
            if hasattr(maybe_model, "predict_proba"):
                proba = maybe_model.predict_proba(X_test)[:, 1]
                pred = (proba >= 0.5).astype(int)
                metrics = compute_metrics_with_ci(y_test, pred, proba, n_bootstraps=N_BOOTSTRAPS)
                print_metrics_with_ci(metrics, "MLP (pkl) Test Results")
                baseline_results["Enhanced MLP"] = {"model": maybe_model, "predictions": pred, "probabilities": proba, "metrics": metrics}
            else:
                print(f"MLP skipped: pkl file not a model with predict_proba ({mlp_state_path})")
        else:
            state = torch.load(mlp_state_path, map_location=device, weights_only=False)
            if "EnhancedMLP" in globals():
                mlp_model = EnhancedMLP(input_dim=X_test.shape[1])
                try:
                    mlp_model.load_state_dict(state)
                except RuntimeError:
                    mlp_model.load_state_dict(state, strict=False)
            else:
                mlp_model = build_mlp_from_state(state, input_dim=X_test.shape[1])
                mlp_model.load_state_dict(state, strict=False)
            proba = eval_torch_model(mlp_model, X_test)
            pred = (proba >= 0.5).astype(int)
            metrics = compute_metrics_with_ci(y_test, pred, proba, n_bootstraps=N_BOOTSTRAPS)
            print_metrics_with_ci(metrics, "Enhanced MLP Test Results")
            baseline_results["Enhanced MLP"] = {"model": mlp_model, "predictions": pred, "probabilities": proba, "metrics": metrics}
    except Exception as e2:
        print(f"Enhanced MLP skipped: {e2}")
else:
    print("Enhanced MLP skipped (state file missing)")

saint_path = MODELS_DIR / "saint_model.pt"
if saint_path.exists() and "SAINTModel" in globals():
    saint_model = SAINTModel(input_dim=X_test.shape[1])
    saint_model.load_state_dict(torch.load(saint_path, map_location=device))
    proba = eval_torch_model(saint_model, X_test)
    pred = (proba >= 0.5).astype(int)
    metrics = compute_metrics_with_ci(y_test, pred, proba, n_bootstraps=N_BOOTSTRAPS)
    print_metrics_with_ci(metrics, "SAINT Test Results")
    baseline_results["SAINT"] = {"model": saint_model, "predictions": pred, "probabilities": proba, "metrics": metrics}
else:
    print("SAINT skipped (state file missing or class not in scope)")

ftt_path = MODELS_DIR / "fttransformer_model.pt"
if ftt_path.exists() and "FTTransformer" in globals():
    ftt_model = FTTransformer(
        n_cont_features=X_test.shape[1],
        d_token=192,
        n_layers=4,
        n_heads=4,
        d_ffn=int(192 * 1.02),
        dropout=0.408,
        attention_dropout=0.408,
    )
    ftt_model.load_state_dict(torch.load(ftt_path, map_location=device))
    proba = eval_torch_model(ftt_model, X_test)
    pred = (proba >= 0.5).astype(int)
    metrics = compute_metrics_with_ci(y_test, pred, proba, n_bootstraps=N_BOOTSTRAPS)
    print_metrics_with_ci(metrics, "FT-Transformer Test Results")
    baseline_results["FT-Transformer"] = {"model": ftt_model, "predictions": pred, "probabilities": proba, "metrics": metrics}
else:
    print("FT-Transformer skipped (state file missing or class not in scope)")

resnet_path = MODELS_DIR / "resnet_model_optimized.pt"
if resnet_path.exists() and "ClinicalResNet" in globals():
    state = torch.load(resnet_path, map_location=device)
    block_ids = []
    for k in state.keys():
        if k.startswith("blocks."):
            parts = k.split(".")
            if len(parts) > 1 and parts[1].isdigit():
                block_ids.append(int(parts[1]))
    num_blocks = max(block_ids) + 1 if block_ids else 2

    rn_model = ClinicalResNet(
        input_dim=X_test.shape[1],
        hidden_dim=128,
        num_blocks=num_blocks,
        dropout=0.4,
    )
    rn_model.load_state_dict(state, strict=False)
    proba = eval_torch_model(rn_model, X_test)
    pred = (proba >= 0.5).astype(int)
    metrics = compute_metrics_with_ci(y_test, pred, proba, n_bootstraps=N_BOOTSTRAPS)
    print_metrics_with_ci(metrics, "Clinical ResNet Test Results")
    baseline_results["ClinicalResNet"] = {"model": rn_model, "predictions": pred, "probabilities": proba, "metrics": metrics}
else:
    print("Clinical ResNet skipped (state file missing or class not in scope)")

In [ ]:
from sklearn.linear_model import LogisticRegression
import numpy as np, torch

def loader_to_array(loader):
    xs = []
    for batch in loader:
        xb = batch[0] if isinstance(batch, (list, tuple)) else batch
        xs.append(xb)
    return torch.cat(xs).cpu().numpy()

def torch_probs(model, loader):
    model.eval().to(device)
    out = []
    with torch.no_grad():
        for batch in loader:
            xb = batch[0] if isinstance(batch, (list, tuple)) else batch
            xb = xb.to(device)
            out.append(torch.sigmoid(model(xb)).cpu())
    return torch.cat(out).numpy().squeeze()

def get_model_probs(model_name, loader):
    model = baseline_models[model_name]
    if hasattr(model, "parameters"):
        return torch_probs(model, loader)
    else:
        X_data = loader_to_array(loader)
        return model.predict_proba(X_data)[:, 1]

sorted_models = sorted(
    baseline_results.items(),
    key=lambda item: item[1]["metrics"]["auroc"][0],
    reverse=True,
)
top_3_names = [name for name, _ in sorted_models[:3]]
print(f"Top 3 Models selected for Stacking: {top_3_names}")

val_meta_features = [get_model_probs(name, val_loader) for name in top_3_names]
test_meta_features = [get_model_probs(name, test_loader) for name in top_3_names]

X_val_meta = np.column_stack(val_meta_features)
X_test_meta = np.column_stack(test_meta_features)

meta_model = LogisticRegression(random_state=config.RANDOM_STATE)
meta_model.fit(X_val_meta, y_val)

print("\nEnsemble Weights:")
for name, weight in zip(top_3_names, meta_model.coef_[0]):
    print(f"  {name}: {weight:.4f}")

val_probs_ens = meta_model.predict_proba(X_val_meta)[:, 1]
test_probs_ens = meta_model.predict_proba(X_test_meta)[:, 1]

best_thresh_ens, best_f1_ens = get_optimal_threshold(y_val, val_probs_ens)
print(f"\nStacked Ensemble - Optimal Threshold: {best_thresh_ens:.4f}")

y_test_pred_ens = (test_probs_ens >= best_thresh_ens).astype(int)

metrics_ens = compute_metrics_with_ci(y_test, y_test_pred_ens, test_probs_ens, n_bootstraps=config.N_BOOTSTRAP)
print_metrics_with_ci(metrics_ens, "Stacked Ensemble Results")

baseline_results["StackedEnsemble"] = {
    "predictions": y_test_pred_ens,
    "probabilities": test_probs_ens,
    "metrics": metrics_ens,
    "optimal_threshold": best_thresh_ens,
    "constituent_models": top_3_names,
}

with open(config.MODELS_DIR / "stacking_meta_model.pkl", "wb") as f:
    pickle.dump(meta_model, f)

## Model Comparison and Statistical Testing


In [ ]:
model_names = list(baseline_results.keys())
n_models = len(model_names)
delong_results = []
for i in range(n_models):
    for j in range(i+1, n_models):
        model1_name = model_names[i]
        model2_name = model_names[j]
        proba1 = baseline_results[model1_name]['probabilities']
        proba2 = baseline_results[model2_name]['probabilities']
        z_score, p_value = delong_test(y_test, proba1, proba2)
        delong_results.append({
            'Model 1': model1_name,
            'Model 2': model2_name,
            'Z-Score': z_score,
            'P-Value': p_value,
            'Significant': 'Yes' if p_value < 0.05 else 'No'
        })
delong_df = pd.DataFrame(delong_results)
print(delong_df.to_string(index=False))

# Fix: Use Path directly
from pathlib import Path
RESULTS_DIR = Path("/kaggle/working/results")
delong_df.to_csv(RESULTS_DIR / "delong_test_results.csv", index=False)

In [ ]:
results_summary = []

for model_name, model_results in baseline_results.items():
    metrics = model_results['metrics']

    row = {
        'Model': model_name,
        'AUROC': f"{metrics['auroc'][0]:.4f} [{metrics['auroc'][1]:.4f}, {metrics['auroc'][2]:.4f}]",
        'AUPRC': f"{metrics['auprc'][0]:.4f} [{metrics['auprc'][1]:.4f}, {metrics['auprc'][2]:.4f}]",
        'Accuracy': f"{metrics['accuracy'][0]:.4f} [{metrics['accuracy'][1]:.4f}, {metrics['accuracy'][2]:.4f}]",
        'F1': f"{metrics['f1'][0]:.4f} [{metrics['f1'][1]:.4f}, {metrics['f1'][2]:.4f}]",
        'Brier': f"{metrics['brier'][0]:.4f} [{metrics['brier'][1]:.4f}, {metrics['brier'][2]:.4f}]"
    }

    results_summary.append(row)

results_df = pd.DataFrame(results_summary)

print(results_df.to_string(index=False))

results_df.to_csv(config.RESULTS_DIR / "baseline_results_summary.csv", index=False)

## Visualization: Model Performance Comparison


In [ ]:
fig, axes = plt.subplots(1, 2, figsize=config.FIG_SIZE_WIDE)

ax = axes[0]
for model_name, model_results in baseline_results.items():
    proba = model_results['probabilities']
    fpr, tpr, _ = roc_curve(y_test, proba)
    auroc = model_results['metrics']['auroc'][0]
    ax.plot(fpr, tpr, label=f"{model_name} (AUROC={auroc:.3f})", linewidth=2)

ax.plot([0, 1], [0, 1], 'k--', label='Random', linewidth=1)
ax.set_xlabel('False Positive Rate')
ax.set_ylabel('True Positive Rate')
ax.set_title('ROC Curves - All Models')
ax.legend(loc='lower right', fontsize=8)
ax.grid(True, alpha=0.3)

ax = axes[1]
for model_name, model_results in baseline_results.items():
    proba = model_results['probabilities']
    precision, recall, _ = precision_recall_curve(y_test, proba)
    auprc = model_results['metrics']['auprc'][0]
    ax.plot(recall, precision, label=f"{model_name} (AUPRC={auprc:.3f})", linewidth=2)

baseline_precision = y_test.mean()
ax.axhline(baseline_precision, color='k', linestyle='--', label=f'Baseline ({baseline_precision:.3f})', linewidth=1)
ax.set_xlabel('Recall')
ax.set_ylabel('Precision')
ax.set_title('Precision-Recall Curves - All Models')
ax.legend(loc='lower left', fontsize=8)
ax.grid(True, alpha=0.3)

save_figure(fig, 'model_comparison_curves')
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=config.FIG_SIZE_LARGE)

model_names_plot = list(baseline_results.keys())
auroc_values = [baseline_results[name]['metrics']['auroc'][0] for name in model_names_plot]
auroc_ci_lower = [baseline_results[name]['metrics']['auroc'][1] for name in model_names_plot]
auroc_ci_upper = [baseline_results[name]['metrics']['auroc'][2] for name in model_names_plot]

yerr_lower = [val - ci_low for val, ci_low in zip(auroc_values, auroc_ci_lower)]
yerr_upper = [ci_up - val for val, ci_up in zip(auroc_values, auroc_ci_upper)]

x_pos = np.arange(len(model_names_plot))
ax.bar(x_pos, auroc_values, yerr=[yerr_lower, yerr_upper],
       capsize=5, alpha=0.7, color='steelblue')
ax.set_xticks(x_pos)
ax.set_xticklabels(model_names_plot, rotation=45, ha='right')
ax.set_ylabel('AUROC')
ax.set_title('Model Performance Comparison (with 95% CI)')
ax.set_ylim([min(auroc_values) - 0.05, max(auroc_values) + 0.05])
ax.grid(True, alpha=0.3, axis='y')

save_figure(fig, 'model_auroc_comparison')
plt.show()

# PART 4: Calibration


## Calibration Utilities


In [ ]:
class TemperatureScaling:
    def __init__(self):
        self.temperature = None

    def fit(self, y_true: np.ndarray, y_pred_logits: np.ndarray):
        from scipy.optimize import minimize

        def neg_log_likelihood(temp):
            scaled_probs = 1 / (1 + np.exp(-y_pred_logits / temp))
            epsilon = 1e-15
            scaled_probs = np.clip(scaled_probs, epsilon, 1 - epsilon)
            loss = -np.mean(y_true * np.log(scaled_probs) + (1 - y_true) * np.log(1 - scaled_probs))
            return loss

        result = minimize(neg_log_likelihood, x0=1.0, bounds=[(0.01, 10.0)], method='L-BFGS-B')
        self.temperature = result.x[0]

    def transform(self, y_pred_logits: np.ndarray) -> np.ndarray:
        if self.temperature is None:
            raise ValueError("Must fit before transform")

        scaled_probs = 1 / (1 + np.exp(-y_pred_logits / self.temperature))
        return scaled_probs

In [ ]:
def compute_ece(y_true: np.ndarray, y_prob: np.ndarray, n_bins: int = 10) -> float:
    bin_boundaries = np.linspace(0, 1, n_bins + 1)
    bin_lowers = bin_boundaries[:-1]
    bin_uppers = bin_boundaries[1:]

    ece = 0.0
    for bin_lower, bin_upper in zip(bin_lowers, bin_uppers):
        in_bin = np.logical_and(y_prob > bin_lower, y_prob <= bin_upper)
        prop_in_bin = np.mean(in_bin)

        if prop_in_bin > 0:
            accuracy_in_bin = np.mean(y_true[in_bin])
            avg_confidence_in_bin = np.mean(y_prob[in_bin])
            ece += np.abs(accuracy_in_bin - avg_confidence_in_bin) * prop_in_bin

    return ece

def compute_mce(y_true: np.ndarray, y_prob: np.ndarray, n_bins: int = 10) -> float:
    bin_boundaries = np.linspace(0, 1, n_bins + 1)
    bin_lowers = bin_boundaries[:-1]
    bin_uppers = bin_boundaries[1:]

    max_ce = 0.0
    for bin_lower, bin_upper in zip(bin_lowers, bin_uppers):
        in_bin = np.logical_and(y_prob > bin_lower, y_prob <= bin_upper)

        if np.sum(in_bin) > 0:
            accuracy_in_bin = np.mean(y_true[in_bin])
            avg_confidence_in_bin = np.mean(y_prob[in_bin])
            ce = np.abs(accuracy_in_bin - avg_confidence_in_bin)
            max_ce = max(max_ce, ce)

    return max_ce

## Calibrate All Models


In [ ]:
import numpy as np, torch
from sklearn.calibration import IsotonicRegression
from sklearn.metrics import brier_score_loss
import pandas as pd, matplotlib.pyplot as plt, seaborn as sns

def loader_to_array(loader):
    xs = []
    for batch in loader:
        xb = batch[0] if isinstance(batch, (list, tuple)) else batch
        xs.append(xb)
    return torch.cat(xs).cpu().numpy()

def get_probs_robust(model, loader, is_torch):
    if is_torch:
        model.eval().to(device)
        out = []
        with torch.no_grad():
            for batch in loader:
                xb = batch[0] if isinstance(batch, (list, tuple)) else batch
                xb = xb.to(device)
                out.append(torch.sigmoid(model(xb)).cpu())
        return torch.cat(out).numpy().squeeze()
    else:
        X_data = loader_to_array(loader)
        return model.predict_proba(X_data)[:, 1]

calibrated_results = {}
calibration_stats = []
models_to_calibrate = list(baseline_results.keys())

for name in models_to_calibrate:
    if name == "StackedEnsemble":
        top_3 = baseline_results['StackedEnsemble']['constituent_models']
        val_metas = [get_probs_robust(baseline_models[m], val_loader, hasattr(baseline_models[m],'parameters')) for m in top_3]
        X_val_meta_re = np.column_stack(val_metas)
        val_probs_uncal = meta_model.predict_proba(X_val_meta_re)[:, 1]
        test_probs_uncal = baseline_results[name]['probabilities']
    else:
        model = baseline_models[name]
        is_torch = hasattr(model, 'parameters')
        val_probs_uncal = get_probs_robust(model, val_loader, is_torch)
        test_probs_uncal = baseline_results[name]['probabilities']

    iso_reg = IsotonicRegression(out_of_bounds='clip')
    iso_reg.fit(val_probs_uncal, y_val)

    test_probs_cal = iso_reg.predict(test_probs_uncal)

    ece_pre = compute_ece(y_test, test_probs_uncal)
    brier_pre = brier_score_loss(y_test, test_probs_uncal)

    ece_post = compute_ece(y_test, test_probs_cal)
    brier_post = brier_score_loss(y_test, test_probs_cal)

    print(f"[{name}] ECE: {ece_pre:.4f} -> {ece_post:.4f} | Brier: {brier_pre:.4f} -> {brier_post:.4f}")

    calibrated_results[name] = {
        'probs_uncal': test_probs_uncal,
        'probs_cal': test_probs_cal,
        'ece_post': ece_post,
        'brier_post': brier_post
    }
    calibration_stats.append({
        'Model': name,
        'ECE_Pre': ece_pre,
        'ECE_Post': ece_post,
        'Brier_Pre': brier_pre,
        'Brier_Post': brier_post,
        'Improvement_ECE': (ece_pre - ece_post) / ece_pre if ece_pre else 0.0
    })

## Calibration Visualization


In [ ]:
from sklearn.calibration import calibration_curve
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from pathlib import Path

RESULTS_DIR = Path("/kaggle/working/results")
FIGURES_DIR = Path("/kaggle/working/figures")

fig, axes = plt.subplots(1, 3, figsize=(20, 6))

# Plot 1: Reliability Diagram
ax1 = axes[0]
ax1.plot([0, 1], [0, 1], "k:", label="Perfectly Calibrated", linewidth=2)

plot_names = ['XGBoost', 'ClinicalResNet', 'StackedEnsemble']
colors = ['#1f77b4', '#ff7f0e', '#2ca02c']

for name, color in zip(plot_names, colors):
    if name in calibrated_results:
        probs_uncal = calibrated_results[name]['probs_uncal']
        probs_cal = calibrated_results[name]['probs_cal']
        
        prob_true, prob_pred = calibration_curve(y_test, probs_uncal, n_bins=10, strategy='uniform')
        ax1.plot(prob_pred, prob_true, linestyle="--", color=color, alpha=0.5, label=f"{name} (Uncal)")
        
        prob_true_cal, prob_pred_cal = calibration_curve(y_test, probs_cal, n_bins=10, strategy='uniform')
        ax1.plot(prob_pred_cal, prob_true_cal, marker='o', linestyle="-", color=color, linewidth=2,
                 label=f"{name} (Calibrated) ECE={calibrated_results[name]['ece_post']:.3f}")

ax1.set_xlabel("Mean Predicted Probability", fontsize=11)
ax1.set_ylabel("Fraction of Positives", fontsize=11)
ax1.set_title("Calibration Curves (Reliability Diagram)", fontsize=12)
ax1.legend(loc="lower right", fontsize=9)
ax1.grid(True, alpha=0.3)
ax1.set_xlim([0, 1])
ax1.set_ylim([0, 1])

# Plot 2: Probability Distribution
ax2 = axes[1]
if 'StackedEnsemble' in calibrated_results:
    probs_cal = calibrated_results['StackedEnsemble']['probs_cal']
    sns.histplot(probs_cal[y_test == 0], color="blue", alpha=0.5, label="Survivors", bins=20, ax=ax2)
    sns.histplot(probs_cal[y_test == 1], color="red", alpha=0.5, label="Deaths", bins=20, ax=ax2)
    ax2.set_xlabel("Calibrated Probability", fontsize=11)
    ax2.set_ylabel("Count", fontsize=11)
    ax2.set_title("Calibrated Probability Distribution (Stacked Ensemble)", fontsize=12)
    ax2.legend(fontsize=10)
    ax2.grid(True, alpha=0.3)

# Plot 3: Bin sample counts
ax3 = axes[2]
n_bins = 10
bins = np.linspace(0, 1, n_bins + 1)

if 'StackedEnsemble' in calibrated_results:
    probs_cal = calibrated_results['StackedEnsemble']['probs_cal']
    bin_counts = []
    bin_centers = []
    bin_accuracies = []
    bin_confidences = []
    
    for i in range(n_bins):
        mask = (probs_cal >= bins[i]) & (probs_cal < bins[i + 1])
        if i == n_bins - 1:
            mask = (probs_cal >= bins[i]) & (probs_cal <= bins[i + 1])
        
        count = mask.sum()
        bin_counts.append(count)
        bin_centers.append((bins[i] + bins[i + 1]) / 2)
        
        if count > 0:
            bin_accuracies.append(y_test[mask].mean())
            bin_confidences.append(probs_cal[mask].mean())
        else:
            bin_accuracies.append(0)
            bin_confidences.append(0)
    
    bars = ax3.bar(bin_centers, bin_counts, width=0.08, color='steelblue', edgecolor='black', alpha=0.7)
    
    for bar, count in zip(bars, bin_counts):
        if count > 0:
            ax3.annotate(f'{count}', xy=(bar.get_x() + bar.get_width() / 2, bar.get_height()),
                        ha='center', va='bottom', fontsize=8)
    
    ax3.set_xlabel("Probability Bin", fontsize=11)
    ax3.set_ylabel("Sample Count", fontsize=11)
    ax3.set_title("Samples per Calibration Bin (Stacked Ensemble)", fontsize=12)
    ax3.set_xticks(bin_centers)
    ax3.set_xticklabels([f'{bins[i]:.1f}-{bins[i+1]:.1f}' for i in range(n_bins)], rotation=45, ha='right', fontsize=8)
    ax3.grid(True, alpha=0.3, axis='y')
    ax3.text(0.02, 0.98, f'Total n={len(probs_cal)}', transform=ax3.transAxes, fontsize=9, verticalalignment='top')

plt.tight_layout()
plt.savefig(FIGURES_DIR / "calibration_plots_detailed.png", dpi=300, bbox_inches='tight')
plt.show()

# Print bin statistics
print("\nBin Statistics (Stacked Ensemble):")
print(f"{'Bin':<12} {'Count':>8} {'Accuracy':>10} {'Confidence':>12} {'|Error|':>10}")
print("-" * 55)
for i in range(n_bins):
    error = abs(bin_accuracies[i] - bin_confidences[i]) if bin_counts[i] > 0 else 0
    print(f"{bins[i]:.1f}-{bins[i+1]:.1f}    {bin_counts[i]:>8} {bin_accuracies[i]:>10.4f} {bin_confidences[i]:>12.4f} {error:>10.4f}")

df_cal = pd.DataFrame(calibration_stats).sort_values("Brier_Post")
print("\n" + df_cal.to_string(index=False))
df_cal.to_csv(RESULTS_DIR / "calibration_summary.csv", index=False)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
from sklearn.metrics import roc_curve, auc, precision_recall_curve, confusion_matrix, brier_score_loss
from sklearn.calibration import calibration_curve
import warnings
import os

warnings.filterwarnings('ignore')

plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
plt.rcParams['figure.dpi'] = 300
plt.rcParams['savefig.dpi'] = 300
plt.rcParams['font.size'] = 10
plt.rcParams['axes.labelsize'] = 11
plt.rcParams['axes.titlesize'] = 12
plt.rcParams['legend.fontsize'] = 9

os.makedirs(config.BASE_DIR / 'figures', exist_ok=True)

colors = {
    'Ensemble (Calibrated)': '#e74c3c',
    'Stacked Ensemble': '#e74c3c',
    'LightGBM': '#3498db',
    'XGBoost': '#2ecc71',
    'ClinicalResNet': '#9b59b6',
    'SAINT': '#9b59b6',
    'Enhanced MLP': '#f39c12',
    'Random Forest': '#1abc9c',
    'Logistic Regression': '#95a5a6'
}

fig, ax = plt.subplots(figsize=(10, 8))

available_models = [m for m in baseline_results.keys() if m in colors or m == 'Ensemble (Calibrated)']

for model_name in available_models:
    if model_name in baseline_results:
        y_proba = baseline_results[model_name]['probabilities']
        fpr, tpr, _ = roc_curve(y_test, y_proba)
        roc_auc = auc(fpr, tpr)

        auroc_metric = baseline_results[model_name]['metrics']['auroc']
        if isinstance(auroc_metric, tuple):
            auroc_mean = auroc_metric[0]
            label = f'{model_name} (AUC={auroc_mean:.3f})'
        else:
            label = f'{model_name} (AUC={roc_auc:.3f})'

        linewidth = 3 if 'Ensemble' in model_name else 2
        linestyle = '-' if 'Ensemble' in model_name else '--'

        c = colors.get(model_name, '#333333')

        ax.plot(fpr, tpr, color=c, lw=linewidth, linestyle=linestyle, label=label)

ax.plot([0, 1], [0, 1], 'k--', lw=1, label='Random Classifier (AUC=0.500)')

ax.set_xlim([0.0, 1.0])
ax.set_ylim([0.0, 1.05])
ax.set_xlabel('False Positive Rate (1 - Specificity)', fontweight='bold')
ax.set_ylabel('True Positive Rate (Sensitivity)', fontweight='bold')
ax.set_title('ROC Curves - All Models Comparison', fontweight='bold')
ax.legend(loc="lower right", frameon=True, shadow=True)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(config.BASE_DIR / 'figures' / 'roc_curves_all_models.png')
plt.close()

n_models = len(available_models)
n_cols = 3
n_rows = (n_models + n_cols - 1) // n_cols

fig, axes = plt.subplots(n_rows, n_cols, figsize=(5*n_cols, 4*n_rows))
axes = axes.flatten()

for idx, model_name in enumerate(available_models):
    if idx >= len(axes): break
    ax = axes[idx]

    y_proba = baseline_results[model_name]['probabilities']

    if model_name in calibrated_results:
        ece = calibrated_results[model_name]['ece_post']
        brier = calibrated_results[model_name]['brier_post']
        plot_proba = calibrated_results[model_name]['probs_cal']
        label_txt = 'Calibrated'
    else:
        ece = compute_ece(y_test, y_proba)
        brier = brier_score_loss(y_test, y_proba)
        plot_proba = y_proba
        label_txt = 'Uncalibrated'

    fraction_of_positives, mean_predicted_value = calibration_curve(
        y_test, plot_proba, n_bins=10, strategy='uniform'
    )

    ax.plot(mean_predicted_value, fraction_of_positives, 's-',
            color='#3498db', linewidth=2, markersize=8, label=label_txt)
    ax.plot([0, 1], [0, 1], 'k--', lw=1.5, label='Perfect')

    ax.set_xlabel('Mean Predicted Probability')
    ax.set_ylabel('Fraction of Positives')
    ax.set_title(f'{model_name}\nECE={ece:.4f}, Brier={brier:.4f}', fontsize=10)
    ax.legend(loc='upper left', fontsize=8)
    ax.grid(True, alpha=0.3)

for i in range(idx + 1, len(axes)):
    axes[i].axis('off')

plt.tight_layout()
plt.savefig(config.BASE_DIR / 'figures' / 'calibration_curves.png')
plt.close()

model_names = []
aurocs = []
auprcs = []
f1s = []

for name in available_models:
    if name in baseline_results:
        model_names.append(name)
        metrics = baseline_results[name]['metrics']

        aurocs.append(metrics['auroc'][0] if isinstance(metrics['auroc'], tuple) else metrics['auroc'])
        auprcs.append(metrics['auprc'][0] if isinstance(metrics['auprc'], tuple) else metrics['auprc'])
        f1s.append(metrics['f1'][0] if isinstance(metrics['f1'], tuple) else metrics['f1'])

fig, axes = plt.subplots(1, 3, figsize=(18, 6))

def plot_bar(ax, data, title, color):
    bars = ax.barh(model_names, data, color=color, edgecolor='black', alpha=0.7)
    ax.set_title(title, fontweight='bold')
    ax.grid(axis='x', alpha=0.3)
    for i, v in enumerate(data):
        ax.text(v + 0.005, i, f'{v:.4f}', va='center', fontweight='bold', fontsize=9)

plot_bar(axes[0], aurocs, 'Discrimination (AUROC)', '#3498db')
axes[0].axvline(x=0.87, color='red', linestyle='--', label='SOTA Target')
axes[0].legend()

plot_bar(axes[1], auprcs, 'Precision-Recall (AUPRC)', '#2ecc71')
plot_bar(axes[2], f1s, 'F1 Score (Optimized Threshold)', '#f39c12')

plt.tight_layout()
plt.savefig(config.BASE_DIR / 'figures' / 'model_performance_comparison.png')
plt.close()

fig, axes = plt.subplots(1, 2, figsize=(14, 6))
has_tree_model = False

if 'LightGBM' in baseline_models:
    ax = axes[0]
    lgbm_model = baseline_models['LightGBM']
    importances = lgbm_model.feature_importances_
    importances = importances / importances.sum()
    indices = np.argsort(importances)[-15:]

    ax.barh(range(15), importances[indices], color='#3498db', edgecolor='black')
    ax.set_yticks(range(15))
    ax.set_yticklabels([X_train_scaled.columns[i] for i in indices])
    ax.set_title('LightGBM - Top 15 Features')
    has_tree_model = True

if 'XGBoost' in baseline_models:
    ax = axes[1]
    xgb_model = baseline_models['XGBoost']
    importances = xgb_model.feature_importances_
    importances = importances / importances.sum()
    indices = np.argsort(importances)[-15:]

    ax.barh(range(15), importances[indices], color='#2ecc71', edgecolor='black')
    ax.set_yticks(range(15))
    ax.set_yticklabels([X_train_scaled.columns[i] for i in indices])
    ax.set_title('XGBoost - Top 15 Features')
    has_tree_model = True

if has_tree_model:
    plt.tight_layout()
    plt.savefig(config.BASE_DIR / 'figures' / 'feature_importance.png')
    plt.close()
else:
    plt.close()

print("All visualizations generated and saved to 'figures/' folder.")

In [ ]:
def compute_mce(y_true, y_prob, n_bins=10):
    bin_boundaries = np.linspace(0, 1, n_bins + 1)
    mce = 0.0
    for i in range(n_bins):
        bin_mask = (y_prob > bin_boundaries[i]) & (y_prob <= bin_boundaries[i+1])
        if np.sum(bin_mask) > 0:
            bin_acc = np.mean(y_true[bin_mask])
            bin_conf = np.mean(y_prob[bin_mask])
            abs_err = np.abs(bin_acc - bin_conf)
            if abs_err > mce:
                mce = abs_err
    return mce

cal_summary = []

for model_name, res in calibrated_results.items():
    probs_uncal = res['probs_uncal']
    probs_cal = res['probs_cal']

    ece_pre = compute_ece(y_test, probs_uncal)
    mce_pre = compute_mce(y_test, probs_uncal)
    brier_pre = brier_score_loss(y_test, probs_uncal)

    if 'ece_post' in res:
        ece_post = res['ece_post']
    else:
        ece_post = compute_ece(y_test, probs_cal)

    mce_post = compute_mce(y_test, probs_cal)

    if 'brier_post' in res:
        brier_post = res['brier_post']
    else:
        brier_post = brier_score_loss(y_test, probs_cal)

    cal_summary.append({
        'Model': model_name,
        'ECE Before': ece_pre,
        'ECE After': ece_post,
        'ECE Improvement': ece_pre - ece_post,
        'MCE Before': mce_pre,
        'MCE After': mce_post,
        'Brier Before': brier_pre,
        'Brier After': brier_post
    })

cal_df = pd.DataFrame(cal_summary)
cal_df = cal_df.sort_values('ECE After', ascending=True)

cal_df_display = cal_df.copy()
for col in cal_df_display.columns:
    if col != 'Model':
        cal_df_display[col] = cal_df_display[col].apply(lambda x: f"{x:.4f}")

print("Calibration Summary:")
print(cal_df_display.to_string(index=False))

best_ece_row = cal_df.loc[cal_df['ECE After'].idxmin()]
best_brier_row = cal_df.loc[cal_df['Brier After'].idxmin()]

print(f"\nLowest ECE:    {best_ece_row['Model']} ({best_ece_row['ECE After']:.4f})")
print(f"Lowest Brier: {best_brier_row['Model']} ({best_brier_row['Brier After']:.4f})")

cal_df.to_csv(config.RESULTS_DIR / "calibration_results.csv", index=False)
print(f"Calibration results saved to: {config.RESULTS_DIR / 'calibration_results.csv'}")

latex_table = cal_df.to_latex(
    index=False,
    float_format="%.4f",
    caption="Calibration metrics (ECE, MCE, Brier Score) before and after isotonic regression.",
    label="tab:calibration_results"
)

with open(config.RESULTS_DIR / "calibration_results.tex", 'w') as f:
    f.write(latex_table)
print(f"LaTeX table saved to: {config.RESULTS_DIR / 'calibration_results.tex'}")

# PART 5: Federated Learning

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
import copy
from typing import Dict, List, Tuple, Any, Optional
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.auto import tqdm
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)


print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")

## 5.2 Data Partitioning for Federated Learning


In [ ]:
import numpy as np
import pickle
from pathlib import Path
import matplotlib.pyplot as plt

enhanced_features_path = Path("/kaggle/input/databefore/processed_data_final/enhanced_features.pkl")
client_data_dir = Path("/kaggle/input/databefore/processed_data_final")

with open(enhanced_features_path, 'rb') as f:
    enhanced_data = pickle.load(f)

X_train_enhanced = enhanced_data['X_train']
X_val_enhanced = enhanced_data['X_val']
X_test_enhanced = enhanced_data['X_test']

y_train = enhanced_data['y_train']
y_val = enhanced_data['y_val']
y_test = enhanced_data['y_test']

if 'icu_types_train' in enhanced_data:
    icu_types_train = enhanced_data['icu_types_train']
    icu_types_val = enhanced_data['icu_types_val']
    icu_types_test = enhanced_data['icu_types_test']
else:
    icu_types_train = np.load(client_data_dir / "icu_types_train.npy", allow_pickle=True)
    icu_types_val = np.load(client_data_dir / "icu_types_val.npy", allow_pickle=True)
    icu_types_test = np.load(client_data_dir / "icu_types_test.npy", allow_pickle=True)

def to_numpy(obj):
    return obj.values if hasattr(obj, "values") else obj

X_train_np = to_numpy(X_train_enhanced)
X_val_np = to_numpy(X_val_enhanced)
X_test_np = to_numpy(X_test_enhanced)
y_train_np = to_numpy(y_train)
y_val_np = to_numpy(y_val)
y_test_np = to_numpy(y_test)

ICU_TYPES = sorted(np.unique(icu_types_train))

client_data = {}

for icu_type in ICU_TYPES:
    train_mask = (icu_types_train == icu_type)
    val_mask = (icu_types_val == icu_type)
    test_mask = (icu_types_test == icu_type)

    client_data[icu_type] = {
        'X_train': X_train_np[train_mask],
        'y_train': y_train_np[train_mask],
        'X_val': X_val_np[val_mask],
        'y_val': y_val_np[val_mask],
        'X_test': X_test_np[test_mask],
        'y_test': y_test_np[test_mask],
    }

    n_train = client_data[icu_type]['X_train'].shape[0]
    n_val = client_data[icu_type]['X_val'].shape[0]
    n_test = client_data[icu_type]['X_test'].shape[0]
    mortality = client_data[icu_type]['y_train'].mean() * 100
    n_features = client_data[icu_type]['X_train'].shape[1]

client_data = {
    cid: data for cid, data in client_data.items()
    if cid != "Mixed"
}

total_samples = sum(len(d['y_train']) for d in client_data.values())

icu_mortality_rates = {}
icu_sample_sizes = {}

for icu_type in client_data.keys():
    mortality = client_data[icu_type]['y_train'].mean()
    n_samples = len(client_data[icu_type]['y_train'])

    icu_mortality_rates[icu_type] = mortality
    icu_sample_sizes[icu_type] = n_samples

mortality_mean = np.mean(list(icu_mortality_rates.values()))
mortality_std = np.std(list(icu_mortality_rates.values()))
mortality_cv = mortality_std / mortality_mean

size_mean = np.mean(list(icu_sample_sizes.values()))
size_std = np.std(list(icu_sample_sizes.values()))
size_cv = size_std / size_mean

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

icu_names = list(icu_sample_sizes.keys())
sample_counts = list(icu_sample_sizes.values())

axes[0].bar(icu_names, sample_counts, color='steelblue', alpha=0.7)
axes[0].set_xlabel('ICU Type', fontsize=12)
axes[0].set_ylabel('Number of Training Samples', fontsize=12)
axes[0].set_title('FL Client Data Distribution (Non-IID) - Mixed Removed', fontsize=14, fontweight='bold')
axes[0].tick_params(axis='x', rotation=45)
axes[0].grid(axis='y', alpha=0.3)

mortality_rates = [v * 100 for v in icu_mortality_rates.values()]
colors = ['#2ecc71' if m < 10 else '#e74c3c' if m > 12 else '#f39c12' for m in mortality_rates]

axes[1].bar(icu_names, mortality_rates, color=colors, alpha=0.7)
axes[1].axhline(y=mortality_mean*100, color='black', linestyle='--', linewidth=2, label=f'Mean: {mortality_mean*100:.1f}%')
axes[1].set_xlabel('ICU Type', fontsize=12)
axes[1].set_ylabel('Mortality Rate (%)', fontsize=12)
axes[1].set_title('Mortality Rate by ICU Type (5 clients, Mixed removed)', fontsize=14, fontweight='bold')
axes[1].tick_params(axis='x', rotation=45)
axes[1].legend()
axes[1].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.savefig('/kaggle/working/fl_icu_partitioning_heterogeneity.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
import pickle
from pathlib import Path

save_path = Path("/kaggle/working/client_data_fl.pkl")
with open(save_path, 'wb') as f:
    pickle.dump(client_data, f)
print(f"Saved to: {save_path}")

## 5.3 Data Heterogeneity Analysis


In [ ]:
from pathlib import Path

figures_dir = Path("/kaggle/working/figures")
figures_dir.mkdir(exist_ok=True)

plt.savefig(figures_dir / 'fl_data_distribution.png', dpi=300, bbox_inches='tight')
plt.savefig(figures_dir / 'fl_data_distribution.pdf', bbox_inches='tight')
plt.show()

print(f"Data heterogeneity visualization saved to: {figures_dir}")

## 5.4 Federated Learning Algorithms


In [ ]:
import numpy as np
import torch
import torch.nn as nn
import copy
import warnings
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import roc_auc_score, f1_score, precision_recall_curve

warnings.filterwarnings('ignore')

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def find_best_threshold_f1(y_true, y_probs):
    precision, recall, thresholds = precision_recall_curve(y_true, y_probs)
    f1_scores = 2 * (precision * recall) / (precision + recall + 1e-10)
    if len(thresholds) > 0:
        idx = np.argmax(f1_scores)
        return thresholds[idx], f1_scores[idx]
    return 0.5, 0.0

class EnhancedMLP(nn.Module):
    def __init__(self, input_dim, hidden_dims=[451, 349, 132], dropout=0.45):
        super(EnhancedMLP, self).__init__()
        layers = []
        prev_dim = input_dim
        for h_dim in hidden_dims:
            layers.append(nn.Linear(prev_dim, h_dim))
            layers.append(nn.BatchNorm1d(h_dim))
            layers.append(nn.ReLU())
            layers.append(nn.Dropout(dropout))
            prev_dim = h_dim
        layers.append(nn.Linear(prev_dim, 1))
        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x).squeeze(-1)

class FLTrainer:
    def __init__(self, client_data, input_dim, device):
        self.client_data = client_data
        self.clients = list(client_data.keys())
        self.device = device

        self.global_model = EnhancedMLP(input_dim=input_dim).to(device)

        self.X_val_global = np.concatenate([d['X_val'] for d in client_data.values()])
        self.y_val_global = np.concatenate([d['y_val'] for d in client_data.values()])

        self.client_weights = {}
        for cid, data in client_data.items():
            neg = (data['y_train'] == 0).sum()
            pos = (data['y_train'] == 1).sum()
            ratio = min(neg / (pos + 1e-6), 20.0)
            self.client_weights[cid] = torch.tensor([ratio], dtype=torch.float32).to(device)

        self.client_bn_states = {cid: None for cid in self.clients}

    def _get_bn_keys(self, model):
        bn_keys = []
        for name, module in model.named_modules():
            if isinstance(module, nn.BatchNorm1d):
                bn_keys.append(f"{name}.weight")
                bn_keys.append(f"{name}.bias")
                bn_keys.append(f"{name}.running_mean")
                bn_keys.append(f"{name}.running_var")
                bn_keys.append(f"{name}.num_batches_tracked")
        return set(bn_keys)

    def train_algorithm(self, algo_name, rounds=20, local_epochs=3, lr=1e-4, mu=0.0, lam=0.0):
        if algo_name == 'ditto':
            self.personalized_models = {cid: copy.deepcopy(self.global_model) for cid in self.clients}

        history = {'auroc': [], 'f1': []}

        for r in range(rounds):
            local_weights = []
            local_sizes = []

            global_state_dict = copy.deepcopy(self.global_model.state_dict())
            bn_keys = set()
            if algo_name == 'fedbn':
                bn_keys = self._get_bn_keys(self.global_model)

            for cid in self.clients:
                data = self.client_data[cid]

                temp_model = copy.deepcopy(self.global_model)

                if algo_name == 'fedbn' and self.client_bn_states[cid] is not None:
                    current_state = temp_model.state_dict()
                    local_bn = self.client_bn_states[cid]
                    for k in bn_keys:
                        if k in local_bn:
                            current_state[k] = local_bn[k]
                    temp_model.load_state_dict(current_state)

                temp_model.train()
                optimizer = torch.optim.AdamW(temp_model.parameters(), lr=lr, weight_decay=1e-3)
                criterion = nn.BCEWithLogitsLoss(pos_weight=self.client_weights[cid])

                ds = TensorDataset(torch.tensor(data['X_train'], dtype=torch.float32),
                                   torch.tensor(data['y_train'], dtype=torch.float32))
                loader = DataLoader(ds, batch_size=64, shuffle=True)

                for _ in range(local_epochs):
                    for xb, yb in loader:
                        xb, yb = xb.to(self.device), yb.to(self.device)
                        optimizer.zero_grad()
                        logits = temp_model(xb)
                        loss = criterion(logits, yb)

                        if algo_name == 'fedprox':
                            prox = 0
                            for w, w_t in zip(temp_model.parameters(), self.global_model.parameters()):
                                prox += (w - w_t).norm(2)
                            loss += (mu / 2) * prox

                        loss.backward()
                        optimizer.step()

                if algo_name == 'fedbn':
                    st = temp_model.state_dict()
                    self.client_bn_states[cid] = {k: st[k].clone() for k in bn_keys if k in st}

                local_weights.append(copy.deepcopy(temp_model.state_dict()))
                local_sizes.append(len(data['y_train']))

                if algo_name == 'ditto':
                    p_model = self.personalized_models[cid]
                    p_model.train()
                    p_opt = torch.optim.AdamW(p_model.parameters(), lr=lr, weight_decay=1e-3)

                    for _ in range(local_epochs):
                        for xb, yb in loader:
                            xb, yb = xb.to(self.device), yb.to(self.device)
                            p_opt.zero_grad()
                            logits = p_model(xb)
                            loss = criterion(logits, yb)

                            ditto_term = 0
                            for w_p, w_g in zip(p_model.parameters(), self.global_model.parameters()):
                                ditto_term += (w_p - w_g).norm(2)

                            loss += (lam / 2) * ditto_term
                            loss.backward()
                            p_opt.step()

            new_state = copy.deepcopy(global_state_dict)
            total = sum(local_sizes)
            for key in new_state.keys():
                if algo_name == 'fedbn' and key in bn_keys: continue

                w_sum = torch.stack([local_weights[i][key].float() * local_sizes[i] for i in range(len(self.clients))], 0).sum(0)
                new_state[key] = w_sum / total
            self.global_model.load_state_dict(new_state)

            val_auc, val_f1 = 0, 0

            if algo_name in ['ditto', 'fedbn']:
                aucs, f1s = [], []
                for cid in self.clients:
                    if algo_name == 'ditto':
                        eval_m = self.personalized_models[cid]
                    else:
                        eval_m = copy.deepcopy(self.global_model)
                        if self.client_bn_states[cid]:
                            curr = eval_m.state_dict()
                            for k, v in self.client_bn_states[cid].items(): curr[k] = v
                            eval_m.load_state_dict(curr)

                    a, f = self._evaluate_metrics(eval_m, self.client_data[cid]['X_val'], self.client_data[cid]['y_val'])
                    weight = len(self.client_data[cid]['y_val']) / len(self.y_val_global)
                    aucs.append(a * weight)
                    f1s.append(f * weight)
                val_auc = sum(aucs)
                val_f1 = sum(f1s)
            else:
                val_auc, val_f1 = self._evaluate_metrics(self.global_model, self.X_val_global, self.y_val_global)

            history['auroc'].append(val_auc)
            history['f1'].append(val_f1)

            if (r+1) % 5 == 0:
                print(f"Round {r+1:2d} | Val AUROC: {val_auc:.4f} | Val F1: {val_f1:.4f}")

        return history

    def _evaluate_metrics(self, model, X, y):
        model.eval()
        ds = TensorDataset(torch.tensor(X, dtype=torch.float32))
        ld = DataLoader(ds, batch_size=256, shuffle=False)
        probs = []
        with torch.no_grad():
            for xb, in ld:
                probs.append(torch.sigmoid(model(xb.to(self.device))).cpu())

        y_probs = torch.cat(probs).numpy()

        try:
            auc = roc_auc_score(y, y_probs)
        except:
            auc = 0.5

        thresh, f1 = find_best_threshold_f1(y, y_probs)

        return auc, f1

In [ ]:
import copy
import warnings
from typing import Dict, List, Tuple

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.metrics import average_precision_score, f1_score, precision_recall_curve, roc_auc_score
from torch.utils.data import DataLoader, TensorDataset

warnings.filterwarnings("ignore")

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {DEVICE}")

OPTUNA_CONFIG = {
    "hidden_dims": [451, 349, 132],
    "dropout": 0.446,
    "lr": 9.6e-05,
    "weight_decay": 7.5e-04,
    "batch_size": 64,
}

class MortalityMLP(nn.Module):
    def __init__(self, input_dim: int, hidden_dims: List[int], dropout: float, use_bn: bool = False):
        super().__init__()
        self.use_bn = use_bn
        self.bns = nn.ModuleList()

        if use_bn:
            self.linears = nn.ModuleList()
            prev_dim = input_dim
            for hidden_dim in hidden_dims:
                self.linears.append(nn.Linear(prev_dim, hidden_dim))
                self.bns.append(nn.BatchNorm1d(hidden_dim))
                prev_dim = hidden_dim
            self.dropout = nn.Dropout(dropout)
        else:
            layers = []
            prev_dim = input_dim
            for hidden_dim in hidden_dims:
                layers.extend(
                    [
                        nn.Linear(prev_dim, hidden_dim),
                        nn.LayerNorm(hidden_dim),
                        nn.ReLU(inplace=True),
                        nn.Dropout(dropout),
                    ]
                )
                prev_dim = hidden_dim
            self.encoder = nn.Sequential(*layers)

        self.classifier = nn.Linear(hidden_dims[-1], 1)
        self._init_weights()

    def _init_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.kaiming_normal_(m.weight, mode="fan_out", nonlinearity="relu")
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        if self.use_bn:
            for linear, bn in zip(self.linears, self.bns):
                x = linear(x)
                x = bn(x)
                x = F.relu(x, inplace=True)
                x = self.dropout(x)
        else:
            x = self.encoder(x)
        return self.classifier(x)

def find_optimal_threshold(y_true: np.ndarray, y_probs: np.ndarray) -> Tuple[float, float]:
    precision, recall, thresholds = precision_recall_curve(y_true, y_probs)
    f1_scores = np.where((precision + recall) > 0, 2 * precision * recall / (precision + recall), 0)
    if len(thresholds) > 0:
        idx = np.argmax(f1_scores[:-1])
        return thresholds[idx], f1_scores[idx]
    return 0.5, 0.0

class BaseFLTrainer:
    def __init__(self, client_data: Dict[str, Dict], use_bn: bool = False):
        self.device = DEVICE
        self.client_data = client_data
        self.client_ids = list(client_data.keys())

        self.class_weights = {}
        for cid, data in client_data.items():
            n_pos = np.sum(data["y_train"] == 1)
            n_neg = np.sum(data["y_train"] == 0)
            self.class_weights[cid] = n_neg / (n_pos + 1e-8)

        input_dim = client_data[self.client_ids[0]]["X_train"].shape[1]
        self.global_model = MortalityMLP(
            input_dim,
            OPTUNA_CONFIG["hidden_dims"],
            OPTUNA_CONFIG["dropout"],
            use_bn,
        ).to(DEVICE)

        self.X_val = np.vstack([d["X_val"] for d in client_data.values()])
        self.y_val = np.concatenate([d["y_val"] for d in client_data.values()])
        self.X_test = np.vstack([d["X_test"] for d in client_data.values()])
        self.y_test = np.concatenate([d["y_test"] for d in client_data.values()])

        self.optimal_threshold = 0.5
        self.best_threshold = 0.5
        self.best_model_state = copy.deepcopy(self.global_model.state_dict())
        self.best_val_f1 = 0.0

        self.history = {"val_f1": [], "val_auroc": [], "val_auprc": []}

    def _get_probs(self, model: nn.Module, X: np.ndarray) -> np.ndarray:
        model.eval()
        with torch.no_grad():
            X_t = torch.FloatTensor(X).to(self.device)
            return torch.sigmoid(model(X_t)).cpu().numpy().flatten()

    def _evaluate(self, model: nn.Module, X: np.ndarray, y: np.ndarray, threshold: float = None) -> Dict[str, float]:
        probs = self._get_probs(model, X)
        threshold = self.optimal_threshold if threshold is None else threshold
        preds = (probs >= threshold).astype(int)
        return {
            "auroc": roc_auc_score(y, probs),
            "auprc": average_precision_score(y, probs),
            "f1": f1_score(y, preds),
        }

    def _update_threshold(self, model: nn.Module) -> Tuple[float, float, float]:
        probs = self._get_probs(model, self.X_val)
        self.optimal_threshold, val_f1 = find_optimal_threshold(self.y_val, probs)
        val_auroc = roc_auc_score(self.y_val, probs)
        val_auprc = average_precision_score(self.y_val, probs)
        return val_f1, val_auroc, val_auprc

    def _finalize_and_eval(self, eval_test: bool = True):
        self.global_model.load_state_dict(self.best_model_state)
        self.optimal_threshold = self.best_threshold
        if not eval_test:
            return None
        return self._evaluate(self.global_model, self.X_test, self.y_test)

class FedAvgTrainer(BaseFLTrainer):
    def __init__(self, client_data: Dict[str, Dict]):
        super().__init__(client_data, use_bn=False)
        self.name = "FedAvg"

    def _local_train(self, client_id: str, local_epochs: int):
        data = self.client_data[client_id]
        local_model = copy.deepcopy(self.global_model)
        local_model.train()

        pos_weight = torch.tensor([self.class_weights[client_id]], device=self.device)
        criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)

        X_t = torch.FloatTensor(data["X_train"]).to(self.device)
        y_t = torch.FloatTensor(data["y_train"]).to(self.device)
        loader = DataLoader(TensorDataset(X_t, y_t), batch_size=OPTUNA_CONFIG["batch_size"], shuffle=True)

        optimizer = torch.optim.AdamW(
            local_model.parameters(),
            lr=OPTUNA_CONFIG["lr"],
            weight_decay=OPTUNA_CONFIG["weight_decay"],
        )

        for _ in range(local_epochs):
            for batch_X, batch_y in loader:
                optimizer.zero_grad()
                loss = criterion(local_model(batch_X), batch_y.unsqueeze(1))
                loss.backward()
                torch.nn.utils.clip_grad_norm_(local_model.parameters(), 1.0)
                optimizer.step()

        return local_model.state_dict(), len(data["y_train"])

    def _aggregate(self, states, counts):
        total = sum(counts)
        weights = [c / total for c in counts]
        global_state = self.global_model.state_dict()
        for key in global_state:
            global_state[key] = sum(weights[i] * states[i][key].float() for i in range(len(states)))
        self.global_model.load_state_dict(global_state)

    def train(self, num_rounds: int = 30, local_epochs: int = 5, patience: int = 7, eval_test: bool = True):
        no_improve = 0
        for r in range(num_rounds):
            states, counts = [], []
            for cid in self.client_ids:
                s, n = self._local_train(cid, local_epochs)
                states.append(s)
                counts.append(n)
            self._aggregate(states, counts)

            val_f1, val_auroc, val_auprc = self._update_threshold(self.global_model)
            self.history["val_f1"].append(val_f1)
            self.history["val_auroc"].append(val_auroc)
            self.history["val_auprc"].append(val_auprc)

            if (r + 1) % 5 == 0:
                print(f"  Round {r+1}: Val F1={val_f1:.4f}, AUROC={val_auroc:.4f}, AUPRC={val_auprc:.4f}")

            if val_f1 > self.best_val_f1:
                self.best_val_f1 = val_f1
                self.best_model_state = copy.deepcopy(self.global_model.state_dict())
                self.best_threshold = self.optimal_threshold
                no_improve = 0
            else:
                no_improve += 1
                if no_improve >= patience:
                    print(f"  Early stopping at round {r+1}")
                    break

        test_metrics = self._finalize_and_eval(eval_test)
        return {"val_f1": self.best_val_f1, "test": test_metrics}

class FedProxTrainer(BaseFLTrainer):
    def __init__(self, client_data: Dict[str, Dict], mu: float = 0.01):
        super().__init__(client_data, use_bn=False)
        self.mu = mu
        self.name = f"FedProx(mu={mu})"

    def _local_train(self, client_id: str, local_epochs: int):
        data = self.client_data[client_id]
        local_model = copy.deepcopy(self.global_model)
        local_model.train()

        global_weights = {n: p.clone().detach() for n, p in self.global_model.named_parameters()}

        pos_weight = torch.tensor([self.class_weights[client_id]], device=self.device)
        criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)

        X_t = torch.FloatTensor(data["X_train"]).to(self.device)
        y_t = torch.FloatTensor(data["y_train"]).to(self.device)
        loader = DataLoader(TensorDataset(X_t, y_t), batch_size=OPTUNA_CONFIG["batch_size"], shuffle=True)

        optimizer = torch.optim.AdamW(
            local_model.parameters(),
            lr=OPTUNA_CONFIG["lr"],
            weight_decay=OPTUNA_CONFIG["weight_decay"],
        )

        for _ in range(local_epochs):
            for batch_X, batch_y in loader:
                optimizer.zero_grad()
                loss = criterion(local_model(batch_X), batch_y.unsqueeze(1))
                prox = sum(torch.norm(p - global_weights[n]) ** 2 for n, p in local_model.named_parameters())
                loss = loss + (self.mu / 2) * prox
                loss.backward()
                torch.nn.utils.clip_grad_norm_(local_model.parameters(), 1.0)
                optimizer.step()

        return local_model.state_dict(), len(data["y_train"])

    def _aggregate(self, states, counts):
        total = sum(counts)
        weights = [c / total for c in counts]
        global_state = self.global_model.state_dict()
        for key in global_state:
            global_state[key] = sum(weights[i] * states[i][key].float() for i in range(len(states)))
        self.global_model.load_state_dict(global_state)

    def train(self, num_rounds: int = 30, local_epochs: int = 5, patience: int = 7, eval_test: bool = True):
        no_improve = 0
        for r in range(num_rounds):
            states, counts = [], []
            for cid in self.client_ids:
                s, n = self._local_train(cid, local_epochs)
                states.append(s)
                counts.append(n)
            self._aggregate(states, counts)

            val_f1, val_auroc, val_auprc = self._update_threshold(self.global_model)
            self.history["val_f1"].append(val_f1)
            self.history["val_auroc"].append(val_auroc)
            self.history["val_auprc"].append(val_auprc)

            if (r + 1) % 5 == 0:
                print(f"  Round {r+1}: Val F1={val_f1:.4f}, AUROC={val_auroc:.4f}, AUPRC={val_auprc:.4f}")

            if val_f1 > self.best_val_f1:
                self.best_val_f1 = val_f1
                self.best_model_state = copy.deepcopy(self.global_model.state_dict())
                self.best_threshold = self.optimal_threshold
                no_improve = 0
            else:
                no_improve += 1
                if no_improve >= patience:
                    print(f"  Early stopping at round {r+1}")
                    break

        test_metrics = self._finalize_and_eval(eval_test)
        return {"val_f1": self.best_val_f1, "test": test_metrics}

class FedBNTrainer(BaseFLTrainer):
    def __init__(self, client_data: Dict[str, Dict]):
        super().__init__(client_data, use_bn=True)
        self.name = "FedBN"
        self.client_bn_states = {cid: None for cid in self.client_ids}

    def _local_train(self, client_id: str, local_epochs: int):
        data = self.client_data[client_id]
        local_model = copy.deepcopy(self.global_model)

        if self.client_bn_states[client_id] is not None:
            for i, bn in enumerate(local_model.bns):
                bn.load_state_dict(self.client_bn_states[client_id][i])

        local_model.train()

        pos_weight = torch.tensor([self.class_weights[client_id]], device=self.device)
        criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)

        X_t = torch.FloatTensor(data["X_train"]).to(self.device)
        y_t = torch.FloatTensor(data["y_train"]).to(self.device)
        loader = DataLoader(TensorDataset(X_t, y_t), batch_size=OPTUNA_CONFIG["batch_size"], shuffle=True)

        optimizer = torch.optim.AdamW(
            local_model.parameters(),
            lr=OPTUNA_CONFIG["lr"],
            weight_decay=OPTUNA_CONFIG["weight_decay"],
        )

        for _ in range(local_epochs):
            for batch_X, batch_y in loader:
                optimizer.zero_grad()
                loss = criterion(local_model(batch_X), batch_y.unsqueeze(1))
                loss.backward()
                torch.nn.utils.clip_grad_norm_(local_model.parameters(), 1.0)
                optimizer.step()

        self.client_bn_states[client_id] = [bn.state_dict() for bn in local_model.bns]
        return local_model.state_dict(), len(data["y_train"])

    def _aggregate(self, states, counts):
        total = sum(counts)
        weights = [c / total for c in counts]
        global_state = self.global_model.state_dict()

        bn_keys = set()
        for i in range(len(self.global_model.bns)):
            for k in self.global_model.bns[i].state_dict().keys():
                bn_keys.add(f"bns.{i}.{k}")

        for key in global_state:
            if key not in bn_keys:
                global_state[key] = sum(weights[i] * states[i][key].float() for i in range(len(states)))

        self.global_model.load_state_dict(global_state)

    def train(self, num_rounds: int = 30, local_epochs: int = 5, patience: int = 7, eval_test: bool = True):
        no_improve = 0
        for r in range(num_rounds):
            states, counts = [], []
            for cid in self.client_ids:
                s, n = self._local_train(cid, local_epochs)
                states.append(s)
                counts.append(n)
            self._aggregate(states, counts)

            val_f1, val_auroc, val_auprc = self._update_threshold(self.global_model)
            self.history["val_f1"].append(val_f1)
            self.history["val_auroc"].append(val_auroc)
            self.history["val_auprc"].append(val_auprc)

            if (r + 1) % 5 == 0:
                print(f"  Round {r+1}: Val F1={val_f1:.4f}, AUROC={val_auroc:.4f}, AUPRC={val_auprc:.4f}")

            if val_f1 > self.best_val_f1:
                self.best_val_f1 = val_f1
                self.best_model_state = copy.deepcopy(self.global_model.state_dict())
                self.best_threshold = self.optimal_threshold
                no_improve = 0
            else:
                no_improve += 1
                if no_improve >= patience:
                    print(f"  Early stopping at round {r+1}")
                    break

        test_metrics = self._finalize_and_eval(eval_test)
        return {"val_f1": self.best_val_f1, "test": test_metrics}

class DittoTrainer(BaseFLTrainer):
    def __init__(self, client_data: Dict[str, Dict], lambda_reg: float = 0.1):
        super().__init__(client_data, use_bn=False)
        self.lambda_reg = lambda_reg
        self.name = f"Ditto(lambda={lambda_reg})"
        self.personalized_models = {cid: copy.deepcopy(self.global_model) for cid in self.client_ids}
        self.best_personal_states = None

    def _local_train_global(self, client_id: str, local_epochs: int):
        data = self.client_data[client_id]
        local_model = copy.deepcopy(self.global_model)
        local_model.train()

        pos_weight = torch.tensor([self.class_weights[client_id]], device=self.device)
        criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)

        X_t = torch.FloatTensor(data["X_train"]).to(self.device)
        y_t = torch.FloatTensor(data["y_train"]).to(self.device)
        loader = DataLoader(TensorDataset(X_t, y_t), batch_size=OPTUNA_CONFIG["batch_size"], shuffle=True)

        optimizer = torch.optim.AdamW(
            local_model.parameters(),
            lr=OPTUNA_CONFIG["lr"],
            weight_decay=OPTUNA_CONFIG["weight_decay"],
        )

        for _ in range(local_epochs):
            for batch_X, batch_y in loader:
                optimizer.zero_grad()
                loss = criterion(local_model(batch_X), batch_y.unsqueeze(1))
                loss.backward()
                torch.nn.utils.clip_grad_norm_(local_model.parameters(), 1.0)
                optimizer.step()

        return local_model.state_dict(), len(data["y_train"])

    def _local_train_personalized(self, client_id: str, local_epochs: int):
        data = self.client_data[client_id]
        pers_model = self.personalized_models[client_id]
        pers_model.train()

        global_weights = {n: p.clone().detach() for n, p in self.global_model.named_parameters()}

        pos_weight = torch.tensor([self.class_weights[client_id]], device=self.device)
        criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)

        X_t = torch.FloatTensor(data["X_train"]).to(self.device)
        y_t = torch.FloatTensor(data["y_train"]).to(self.device)
        loader = DataLoader(TensorDataset(X_t, y_t), batch_size=OPTUNA_CONFIG["batch_size"], shuffle=True)

        optimizer = torch.optim.AdamW(
            pers_model.parameters(),
            lr=OPTUNA_CONFIG["lr"],
            weight_decay=OPTUNA_CONFIG["weight_decay"],
        )

        for _ in range(local_epochs):
            for batch_X, batch_y in loader:
                optimizer.zero_grad()
                loss = criterion(pers_model(batch_X), batch_y.unsqueeze(1))
                reg = sum(torch.norm(p - global_weights[n]) ** 2 for n, p in pers_model.named_parameters())
                loss = loss + (self.lambda_reg / 2) * reg
                loss.backward()
                torch.nn.utils.clip_grad_norm_(pers_model.parameters(), 1.0)
                optimizer.step()

    def _aggregate(self, states, counts):
        total = sum(counts)
        weights = [c / total for c in counts]
        global_state = self.global_model.state_dict()
        for key in global_state:
            global_state[key] = sum(weights[i] * states[i][key].float() for i in range(len(states)))
        self.global_model.load_state_dict(global_state)

    def _evaluate_personalized(self, dataset: str = "test"):
        all_probs, all_labels = [], []
        for cid in self.client_ids:
            data = self.client_data[cid]
            X = data["X_test"] if dataset == "test" else data["X_val"]
            y = data["y_test"] if dataset == "test" else data["y_val"]
            probs = self._get_probs(self.personalized_models[cid], X)
            all_probs.append(probs)
            all_labels.append(y)
        return np.concatenate(all_probs), np.concatenate(all_labels)

    def train(self, num_rounds: int = 30, local_epochs: int = 5, patience: int = 7, eval_test: bool = True):
        no_improve = 0
        for r in range(num_rounds):
            states, counts = [], []
            for cid in self.client_ids:
                s, n = self._local_train_global(cid, local_epochs)
                states.append(s)
                counts.append(n)
            self._aggregate(states, counts)

            for cid in self.client_ids:
                self._local_train_personalized(cid, local_epochs)

            val_probs, val_labels = self._evaluate_personalized("val")
            self.optimal_threshold, val_f1 = find_optimal_threshold(val_labels, val_probs)
            val_auroc = roc_auc_score(val_labels, val_probs)
            val_auprc = average_precision_score(val_labels, val_probs)

            self.history["val_f1"].append(val_f1)
            self.history["val_auroc"].append(val_auroc)
            self.history["val_auprc"].append(val_auprc)

            if (r + 1) % 5 == 0:
                print(f"  Round {r+1}: Val F1={val_f1:.4f}, AUROC={val_auroc:.4f}, AUPRC={val_auprc:.4f}")

            if val_f1 > self.best_val_f1:
                self.best_val_f1 = val_f1
                self.best_threshold = self.optimal_threshold
                self.best_model_state = copy.deepcopy(self.global_model.state_dict())
                self.best_personal_states = {cid: copy.deepcopy(model.state_dict()) for cid, model in self.personalized_models.items()}
                no_improve = 0
            else:
                no_improve += 1
                if no_improve >= patience:
                    print(f"  Early stopping at round {r+1}")
                    break

        if self.best_personal_states:
            for cid in self.client_ids:
                self.personalized_models[cid].load_state_dict(self.best_personal_states[cid])
        self.global_model.load_state_dict(self.best_model_state)
        self.optimal_threshold = self.best_threshold

        if not eval_test:
            return {"val_f1": self.best_val_f1, "test": None}

        test_probs, test_labels = self._evaluate_personalized("test")
        test_preds = (test_probs >= self.optimal_threshold).astype(int)
        m = {
            "auroc": roc_auc_score(test_labels, test_probs),
            "auprc": average_precision_score(test_labels, test_probs),
            "f1": f1_score(test_labels, test_preds),
        }
        return {"val_f1": self.best_val_f1, "test": m}

    def evaluate_per_client(self):
        print("\n  Per-client results:")
        for cid in self.client_ids:
            data = self.client_data[cid]
            probs = self._get_probs(self.personalized_models[cid], data["X_test"])
            preds = (probs >= self.optimal_threshold).astype(int)
            m = {
                "auroc": roc_auc_score(data["y_test"], probs),
                "auprc": average_precision_score(data["y_test"], probs),
                "f1": f1_score(data["y_test"], preds),
            }
            print(f"    {cid}: AUROC={m['auroc']:.4f}, AUPRC={m['auprc']:.4f}, F1={m['f1']:.4f}")

class SCAFFOLDTrainer(BaseFLTrainer):
    def __init__(self, client_data: Dict[str, Dict]):
        super().__init__(client_data, use_bn=False)
        self.name = "SCAFFOLD"
        self.c_global = {n: torch.zeros_like(p) for n, p in self.global_model.named_parameters()}
        self.c_local = {
            cid: {n: torch.zeros_like(p) for n, p in self.global_model.named_parameters()} for cid in self.client_ids
        }

    def _local_train(self, client_id: str, local_epochs: int):
        data = self.client_data[client_id]
        local_model = copy.deepcopy(self.global_model)
        local_model.train()

        pos_weight = torch.tensor([self.class_weights[client_id]], device=self.device)
        criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)

        X_t = torch.FloatTensor(data["X_train"]).to(self.device)
        y_t = torch.FloatTensor(data["y_train"]).to(self.device)
        loader = DataLoader(TensorDataset(X_t, y_t), batch_size=OPTUNA_CONFIG["batch_size"], shuffle=True)

        optimizer = torch.optim.AdamW(
            local_model.parameters(),
            lr=OPTUNA_CONFIG["lr"],
            weight_decay=OPTUNA_CONFIG["weight_decay"],
        )

        c_global = self.c_global
        old_c_local = {n: p.clone().detach() for n, p in self.c_local[client_id].items()}

        steps = 0
        for _ in range(local_epochs):
            for batch_X, batch_y in loader:
                optimizer.zero_grad()
                loss = criterion(local_model(batch_X), batch_y.unsqueeze(1))
                loss.backward()

                with torch.no_grad():
                    for n, p in local_model.named_parameters():
                        if p.grad is not None:
                            p.grad.data += c_global[n].to(self.device) - old_c_local[n].to(self.device)

                torch.nn.utils.clip_grad_norm_(local_model.parameters(), 1.0)
                optimizer.step()
                steps += 1

        new_c_local = {}
        with torch.no_grad():
            for n, p in local_model.named_parameters():
                global_p = dict(self.global_model.named_parameters())[n]
                new_c_local[n] = old_c_local[n] - c_global[n] + (global_p - p) / (steps * OPTUNA_CONFIG["lr"])

        delta_c = {n: new_c_local[n] - old_c_local[n] for n in new_c_local}
        self.c_local[client_id] = new_c_local

        return local_model.state_dict(), len(data["y_train"]), delta_c

    def _aggregate(self, states, counts, delta_cs):
        total = sum(counts)
        weights = [c / total for c in counts]

        global_state = self.global_model.state_dict()
        for key in global_state:
            global_state[key] = sum(weights[i] * states[i][key].float() for i in range(len(states)))
        self.global_model.load_state_dict(global_state)

        for n in self.c_global:
            self.c_global[n] = self.c_global[n] + sum(weights[i] * delta_cs[i][n] for i in range(len(delta_cs)))

    def train(self, num_rounds: int = 30, local_epochs: int = 5, patience: int = 7, eval_test: bool = True):
        no_improve = 0
        for r in range(num_rounds):
            states, counts, delta_cs = [], [], []
            for cid in self.client_ids:
                s, n, dc = self._local_train(cid, local_epochs)
                states.append(s)
                counts.append(n)
                delta_cs.append(dc)
            self._aggregate(states, counts, delta_cs)

            val_f1, val_auroc, val_auprc = self._update_threshold(self.global_model)
            self.history["val_f1"].append(val_f1)
            self.history["val_auroc"].append(val_auroc)
            self.history["val_auprc"].append(val_auprc)

            if (r + 1) % 5 == 0:
                print(f"  Round {r+1}: Val F1={val_f1:.4f}, AUROC={val_auroc:.4f}, AUPRC={val_auprc:.4f}")

            if val_f1 > self.best_val_f1:
                self.best_val_f1 = val_f1
                self.best_model_state = copy.deepcopy(self.global_model.state_dict())
                self.best_threshold = self.optimal_threshold
                no_improve = 0
            else:
                no_improve += 1
                if no_improve >= patience:
                    print(f"  Early stopping at round {r+1}")
                    break

        test_metrics = self._finalize_and_eval(eval_test)
        return {"val_f1": self.best_val_f1, "test": test_metrics}

if __name__ == "__main__":
    try:
        _ = client_data
    except NameError as exc:
        raise SystemExit("client_data must be defined before running this script.") from exc

    results = {}
    trainers = {}

    config = {
        "num_rounds": 30,
        "local_epochs": 5,
        "patience": 7,
    }

    trainers["FedAvg"] = FedAvgTrainer(client_data)
    results["FedAvg"] = trainers["FedAvg"].train(**config, eval_test=True)

    trainers["FedProx"] = FedProxTrainer(client_data, mu=0.01)
    results["FedProx"] = trainers["FedProx"].train(**config, eval_test=True)

    trainers["FedBN"] = FedBNTrainer(client_data)
    results["FedBN"] = trainers["FedBN"].train(**config, eval_test=True)

    trainers["SCAFFOLD"] = SCAFFOLDTrainer(client_data)
    results["SCAFFOLD"] = trainers["SCAFFOLD"].train(**config, eval_test=True)

    trainers["Ditto"] = DittoTrainer(client_data, lambda_reg=0.1)
    results["Ditto"] = trainers["Ditto"].train(**config, eval_test=True)

    print(f"{'Method':<20} {'Val_F1':>10} {'Test_AUROC':>12} {'Test_AUPRC':>12} {'Test_F1':>10}")
    for method, m in results.items():
        test = m["test"] or {"auroc": float("nan"), "auprc": float("nan"), "f1": float("nan")}
        print(f"{method:<20} {m['val_f1']:>10.4f} {test['auroc']:>12.4f} {test['auprc']:>12.4f} {test['f1']:>10.4f}")

    best_method = max(results.keys(), key=lambda x: results[x]["val_f1"])
    print(f"\nBest method by validation F1: {best_method}")

    if best_method == "Ditto":
        lambda_values = [0.01, 0.05, 0.1, 0.2, 0.5]
        val_scores = {}
        for lam in lambda_values:
            trainer = DittoTrainer(client_data, lambda_reg=lam)
            res = trainer.train(num_rounds=30, local_epochs=5, patience=7, eval_test=False)
            val_scores[lam] = res["val_f1"]
        best_lambda = max(val_scores.keys(), key=lambda x: val_scores[x])
        print(f"Best lambda (by val F1): {best_lambda}")

        final_trainer = DittoTrainer(client_data, lambda_reg=best_lambda)
        final_metrics = final_trainer.train(num_rounds=40, local_epochs=5, patience=10, eval_test=True)
        final_trainer.evaluate_per_client()
    elif best_method == "FedProx":
        mu_values = [0.001, 0.01, 0.05, 0.1]
        val_scores = {}
        for mu in mu_values:
            trainer = FedProxTrainer(client_data, mu=mu)
            res = trainer.train(num_rounds=30, local_epochs=5, patience=7, eval_test=False)
            val_scores[mu] = res["val_f1"]
        best_mu = max(val_scores.keys(), key=lambda x: val_scores[x])
        print(f"Best mu (by val F1): {best_mu}")

        final_trainer = FedProxTrainer(client_data, mu=best_mu)
        final_metrics = final_trainer.train(num_rounds=40, local_epochs=5, patience=10, eval_test=True)
    else:
        if best_method == "FedAvg":
            final_trainer = FedAvgTrainer(client_data)
        elif best_method == "FedBN":
            final_trainer = FedBNTrainer(client_data)
        elif best_method == "SCAFFOLD":
            final_trainer = SCAFFOLDTrainer(client_data)
        final_metrics = final_trainer.train(num_rounds=40, local_epochs=5, patience=10, eval_test=True)

    print(f"\nFinal test metrics for selected method ({best_method}): {final_metrics['test']}")

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pathlib import Path

FIGURES_DIR = Path("/kaggle/working/figures")
RESULTS_DIR = Path("/kaggle/working/results")

fl_results_data = {
    'FedAvg': {'Val_F1': 0.4722, 'Test_AUROC': 0.8508, 'Test_AUPRC': 0.4452, 'Test_F1': 0.4463},
    'FedProx': {'Val_F1': 0.4711, 'Test_AUROC': 0.8538, 'Test_AUPRC': 0.4470, 'Test_F1': 0.4495},
    'FedBN': {'Val_F1': 0.4451, 'Test_AUROC': 0.8231, 'Test_AUPRC': 0.3995, 'Test_F1': 0.4222},
    'SCAFFOLD': {'Val_F1': 0.3969, 'Test_AUROC': 0.7882, 'Test_AUPRC': 0.3071, 'Test_F1': 0.3762},
    'Ditto': {'Val_F1': 0.4879, 'Test_AUROC': 0.8655, 'Test_AUPRC': 0.4619, 'Test_F1': 0.4725},
}

per_client_results = {
    'CCU': {'AUROC': 0.8651, 'AUPRC': 0.5198, 'F1': 0.5047},
    'CVICU': {'AUROC': 0.8870, 'AUPRC': 0.2918, 'F1': 0.3248},
    'MICU': {'AUROC': 0.8283, 'AUPRC': 0.4769, 'F1': 0.4957},
    'Neuro': {'AUROC': 0.8565, 'AUPRC': 0.0892, 'F1': 0.1463},
    'SICU': {'AUROC': 0.8473, 'AUPRC': 0.4772, 'F1': 0.4587},
}

centralized_auroc = 0.8718

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

ax1 = axes[0, 0]
methods = list(fl_results_data.keys())
aurocs = [fl_results_data[m]['Test_AUROC'] for m in methods]
colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd']

bars = ax1.bar(methods, aurocs, color=colors, edgecolor='black', alpha=0.8)
ax1.axhline(y=centralized_auroc, color='red', linestyle='--', linewidth=2, label=f'Centralized ({centralized_auroc:.4f})')
ax1.set_ylabel('AUROC', fontsize=11)
ax1.set_title('FL Methods: Test AUROC Comparison', fontsize=12)
ax1.set_ylim([0.75, 0.90])
ax1.legend(loc='lower right')
ax1.grid(True, alpha=0.3, axis='y')

for bar, val in zip(bars, aurocs):
    ax1.annotate(f'{val:.4f}', xy=(bar.get_x() + bar.get_width()/2, bar.get_height()),
                ha='center', va='bottom', fontsize=9, fontweight='bold')

ax2 = axes[0, 1]
x = np.arange(len(methods))
width = 0.25

aurocs = [fl_results_data[m]['Test_AUROC'] for m in methods]
auprcs = [fl_results_data[m]['Test_AUPRC'] for m in methods]
f1s = [fl_results_data[m]['Test_F1'] for m in methods]

bars1 = ax2.bar(x - width, aurocs, width, label='AUROC', color='steelblue', edgecolor='black')
bars2 = ax2.bar(x, auprcs, width, label='AUPRC', color='coral', edgecolor='black')
bars3 = ax2.bar(x + width, f1s, width, label='F1', color='mediumseagreen', edgecolor='black')

ax2.set_ylabel('Score', fontsize=11)
ax2.set_title('FL Methods: All Test Metrics', fontsize=12)
ax2.set_xticks(x)
ax2.set_xticklabels(methods, rotation=45, ha='right')
ax2.legend(loc='lower right')
ax2.grid(True, alpha=0.3, axis='y')
ax2.set_ylim([0, 1.0])

ax3 = axes[1, 0]
clients = list(per_client_results.keys())
client_aurocs = [per_client_results[c]['AUROC'] for c in clients]
client_colors = ['#e41a1c', '#377eb8', '#4daf4a', '#984ea3', '#ff7f00']

bars = ax3.bar(clients, client_aurocs, color=client_colors, edgecolor='black', alpha=0.8)
ax3.axhline(y=0.8655, color='black', linestyle='--', linewidth=2, label='Ditto Global (0.8655)')
ax3.set_ylabel('AUROC', fontsize=11)
ax3.set_title('Ditto: Per-Client Test AUROC', fontsize=12)
ax3.set_ylim([0.75, 0.95])
ax3.legend(loc='lower right')
ax3.grid(True, alpha=0.3, axis='y')

for bar, val in zip(bars, client_aurocs):
    ax3.annotate(f'{val:.3f}', xy=(bar.get_x() + bar.get_width()/2, bar.get_height()),
                ha='center', va='bottom', fontsize=9)

ax4 = axes[1, 1]
client_auprcs = [per_client_results[c]['AUPRC'] for c in clients]

bars = ax4.bar(clients, client_auprcs, color=client_colors, edgecolor='black', alpha=0.8)
ax4.set_ylabel('AUPRC', fontsize=11)
ax4.set_title('Ditto: Per-Client Test AUPRC (Shows Data Heterogeneity)', fontsize=12)
ax4.set_ylim([0, 0.6])
ax4.grid(True, alpha=0.3, axis='y')

for bar, val in zip(bars, client_auprcs):
    ax4.annotate(f'{val:.3f}', xy=(bar.get_x() + bar.get_width()/2, bar.get_height()),
                ha='center', va='bottom', fontsize=9)

plt.tight_layout()
plt.savefig(FIGURES_DIR / "fl_training_results.png", dpi=300, bbox_inches='tight')
plt.show()

fl_df = pd.DataFrame([
    {'Method': m, **fl_results_data[m]} for m in methods
]).sort_values('Test_AUROC', ascending=False)

print(fl_df.to_string(index=False))

print(f"{'Method':<12} {'AUROC':>10} {'Gap vs Centralized':>20}")
for m in methods:
    gap = centralized_auroc - fl_results_data[m]['Test_AUROC']
    gap_pct = (gap / centralized_auroc) * 100
    print(f"{m:<12} {fl_results_data[m]['Test_AUROC']:>10.4f} {gap:>10.4f} ({gap_pct:>5.2f}%)")

client_df = pd.DataFrame([
    {'Client': c, **per_client_results[c]} for c in clients
]).sort_values('AUROC', ascending=False)

print(client_df.to_string(index=False))

fl_df.to_csv(RESULTS_DIR / "fl_training_results.csv", index=False)
client_df.to_csv(RESULTS_DIR / "fl_per_client_results.csv", index=False)

In [ ]:
from sklearn.calibration import calibration_curve
from sklearn.metrics import brier_score_loss
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pathlib import Path

RESULTS_DIR = Path("/kaggle/working/results")
FIGURES_DIR = Path("/kaggle/working/figures")

def compute_ece(y_true, y_prob, n_bins=10):
    y_true = np.asarray(y_true).ravel()
    y_prob = np.asarray(y_prob).ravel()
    bins = np.linspace(0, 1, n_bins + 1)
    ece = 0.0
    for i in range(n_bins):
        mask = (y_prob >= bins[i]) & (y_prob < bins[i + 1])
        if i == n_bins - 1:
            mask = (y_prob >= bins[i]) & (y_prob <= bins[i + 1])
        if mask.sum() > 0:
            bin_acc = y_true[mask].mean()
            bin_conf = y_prob[mask].mean()
            ece += mask.sum() * np.abs(bin_acc - bin_conf)
    return ece / len(y_true)

def compute_mce(y_true, y_prob, n_bins=10):
    y_true = np.asarray(y_true).ravel()
    y_prob = np.asarray(y_prob).ravel()
    bins = np.linspace(0, 1, n_bins + 1)
    mce = 0.0
    for i in range(n_bins):
        mask = (y_prob >= bins[i]) & (y_prob < bins[i + 1])
        if i == n_bins - 1:
            mask = (y_prob >= bins[i]) & (y_prob <= bins[i + 1])
        if mask.sum() > 0:
            bin_acc = y_true[mask].mean()
            bin_conf = y_prob[mask].mean()
            mce = max(mce, np.abs(bin_acc - bin_conf))
    return mce

class TemperatureScaling:
    def __init__(self):
        self.temperature = 1.0
    
    def fit(self, y_true, y_prob):
        from scipy.optimize import minimize_scalar
        y_true = np.asarray(y_true).ravel()
        y_prob = np.asarray(y_prob).ravel()
        y_prob = np.clip(y_prob, 1e-10, 1 - 1e-10)
        logits = np.log(y_prob / (1 - y_prob))
        
        def nll(T):
            scaled = 1 / (1 + np.exp(-logits / T))
            scaled = np.clip(scaled, 1e-10, 1 - 1e-10)
            return -np.mean(y_true * np.log(scaled) + (1 - y_true) * np.log(1 - scaled))
        
        result = minimize_scalar(nll, bounds=(0.1, 10.0), method='bounded')
        self.temperature = result.x
        return self
    
    def transform(self, y_prob):
        y_prob = np.asarray(y_prob).ravel()
        y_prob = np.clip(y_prob, 1e-10, 1 - 1e-10)
        logits = np.log(y_prob / (1 - y_prob))
        return 1 / (1 + np.exp(-logits / self.temperature))

fl_probs_test = {}
fl_probs_val = {}

X_val = trainers['FedAvg'].X_val
y_val = trainers['FedAvg'].y_val
X_test = trainers['FedAvg'].X_test
y_test = trainers['FedAvg'].y_test

for method_name, trainer in trainers.items():
    if method_name == 'Ditto':
        test_probs_list = []
        val_probs_list = []
        for cid in trainer.client_ids:
            data = trainer.client_data[cid]
            test_probs_list.append(trainer._get_probs(trainer.personalized_models[cid], data['X_test']))
            val_probs_list.append(trainer._get_probs(trainer.personalized_models[cid], data['X_val']))
        fl_probs_test[method_name] = np.concatenate(test_probs_list)
        fl_probs_val[method_name] = np.concatenate(val_probs_list)
    else:
        fl_probs_test[method_name] = trainer._get_probs(trainer.global_model, X_test)
        fl_probs_val[method_name] = trainer._get_probs(trainer.global_model, X_val)

fl_calibration_results = {}

print(f"{'Method':<12} {'ECE_Pre':>10} {'ECE_Post':>10} {'MCE_Pre':>10} {'MCE_Post':>10} {'Brier_Pre':>10} {'Brier_Post':>10} {'Temp':>8}")

for method_name in fl_probs_test.keys():
    y_prob_test = fl_probs_test[method_name]
    y_prob_val = fl_probs_val[method_name]
    
    ece_pre = compute_ece(y_test, y_prob_test)
    mce_pre = compute_mce(y_test, y_prob_test)
    brier_pre = brier_score_loss(y_test, y_prob_test)
    
    ts = TemperatureScaling()
    ts.fit(y_val, y_prob_val)
    y_prob_cal = ts.transform(y_prob_test)
    
    ece_post = compute_ece(y_test, y_prob_cal)
    mce_post = compute_mce(y_test, y_prob_cal)
    brier_post = brier_score_loss(y_test, y_prob_cal)
    
    fl_calibration_results[method_name] = {
        'probs_uncal': y_prob_test,
        'probs_cal': y_prob_cal,
        'ece_pre': ece_pre,
        'ece_post': ece_post,
        'mce_pre': mce_pre,
        'mce_post': mce_post,
        'brier_pre': brier_pre,
        'brier_post': brier_post,
        'temperature': ts.temperature
    }
    
    print(f"{method_name:<12} {ece_pre:>10.4f} {ece_post:>10.4f} {mce_pre:>10.4f} {mce_post:>10.4f} {brier_pre:>10.4f} {brier_post:>10.4f} {ts.temperature:>8.3f}")

fl_cal_df = pd.DataFrame([
    {
        'Method': name,
        'ECE_Pre': r['ece_pre'],
        'ECE_Post': r['ece_post'],
        'MCE_Pre': r['mce_pre'],
        'MCE_Post': r['mce_post'],
        'Brier_Pre': r['brier_pre'],
        'Brier_Post': r['brier_post'],
        'Temperature': r['temperature'],
    }
    for name, r in fl_calibration_results.items()
]).sort_values('Brier_Post')

print(fl_cal_df.to_string(index=False))
fl_cal_df.to_csv(RESULTS_DIR / "fl_calibration_results.csv", index=False)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

ax1 = axes[0]
ax1.plot([0, 1], [0, 1], "k:", label="Perfect", linewidth=2)

colors = {'FedAvg': '#1f77b4', 'FedProx': '#ff7f0e', 'FedBN': '#2ca02c', 'SCAFFOLD': '#d62728', 'Ditto': '#9467bd'}

for method_name, res in fl_calibration_results.items():
    color = colors.get(method_name, 'gray')
    prob_true, prob_pred = calibration_curve(y_test, res['probs_uncal'], n_bins=10, strategy='uniform')
    ax1.plot(prob_pred, prob_true, linestyle="--", color=color, alpha=0.4)
    prob_true_cal, prob_pred_cal = calibration_curve(y_test, res['probs_cal'], n_bins=10, strategy='uniform')
    ax1.plot(prob_pred_cal, prob_true_cal, marker='o', linestyle="-", color=color, linewidth=2,
             label=f"{method_name} (ECE={res['ece_post']:.3f})")

ax1.set_xlabel("Mean Predicted Probability", fontsize=11)
ax1.set_ylabel("Fraction of Positives", fontsize=11)
ax1.set_title("FL Methods: Calibration Curves (After Temp Scaling)", fontsize=12)
ax1.legend(loc="lower right", fontsize=9)
ax1.grid(True, alpha=0.3)
ax1.set_xlim([0, 1])
ax1.set_ylim([0, 1])

ax2 = axes[1]
methods = list(fl_calibration_results.keys())
ece_pre = [fl_calibration_results[m]['ece_pre'] for m in methods]
ece_post = [fl_calibration_results[m]['ece_post'] for m in methods]

x = np.arange(len(methods))
width = 0.35

bars1 = ax2.bar(x - width/2, ece_pre, width, label='Before', color='salmon', edgecolor='black')
bars2 = ax2.bar(x + width/2, ece_post, width, label='After', color='steelblue', edgecolor='black')

ax2.set_xlabel("FL Method", fontsize=11)
ax2.set_ylabel("ECE", fontsize=11)
ax2.set_title("ECE Before vs After Temperature Scaling", fontsize=12)
ax2.set_xticks(x)
ax2.set_xticklabels(methods, rotation=45, ha='right')
ax2.legend()
ax2.grid(True, alpha=0.3, axis='y')

for bar in bars1:
    ax2.annotate(f'{bar.get_height():.3f}', xy=(bar.get_x() + bar.get_width()/2, bar.get_height()),
                ha='center', va='bottom', fontsize=8)
for bar in bars2:
    ax2.annotate(f'{bar.get_height():.3f}', xy=(bar.get_x() + bar.get_width()/2, bar.get_height()),
                ha='center', va='bottom', fontsize=8)

plt.tight_layout()
plt.savefig(FIGURES_DIR / "fl_calibration_analysis.png", dpi=300, bbox_inches='tight')
plt.show()

print(f"{'Setting':<30} {'ECE':>10} {'Brier':>10}")

if 'calibrated_results' in dir() and 'StackedEnsemble' in calibrated_results:
    print(f"{'StackedEnsemble (Centralized)':<30} {calibrated_results['StackedEnsemble']['ece_post']:>10.4f} {calibrated_results['StackedEnsemble']['brier_post']:>10.4f}")

for method in ['Ditto', 'FedAvg', 'FedProx']:
    if method in fl_calibration_results:
        print(f"{method + ' (FL)':<30} {fl_calibration_results[method]['ece_post']:>10.4f} {fl_calibration_results[method]['brier_post']:>10.4f}")

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.calibration import calibration_curve
from pathlib import Path

FIGURES_DIR = Path("/kaggle/working/figures")

y_test_fl = trainers['FedAvg'].y_test

fig, axes = plt.subplots(1, 3, figsize=(18, 5))

ax1 = axes[0]
ax1.plot([0, 1], [0, 1], "k:", label="Perfect", linewidth=2)

fl_colors = {'Ditto': '#9467bd', 'FedAvg': '#1f77b4', 'FedBN': '#2ca02c'}
for method, color in fl_colors.items():
    if method in fl_calibration_results:
        probs = fl_calibration_results[method]['probs_cal']
        prob_true, prob_pred = calibration_curve(y_test_fl, probs, n_bins=10, strategy='uniform')
        ece = fl_calibration_results[method]['ece_post']
        ax1.plot(prob_pred, prob_true, '-o', color=color, linewidth=2, label=f'{method} (ECE={ece:.3f})')

ax1.plot([0.05, 0.95], [0.05, 0.95], 'g--', linewidth=2, alpha=0.7, label='Centralized (~Perfect)')

ax1.set_xlabel("Mean Predicted Probability", fontsize=11)
ax1.set_ylabel("Fraction of Positives", fontsize=11)
ax1.set_title("FL Methods: Calibration Curves", fontsize=12)
ax1.legend(loc="lower right", fontsize=9)
ax1.grid(True, alpha=0.3)
ax1.set_xlim([0, 1])
ax1.set_ylim([0, 1])

ax2 = axes[1]
methods = ['Centralized', 'FedBN', 'Ditto', 'FedAvg', 'FedProx', 'SCAFFOLD']
ece_values = [
    0.0050,
    fl_calibration_results['FedBN']['ece_post'],
    fl_calibration_results['Ditto']['ece_post'],
    fl_calibration_results['FedAvg']['ece_post'],
    fl_calibration_results['FedProx']['ece_post'],
    fl_calibration_results['SCAFFOLD']['ece_post'],
]
colors = ['green', '#2ca02c', '#9467bd', '#1f77b4', '#ff7f0e', '#d62728']

bars = ax2.bar(methods, ece_values, color=colors, edgecolor='black', alpha=0.8)
ax2.set_ylabel('ECE (After Calibration)', fontsize=11)
ax2.set_title('Calibration Quality: Centralized vs FL', fontsize=12)
ax2.set_xticklabels(methods, rotation=45, ha='right')
ax2.grid(True, alpha=0.3, axis='y')

for bar, val in zip(bars, ece_values):
    ax2.annotate(f'{val:.4f}', xy=(bar.get_x() + bar.get_width()/2, bar.get_height()),
                ha='center', va='bottom', fontsize=9)

ax3 = axes[2]

ax3.scatter([0.8718], [0.005], s=200, c='green', marker='*', label='Centralized', zorder=5, edgecolors='black')

fl_aurocs = {
    'FedAvg': 0.8508, 'FedProx': 0.8538, 'FedBN': 0.8231, 
    'SCAFFOLD': 0.7882, 'Ditto': 0.8655
}
fl_eces = {m: fl_calibration_results[m]['ece_post'] for m in fl_aurocs.keys()}

for method in fl_aurocs.keys():
    color = {'FedAvg': '#1f77b4', 'FedProx': '#ff7f0e', 'FedBN': '#2ca02c', 
             'SCAFFOLD': '#d62728', 'Ditto': '#9467bd'}[method]
    ax3.scatter([fl_aurocs[method]], [fl_eces[method]], s=150, c=color, 
                label=method, edgecolors='black', zorder=4)

ax3.set_xlabel('AUROC', fontsize=11)
ax3.set_ylabel('ECE (Lower is Better)', fontsize=11)
ax3.set_title('Discrimination vs Calibration Tradeoff', fontsize=12)
ax3.legend(loc='upper left', fontsize=9)
ax3.grid(True, alpha=0.3)
ax3.set_xlim([0.75, 0.90])
ax3.set_ylim([0, 0.35])

ax3.annotate('', xy=(0.88, 0.02), xytext=(0.80, 0.25),
            arrowprops=dict(arrowstyle='->', color='gray', lw=2))
ax3.text(0.82, 0.12, 'Better', fontsize=10, color='gray', rotation=-55)

plt.tight_layout()
plt.savefig(FIGURES_DIR / "centralized_vs_fl_calibration.png", dpi=300, bbox_inches='tight')
plt.show()

print(f"{'Method':<20} {'AUROC':>10} {'ECE':>10} {'Brier':>10}")
print(f"{'Centralized':<20} {0.8718:>10.4f} {0.0050:>10.4f} {0.0724:>10.4f}")
for method in ['Ditto', 'FedAvg', 'FedProx', 'FedBN', 'SCAFFOLD']:
    auroc = fl_aurocs[method]
    ece = fl_calibration_results[method]['ece_post']
    brier = fl_calibration_results[method]['brier_post']
    print(f"{method:<20} {auroc:>10.4f} {ece:>10.4f} {brier:>10.4f}")

print(f"AUROC Gap (Ditto vs Centralized): {0.8718 - 0.8655:.4f} ({(0.8718 - 0.8655)/0.8718*100:.1f}%)")
print(f"ECE Gap (Ditto vs Centralized):    {0.2251 - 0.0050:.4f} ({0.2251/0.0050:.0f}x worse)")
print("FL preserves discrimination but severely degrades calibration.")

# DP

In [ ]:
# Install required packages
!pip install -q opacus 


Gemini

In [ ]:
import os
import logging
import warnings
from typing import Dict, Any, List, Tuple

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset

from opacus import PrivacyEngine
from opacus.validators import ModuleValidator
from opacus.accountants.utils import get_noise_multiplier

from sklearn.metrics import (
    roc_auc_score,
    average_precision_score,
    brier_score_loss,
    precision_recall_curve,
)

# ============================================================
# CONFIGURATION
# ============================================================

warnings.filterwarnings("ignore")
logging.getLogger("opacus").setLevel(logging.ERROR)


class Config:
    # Algorithms to benchmark
    ALGORITHMS = ["FedBN", "FedProx", "Ditto", "FedAvg"]
    EPSILONS = [2.0, 4.0, 6.0, 8.0]
    SEEDS = [0, 1, 2]

    # Training hyperparameters
    BATCH_SIZE = 512
    LEARNING_RATE = 0.002
    N_ROUNDS = 40
    LOCAL_EPOCHS = 4
    FINE_TUNE_EPOCHS = 5

    # Algorithm hyperparameters
    FEDPROX_MU = 0.1
    DITTO_LAMBDA = 0.8

    # Privacy settings
    TARGET_DELTA = 1e-5
    MAX_GRAD_NORM = 1.2

    # Model architecture
    HIDDEN_DIMS = (256, 128, 64)
    DROPOUT = 0.2

    DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
    OUTPUT_FILE = "thesis_final_metrics_085_fixed.csv"


conf = Config()


# ============================================================
# UTILITIES
# ============================================================

def set_seed(seed: int) -> None:
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)


def compute_ece(probs: np.ndarray, labels: np.ndarray, n_bins: int = 10) -> float:
    bins = np.linspace(0.0, 1.0, n_bins + 1)
    ece = 0.0
    for i in range(n_bins):
        mask = (probs >= bins[i]) & (probs < bins[i + 1])
        if np.any(mask):
            bin_acc = labels[mask].mean()
            bin_conf = probs[mask].mean()
            ece += np.abs(bin_acc - bin_conf) * mask.mean()
    return float(ece)


def fit_temperature(val_logits: np.ndarray, val_labels: np.ndarray) -> float:
    # Simple 1D search for temperature using NLL
    val_logits = np.clip(val_logits, -20.0, 20.0)
    val_labels = val_labels.astype(np.float32)

    def nll(temp: float) -> float:
        scaled = val_logits / temp
        probs = 1.0 / (1.0 + np.exp(-scaled))
        probs = np.clip(probs, 1e-7, 1.0 - 1e-7)
        loss = -np.mean(
            val_labels * np.log(probs) + (1.0 - val_labels) * np.log(1.0 - probs)
        )
        return float(loss)

    best_t = 1.0
    best_loss = nll(best_t)
    for t in np.linspace(0.5, 3.0, 26):
        loss = nll(t)
        if loss < best_loss:
            best_loss = loss
            best_t = t
    return float(best_t)


def apply_temperature(logits: np.ndarray, temperature: float) -> np.ndarray:
    logits = np.clip(logits / temperature, -20.0, 20.0)
    probs = 1.0 / (1.0 + np.exp(-logits))
    return probs


# ============================================================
# MODEL
# ============================================================

class MortalityMLP(nn.Module):
    def __init__(self, input_dim: int) -> None:
        super().__init__()
        layers: List[nn.Module] = []
        in_dim = input_dim
        for h_dim in conf.HIDDEN_DIMS:
            layers.append(nn.Linear(in_dim, h_dim))
            layers.append(nn.GroupNorm(1, h_dim))
            layers.append(nn.ReLU())
            layers.append(nn.Dropout(conf.DROPOUT))
            in_dim = h_dim
        layers.append(nn.Linear(in_dim, 1))
        self.model = nn.Sequential(*layers)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.model(x)


# ============================================================
# FEDERATED MANAGER
# ============================================================

class FederatedManager:
    def __init__(self, client_data: Dict[str, Dict[str, np.ndarray]]) -> None:
        self.client_ids = list(client_data.keys())
        self.input_dim = client_data[self.client_ids[0]]["X_train"].shape[1]
        self.raw_data = client_data
        self.results: List[Dict[str, Any]] = []

    def _make_loader(
        self,
        X: np.ndarray,
        y: np.ndarray,
        shuffle: bool = True,
    ) -> DataLoader:
        ds = TensorDataset(
            torch.from_numpy(X.astype(np.float32)),
            torch.from_numpy(y.astype(np.float32)),
        )
        loader = DataLoader(
            ds,
            batch_size=conf.BATCH_SIZE,
            shuffle=shuffle,
            drop_last=False,
        )
        return loader

    def _init_client(
        self,
        data: Dict[str, np.ndarray],
        global_state: Dict[str, torch.Tensor],
        noise_mult: float,
    ) -> Dict[str, Any]:
        train_loader = self._make_loader(data["X_train"], data["y_train"], shuffle=True)

        model = MortalityMLP(self.input_dim)
        model = ModuleValidator.fix(model)
        model.to(conf.DEVICE)
        model.load_state_dict(global_state, strict=True)

        optimizer = torch.optim.AdamW(model.parameters(), lr=conf.LEARNING_RATE)

        privacy_engine = PrivacyEngine()
        model, optimizer, train_loader = privacy_engine.make_private(
            module=model,
            optimizer=optimizer,
            data_loader=train_loader,
            noise_multiplier=noise_mult,
            max_grad_norm=conf.MAX_GRAD_NORM,
        )

        val_X = torch.from_numpy(data["X_val"].astype(np.float32)).to(conf.DEVICE)
        val_y = data["y_val"].astype(np.float32)

        test_X = torch.from_numpy(data["X_test"].astype(np.float32)).to(conf.DEVICE)
        test_y = data["y_test"].astype(np.float32)

        return {
            "model": model,
            "opt": optimizer,
            "loader": train_loader,
            "privacy_engine": privacy_engine,
            "val_X": val_X,
            "val_y": val_y,
            "test_X": test_X,
            "test_y": test_y,
        }

    def _get_clean_state(self, model: nn.Module) -> Dict[str, torch.Tensor]:
        if hasattr(model, "_module"):
            return {k: v.detach().clone() for k, v in model._module.state_dict().items()}
        return {k: v.detach().clone() for k, v in model.state_dict().items()}

    def _load_safe(
        self,
        model: nn.Module,
        state: Dict[str, torch.Tensor],
        strict: bool = True,
    ) -> None:
        if hasattr(model, "_module"):
            model._module.load_state_dict(state, strict=strict)
        else:
            model.load_state_dict(state, strict=strict)

    def _build_param_ref(
        self,
        global_state: Dict[str, torch.Tensor],
    ) -> Dict[str, torch.Tensor]:
        ref_model = MortalityMLP(self.input_dim)
        ref_model = ModuleValidator.fix(ref_model)
        ref_model.to(conf.DEVICE)
        ref_model.load_state_dict(global_state, strict=True)

        ref: Dict[str, torch.Tensor] = {}
        for name, param in ref_model.named_parameters():
            ref[name] = param.detach().clone()
        return ref

    def _train_round(
        self,
        client: Dict[str, Any],
        global_state: Dict[str, torch.Tensor],
        algo: str,
    ) -> None:
        model = client["model"]
        model.train()
        criterion = nn.BCEWithLogitsLoss()
        optimizer = client["opt"]

        global_ref = None
        if algo == "FedProx":
            global_ref = self._build_param_ref(global_state)

        for _ in range(conf.LOCAL_EPOCHS):
            for X, y in client["loader"]:
                X = X.to(conf.DEVICE)
                y = y.to(conf.DEVICE)
                optimizer.zero_grad()

                logits = model(X).squeeze()
                loss = criterion(logits, y)

                if algo == "FedProx":
                    prox = 0.0
                    base_model = model._module if hasattr(model, "_module") else model
                    for name, param in base_model.named_parameters():
                        if name in global_ref:
                            prox = prox + (param - global_ref[name]).pow(2).sum()
                    loss = loss + (conf.FEDPROX_MU / 2.0) * prox

                loss.backward()
                optimizer.step()

    def _aggregate(
        self,
        updates: List[Tuple[Dict[str, torch.Tensor], int]],
        algo: str,
        global_state: Dict[str, torch.Tensor],
    ) -> Dict[str, torch.Tensor]:
        total_weight = float(sum(n for _, n in updates))
        keys = list(updates[0][0].keys())
        new_state: Dict[str, torch.Tensor] = {}

        for k in keys:
            if algo == "FedBN" and ("norm" in k or "bn" in k):
                new_state[k] = global_state[k]
                continue
            agg = None
            for state, n in updates:
                w = n / total_weight
                tensor = state[k].float() * w
                if agg is None:
                    agg = tensor
                else:
                    agg = agg + tensor
            new_state[k] = agg

        return new_state

    def _evaluate(
        self,
        clients: Dict[str, Dict[str, Any]],
    ) -> Dict[str, float]:
        all_val_logits: List[np.ndarray] = []
        all_val_labels: List[np.ndarray] = []
        all_test_logits: List[np.ndarray] = []
        all_test_labels: List[np.ndarray] = []
        eps_spent: List[float] = []

        for client in clients.values():
            model = client["model"]
            model.eval()
            eps_spent.append(client["privacy_engine"].get_epsilon(conf.TARGET_DELTA))

            with torch.no_grad():
                val_logits = model(client["val_X"]).squeeze().cpu().numpy()
                test_logits = model(client["test_X"]).squeeze().cpu().numpy()

            all_val_logits.append(val_logits)
            all_val_labels.append(client["val_y"])
            all_test_logits.append(test_logits)
            all_test_labels.append(client["test_y"])

        val_logits = np.concatenate(all_val_logits)
        val_labels = np.concatenate(all_val_labels)
        test_logits = np.concatenate(all_test_logits)
        test_labels = np.concatenate(all_test_labels)

        test_logits = np.clip(test_logits, -20.0, 20.0)
        test_probs = 1.0 / (1.0 + np.exp(-test_logits))

        if np.any(np.isnan(test_probs)):
            test_probs = np.nan_to_num(test_probs)

        auroc = roc_auc_score(test_labels, test_probs)
        auprc = average_precision_score(test_labels, test_probs)
        brier_pre = brier_score_loss(test_labels, test_probs)
        ece_pre = compute_ece(test_probs, test_labels)

        precision, recall, thresholds = precision_recall_curve(test_labels, test_probs)
        f1_scores = 2.0 * precision * recall / (precision + recall + 1e-10)
        best_f1 = float(np.max(f1_scores))

        temperature = fit_temperature(val_logits, val_labels)
        cal_probs = apply_temperature(test_logits, temperature)
        cal_probs = np.clip(cal_probs, 1e-7, 1.0 - 1e-7)

        brier_post = brier_score_loss(test_labels, cal_probs)
        ece_post = compute_ece(cal_probs, test_labels)

        eps_max = float(max(eps_spent)) if eps_spent else 0.0

        return {
            "AUROC": float(auroc),
            "AUPRC": float(auprc),
            "ECE_pre": float(ece_pre),
            "ECE_post": float(ece_post),
            "Brier_pre": float(brier_pre),
            "Brier_post": float(brier_post),
            "F1": best_f1,
            "Total_epsilon_spent": eps_max,
        }

    def run(self) -> pd.DataFrame:
        print("Starting DP-FL benchmark for thesis")
        print(f"Algorithms: {conf.ALGORITHMS}")
        print(f"Epsilons: {conf.EPSILONS}")
        print(f"Seeds: {conf.SEEDS}")
        print(f"Device: {conf.DEVICE}")

        for eps in conf.EPSILONS:
            max_sr = max(
                conf.BATCH_SIZE / len(d["X_train"]) for d in self.raw_data.values()
            )
            noise = get_noise_multiplier(
                target_epsilon=eps,
                target_delta=conf.TARGET_DELTA,
                sample_rate=max_sr,
                epochs=conf.N_ROUNDS * conf.LOCAL_EPOCHS,
            )

            print("\n" + "=" * 60)
            print(f"Epsilon={eps} | noise_multiplier={noise:.4f}")
            print("=" * 60)

            for algo in conf.ALGORITHMS:
                for seed in conf.SEEDS:
                    set_seed(seed)
                    print(f"Running {algo} | seed={seed}...")

                    base_model = MortalityMLP(self.input_dim)
                    base_model = ModuleValidator.fix(base_model)
                    base_model.to(conf.DEVICE)
                    global_state = base_model.state_dict()

                    clients: Dict[str, Dict[str, Any]] = {
                        cid: self._init_client(data, global_state, noise)
                        for cid, data in self.raw_data.items()
                    }

                    for r in range(conf.N_ROUNDS):
                        updates: List[Tuple[Dict[str, torch.Tensor], int]] = []

                        for cid, client in clients.items():
                            if algo == "FedBN":
                                local_state = self._get_clean_state(client["model"])
                                mixed_state: Dict[str, torch.Tensor] = {}
                                for k, v in global_state.items():
                                    if "norm" in k or "bn" in k:
                                        mixed_state[k] = local_state[k]
                                    else:
                                        mixed_state[k] = v
                                self._load_safe(client["model"], mixed_state, strict=True)
                            else:
                                self._load_safe(client["model"], global_state, strict=True)

                            self._train_round(client, global_state, algo)
                            local_clean = self._get_clean_state(client["model"])
                            local_clean_cpu = {k: v.cpu().clone() for k, v in local_clean.items()}

                            n_weight = len(self.raw_data[cid]["y_train"])
                            updates.append((local_clean_cpu, n_weight))

                        global_state = self._aggregate(updates, algo, global_state)

                    for cid, client in clients.items():
                        if algo != "FedBN":
                            self._load_safe(client["model"], global_state, strict=True)

                        client["model"].train()
                        opt = client["opt"]
                        crit = nn.BCEWithLogitsLoss()

                        if algo in ["Ditto", "FedProx"]:
                            param_ref = self._build_param_ref(global_state)
                        else:
                            param_ref = None

                        for _ in range(conf.FINE_TUNE_EPOCHS):
                            for X, y in client["loader"]:
                                X = X.to(conf.DEVICE)
                                y = y.to(conf.DEVICE)
                                opt.zero_grad()

                                logits = client["model"](X).squeeze()
                                loss = crit(logits, y)

                                if algo in ["Ditto", "FedProx"]:
                                    reg = 0.0
                                    base_model = (
                                        client["model"]._module
                                        if hasattr(client["model"], "_module")
                                        else client["model"]
                                    )
                                    for name, param in base_model.named_parameters():
                                        if name in param_ref:
                                            reg = reg + (
                                                param - param_ref[name]
                                            ).pow(2).sum()
                                    coef = (
                                        conf.DITTO_LAMBDA
                                        if algo == "Ditto"
                                        else conf.FEDPROX_MU
                                    )
                                    loss = loss + (coef / 2.0) * reg

                                loss.backward()
                                opt.step()

                    metrics = self._evaluate(clients)
                    print(
                        f"  AUROC={metrics['AUROC']:.4f}, "
                        f"AUPRC={metrics['AUPRC']:.4f}, "
                        f"ECE_pre={metrics['ECE_pre']:.4f}, "
                        f"ECE_post={metrics['ECE_post']:.4f}, "
                        f"Eps_spent={metrics['Total_epsilon_spent']:.4f}"
                    )

                    row = {
                        "Algorithm": algo,
                        "Epsilon": eps,
                        "Seed": seed,
                        **metrics,
                    }
                    self.results.append(row)

                    df = pd.DataFrame(self.results)
                    df.to_csv(conf.OUTPUT_FILE, index=False)

        df_final = pd.DataFrame(self.results)
        return df_final


# ============================================================
# MAIN
# ============================================================

if __name__ == "__main__":
    if "client_data" in globals():
        manager = FederatedManager(client_data)
        df_results = manager.run()
        print("\nSaved results to:", conf.OUTPUT_FILE)
    else:
        print("client_data not found in globals(). Load it before running this script.")


In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Configuration
CSV_PATH = "thesis_final_metrics_085_fixed.csv"
OUT_DIR = "figures_dp_fl"

def load_results(csv_path: str) -> pd.DataFrame:
    df = pd.read_csv(csv_path)
    # Ensure types
    df["Epsilon"] = df["Epsilon"].astype(float)
    df["Seed"] = df["Seed"].astype(int)
    return df

def setup_style() -> None:
    sns.set(style="whitegrid", context="talk")
    plt.rcParams["figure.figsize"] = (8, 5)
    plt.rcParams["axes.titlesize"] = 18
    plt.rcParams["axes.labelsize"] = 14
    plt.rcParams["legend.fontsize"] = 12
    plt.rcParams["xtick.labelsize"] = 12
    plt.rcParams["ytick.labelsize"] = 12

def fig1_auroc_vs_epsilon(df: pd.DataFrame, out_dir: str) -> None:
    methods = sorted(df["Algorithm"].unique())
    plt.figure()
    for algo in methods:
        sub = df[df["Algorithm"] == algo]
        agg = sub.groupby("Epsilon")["AUROC"].agg(["mean", "std"]).reset_index()
        plt.errorbar(
            agg["Epsilon"],
            agg["mean"],
            yerr=agg["std"],
            marker="o",
            linewidth=2,
            capsize=4,
            label=algo,
        )
    plt.xlabel("Privacy budget (ε)")
    plt.ylabel("AUROC")
    plt.title("Privacy–utility tradeoff (AUROC vs ε)")
    plt.xticks(sorted(df["Epsilon"].unique()))
    plt.ylim(0.8, 0.9)
    plt.legend(loc="lower right")
    plt.tight_layout()
    plt.savefig(os.path.join(out_dir, "fig1_auroc_vs_epsilon.png"), dpi=300)
    plt.savefig(os.path.join(out_dir, "fig1_auroc_vs_epsilon.pdf"))
    plt.close()

def fig2_auprc_vs_epsilon(df: pd.DataFrame, out_dir: str) -> None:
    methods = sorted(df["Algorithm"].unique())
    plt.figure()
    for algo in methods:
        sub = df[df["Algorithm"] == algo]
        agg = sub.groupby("Epsilon")["AUPRC"].agg(["mean", "std"]).reset_index()
        plt.errorbar(
            agg["Epsilon"],
            agg["mean"],
            yerr=agg["std"],
            marker="o",
            linewidth=2,
            capsize=4,
            label=algo,
        )
    plt.xlabel("Privacy budget (ε)")
    plt.ylabel("AUPRC")
    plt.title("Precision–recall performance (AUPRC vs ε)")
    plt.xticks(sorted(df["Epsilon"].unique()))
    plt.legend(loc="lower right")
    plt.tight_layout()
    plt.savefig(os.path.join(out_dir, "fig2_auprc_vs_epsilon.png"), dpi=300)
    plt.savefig(os.path.join(out_dir, "fig2_auprc_vs_epsilon.pdf"))
    plt.close()

def fig3_calibration_bars(df: pd.DataFrame, out_dir: str) -> None:
    # Aggregate ECE over seeds for each algorithm and epsilon
    agg = (
        df.groupby(["Algorithm", "Epsilon"])
        .agg({"ECE_pre": "mean", "ECE_post": "mean"})
        .reset_index()
    )
    agg_melt = agg.melt(
        id_vars=["Algorithm", "Epsilon"],
        value_vars=["ECE_pre", "ECE_post"],
        var_name="Phase",
        value_name="ECE",
    )
    plt.figure(figsize=(10, 6))
    sns.barplot(
        data=agg_melt,
        x="Epsilon",
        y="ECE",
        hue="Phase",
        ci=None,
    )
    plt.xlabel("Privacy budget (ε)")
    plt.ylabel("ECE")
    plt.title("Calibration error before and after temperature scaling")
    plt.legend(title="")
    plt.tight_layout()
    plt.savefig(os.path.join(out_dir, "fig3_ece_pre_post.png"), dpi=300)
    plt.savefig(os.path.join(out_dir, "fig3_ece_pre_post.pdf"))
    plt.close()

def fig4_auroc_heatmap(df: pd.DataFrame, out_dir: str) -> None:
    pivot = (
        df.groupby(["Algorithm", "Epsilon"])["AUROC"]
        .mean()
        .reset_index()
        .pivot(index="Algorithm", columns="Epsilon", values="AUROC")
    )
    plt.figure(figsize=(8, 4))
    sns.heatmap(
        pivot,
        annot=True,
        fmt=".3f",
        cmap="RdYlGn",
        vmin=0.80,
        vmax=0.88,
        cbar_kws={"label": "AUROC"},
    )
    plt.title("Mean AUROC by method and privacy budget")
    plt.xlabel("Privacy budget (ε)")
    plt.ylabel("Algorithm")
    plt.tight_layout()
    plt.savefig(os.path.join(out_dir, "fig4_auroc_heatmap.png"), dpi=300)
    plt.savefig(os.path.join(out_dir, "fig4_auroc_heatmap.pdf"))
    plt.close()

def run_significance_tests(df: pd.DataFrame) -> None:
    from scipy.stats import ttest_ind

    methods = sorted(df["Algorithm"].unique())
    epsilons = sorted(df["Epsilon"].unique())

    print("Pairwise AUROC comparisons per ε (Welch t-test, 3 seeds per method):")
    for eps in epsilons:
        sub = df[df["Epsilon"] == eps]
        print(f"\nEpsilon = {eps}")
        for i in range(len(methods)):
            for j in range(i + 1, len(methods)):
                m1 = methods[i]
                m2 = methods[j]
                x = sub[sub["Algorithm"] == m1]["AUROC"].values
                y = sub[sub["Algorithm"] == m2]["AUROC"].values
                if len(x) >= 2 and len(y) >= 2:
                    stat, p = ttest_ind(x, y, equal_var=False)
                    print(f"  {m1} vs {m2}: p = {p:.4f}")

    # Global test: ECE_pre vs ECE_post across all runs
    from scipy.stats import ttest_rel
    if "ECE_pre" in df.columns and "ECE_post" in df.columns:
        stat, p = ttest_rel(df["ECE_pre"].values, df["ECE_post"].values)
        print("\nGlobal paired t-test for ECE_pre vs ECE_post (all runs combined):")
        print(f"  p = {p:.4e}")

def main() -> None:
    if not os.path.exists(CSV_PATH):
        print(f"CSV file not found at {CSV_PATH}")
        return

    os.makedirs(OUT_DIR, exist_ok=True)
    setup_style()
    df = load_results(CSV_PATH)

    fig1_auroc_vs_epsilon(df, OUT_DIR)
    fig2_auprc_vs_epsilon(df, OUT_DIR)
    fig3_calibration_bars(df, OUT_DIR)
    fig4_auroc_heatmap(df, OUT_DIR)
    run_significance_tests(df)

    print(f"\nFigures saved in directory: {OUT_DIR}")

if __name__ == "__main__":
    main()
